# $\color{darkblue}{\text{Kalender, Smiley, Årstall, Måned og Ukenummer}}$

In [ ]:
import sys
print(sys.version)

In [ ]:
# Install necessary library https://www.webfx.com/tools/emoji-cheat-sheet/
# Import necessary libraries
# -*- coding: utf-8 -*-
import sys, unicodedata
print("Python:", sys.version)

try:
    import emoji
    HAS_EMOJI = True
    print("emoji:", emoji.__version__, "fra", emoji.__file__)
except ModuleNotFoundError:
    HAS_EMOJI = False
    print("emoji-pakken ikke funnet – installer i denne kjernen med %pip og restart ved behov.")

def get_flag_emoji(country_code: str) -> str:
    """Bygger flagg-emoji fra landkode (A–Z) ved å bruke Regional Indicator Symbols."""
    return ''.join(chr(0x1F1E6 + (ord(c.upper()) - ord('A'))) for c in country_code if c.isalpha())

# Norsk flagg
norwegian_flag = get_flag_emoji('NO')
print("Norsk flagg:", norwegian_flag)  # 🇳🇴

if HAS_EMOJI:
    # CLDR-koder (emoji v2.x). For "aliaser" som :smile:, bruk language='alias'.
    txt = emoji.emojize(f'Python er helt konge :grinning_face:, :grinning_face_with_smiling_eyes:, {norwegian_flag}')
    print(txt)
    print(emoji.demojize('Python er👍'))  # -> 'Python er :thumbs_up:'
    print(emoji.emojize("Python is fun :grinning_face:"))
    print(emoji.emojize(":smile:", language="alias"))  # alias-eksempel
else:
    # Fallback uten emoji-pakken
    print(f'Python er helt konge :grinning_face:, :grinning_face_with_smiling_eyes:, {norwegian_flag}')

# Direkte Unicode-eksempel (uavhengig av emoji-pakken)
print("Python er gøy \U0001F600")  # 😀

In [ ]:
import calendar
from datetime import date

def display_calendar():
    year = int(input("Skriv år: "))
    month = int(input("Skriv måned (1-12): "))

    cal = calendar.TextCalendar(calendar.MONDAY)
    
    # Print the month calendar
    month_calendar = cal.formatmonth(year, month)
    print(month_calendar)

    # Print week numbers
    print("Ukenumre:")
    month_weeks = cal.monthdayscalendar(year, month)
    for week in month_weeks:
        # Find the first day of the week that is not 0 (which means no day)
        first_day = next((day for day in week if day != 0), None)
        if first_day:
            week_number = date(year, month, first_day).isocalendar()[1]
            print(f"Uke {week_number}: {week}")

display_calendar()

# $\color{violet}{\text{Kapittel 1 - Grunnleggende regning}}$
Forklare og bruke prosent, prosentpoeng og vekstfaktor til modellering av praktiske situasjoner med digitale verktøy

In [ ]:
#1 Grunnleggende regning Addisjon, subtraksjon, multiplikasjon og divisjon

# Addisjon
addition_1 = 70 + 50
addition_2 = 700 + 500

# Subtraksjon
subtraction_1 = 120 - 50
subtraction_2 = 1200 - 500

# Multiplikasjon
multiplication_1 = 42.35 * 10
multiplication_2 = 348 * 1000

# Divisjon
division_1 = 225 / 100
division_2 = 7.75 / 1000

# Utskrift av resultatene
print(f"70 kr + 50 kr = {addition_1:.2f} kr")
print(f"700 kr + 500 kr = {addition_2:.0f} kr")
print(f"120 kr - 50 kr = {subtraction_1:.0f} kr")
print(f"1200 kr - 500 kr = {subtraction_2:.0f} kr")
print(f"42.35 * 10 = {multiplication_1:.1f}")
print(f"348 * 1000 = {multiplication_2:.0f}")
print(f"225 / 100 = {division_1:.2f}")
print(f"7.75 / 1000 = {division_2:.5f}")

In [ ]:
# 1 Grunnleggende regning: 1.5 Regne med brøk
import ipywidgets as widgets
from IPython.display import display, clear_output
from fractions import Fraction
import math
import re
import ast
import operator

# GUI-komponenter
num_terms = widgets.IntSlider(value=2, min=2, max=6, description="Antall ledd:")
confirm_button = widgets.Button(description="OK")
input_container = widgets.VBox()
calculate_button = widgets.Button(description="Beregn")
output = widgets.Output()

term_inputs = []
operator_inputs = []

# Evaluer komplekse uttrykk i brøker
def eval_expr(expr):
    expr = expr.replace('^', '**')
    expr = re.sub(r'√(\d+)', r'math.sqrt(\1)', expr)
    expr = expr.replace('sqrt', 'math.sqrt')

    ops = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.USub: operator.neg
    }

    def _eval(node):
        if isinstance(node, ast.Expression):
            return _eval(node.body)
        elif isinstance(node, ast.BinOp):
            left = _eval(node.left)
            right = _eval(node.right)
            res = ops[type(node.op)](left, right)
            return Fraction(str(res)).limit_denominator(10**6)
        elif isinstance(node, ast.UnaryOp):
            operand = _eval(node.operand)
            return ops[type(node.op)](operand)
        elif isinstance(node, ast.Call):
            if isinstance(node.func, ast.Attribute) and node.func.attr == 'sqrt':
                val = math.sqrt(float(_eval(node.args[0])))
                return Fraction(str(val)).limit_denominator(10**6)
            else:
                raise TypeError("Ugyldig funksjon")
        elif isinstance(node, ast.Constant):
            return Fraction(str(node.value))
        elif isinstance(node, ast.Num):
            return Fraction(str(node.n))
        else:
            raise TypeError(f"Ugyldig uttrykk: {node}")

    parsed = ast.parse(expr, mode='eval')
    return _eval(parsed.body)

# Opprett felt for ledd
def create_inputs(_):
    global term_inputs, operator_inputs
    term_inputs = []
    operator_inputs = []
    children = []

    for idx in range(num_terms.value):
        term_type = widgets.Dropdown(options=["Heltall", "Desimaltall", "Brøk", "Kompleks brøk"], description=f"Ledd {idx+1}:")
        sign = widgets.Dropdown(options=['+', '-'], description="Fortegn:")

        int_input = widgets.IntText(description="Heltall")
        float_input = widgets.FloatText(description="Desimal")
        br_num = widgets.IntText(description="Teller")
        br_den = widgets.IntText(description="Nevner", value=1)
        k_num_expr = widgets.Text(description="Telleruttrykk")
        k_den_expr = widgets.Text(description="Nevneruttrykk")

        container = widgets.VBox()

        # Lager separat funksjon for å unngå referanseproblem
        def make_updater(container_ref, type_widget, sign_widget, i_widget, f_widget, bn_widget, bd_widget, kn_widget, kd_widget):
            def update_term_fields(change=None):
                if type_widget.value == "Heltall":
                    container_ref.children = [sign_widget, type_widget, i_widget]
                elif type_widget.value == "Desimaltall":
                    container_ref.children = [sign_widget, type_widget, f_widget]
                elif type_widget.value == "Brøk":
                    container_ref.children = [sign_widget, type_widget, bn_widget, bd_widget]
                elif type_widget.value == "Kompleks brøk":
                    container_ref.children = [sign_widget, type_widget, kn_widget, kd_widget]
            return update_term_fields

        updater = make_updater(container, term_type, sign, int_input, float_input, br_num, br_den, k_num_expr, k_den_expr)
        term_type.observe(updater, names="value")
        updater()

        term_inputs.append((sign, term_type, int_input, float_input, br_num, br_den, k_num_expr, k_den_expr))
        children.append(container)

        if idx < num_terms.value - 1:
            op = widgets.Dropdown(options=['+', '-', '*', '/'], description=f"Operator {idx+1}:")
            operator_inputs.append(op)
            children.append(op)

    children.append(calculate_button)
    children.append(output)
    input_container.children = children

# Konverter et ledd til Fraction
def to_fraction(sign, typ, i, f, bn, bd, kn_expr, kd_expr):
    if typ == "Heltall":
        val = Fraction(i)
    elif typ == "Desimaltall":
        val = Fraction(str(f)).limit_denominator()
    elif typ == "Brøk":
        if bd == 0:
            raise ZeroDivisionError("Nevner kan ikke være 0")
        val = Fraction(bn, bd)
    elif typ == "Kompleks brøk":
        teller = eval_expr(kn_expr)
        nevner = eval_expr(kd_expr)
        if nevner == 0:
            raise ZeroDivisionError("Nevner kan ikke være 0")
        val = teller / nevner
    else:
        raise ValueError(f"Ukjent type: {typ}")
    return val if sign == '+' else -val

# Kalkuler hele uttrykket
def calculate(_):
    with output:
        output.clear_output()
        try:
            sign, typ, i, f, bn, bd, kn_expr, kd_expr = term_inputs[0]
            result = to_fraction(sign.value, typ.value, i.value, f.value, bn.value, bd.value, kn_expr.value, kd_expr.value)
            expr_str = f"({result})"

            for idx, op_widget in enumerate(operator_inputs):
                op = op_widget.value
                sign, typ, i, f, bn, bd, kn_expr, kd_expr = term_inputs[idx + 1]
                next_val = to_fraction(sign.value, typ.value, i.value, f.value, bn.value, bd.value, kn_expr.value, kd_expr.value)
                expr_str += f" {op} ({next_val})"

                if op == '+':
                    result += next_val
                elif op == '-':
                    result -= next_val
                elif op == '*':
                    result *= next_val
                elif op == '/':
                    if next_val == 0:
                        raise ZeroDivisionError("Kan ikke dele på null")
                    result /= next_val

            desimal = round(float(result), 2)

            if abs(result.numerator) > result.denominator:
                heltall = result.numerator // result.denominator
                rest = abs(result.numerator) % result.denominator
                blandet = f"{heltall} {rest}/{result.denominator}" if rest else str(heltall)
            else:
                blandet = str(result)

            print(f"Uttrykk: {expr_str}")
            print(f"Forenklet brøk: {result}")
            print(f"Blandet tall: {blandet}")
            print(f"Desimaltall: {desimal}")

        except Exception as e:
            print(f"Feil: {e}")

# Koble knapper
confirm_button.on_click(create_inputs)
calculate_button.on_click(calculate)

# Startvisning
display(widgets.HBox([num_terms, confirm_button]))
display(input_container)

In [ ]:
# Omgjøringskalkulator mellom desimaltall, brøk og prosent
from decimal import Decimal, getcontext
from math import gcd

getcontext().prec = 10  # Sett presisjon for desimaler

def hovedmeny():
    print("Velkommen til omgjøringskalkulator mellom desimaltall, brøk og prosent ")
    print("Trykk 'q' når som helst for å avslutte programmet og trykk enter.\n")
    print("Hva vil du beregne?")
    print("1. Prosent til desimaltall og brøk")
    print("2. Brøk til desimaltall og prosent")
    print("3. Desimaltall til brøk og prosent")

def beregn_prosentandel(delen: float, hele: float) -> float:
    if hele == 0:
        raise ValueError("Hele kan ikke være null.")
    return (delen / hele) * 100

def decimal_to_fraction_and_percent(digits: str):
    try:
        n = Decimal(digits)
    except InvalidOperation:
        print("Ugyldig desimaltall.")
        return

    exponent = len(digits.split('.')[1]) if '.' in digits else 0
    numerator = int(n * 10**exponent)
    denominator = 10**exponent
    percent = float(n * 100)
    factor = gcd(numerator, denominator)
    num = numerator // factor
    den = denominator // factor

    print(f"Desimaltallet er {round(n, 3)}")
    print(f"Brøken er {num} / {den}")
    print(f"Prosenten er {round(percent, 3)}%\n")

def percent_to_decimal_and_fraction(percent: float):
    decimal = percent / 100
    digits = str(decimal)
    decimal_to_fraction_and_percent(digits)

def fraction_to_decimal_and_percent(numerator: int, denominator: int):
    if denominator == 0:
        print("Nevneren kan ikke være null.")
        return
    decimal = Decimal(numerator) / Decimal(denominator)
    percent = float(decimal * 100)

    print(f"Desimaltallet er {round(decimal, 3)}")
    print(f"Prosenten er {round(percent, 3)}%\n")

def main():
    while True:
        hovedmeny()
        choice = input("Velg et alternativ (1/2/3): ").strip().lower()
        if choice == 'q':
            print("Programmet avsluttes.")
            break
        elif choice == '1':
            percent = float(input("Skriv inn prosentverdien: "))
            percent_to_decimal_and_fraction(percent)
        elif choice == '2':
            numerator = int(input("Skriv inn telleren til brøken, altså det øverste tallet: "))
            denominator = int(input("Skriv inn nevneren til brøken, altså det nederste tallet: "))
            fraction_to_decimal_and_percent(numerator, denominator)
        elif choice == '3':
            digits = input("Skriv inn ett desimaltall, husk punktum, for å konvertere til brøk og prosent: ")
            decimal_to_fraction_and_percent(digits)
        else:
            print("Ugyldig valg. Vennligst prøv igjen.")
        
        restart = input("Vil du starte på nytt? (ja/nei): ").strip().lower()
        if restart == 'q':
            print("Programmet avsluttes.")
            break
        elif restart != 'ja':
            break

if __name__ == "__main__":
    main()

# $\color{red}{\text{Kapittel 1 - Prosentregler}}$
Forklare og bruke prosent, prosentpoeng og vekstfaktor til modellering av praktiske situasjoner med digitale verktøy

In [ ]:
# Finn % av ett tall. Formel: p % av ett tall = p/100 * tallet

import re
from typing import Optional
import ipywidgets as widgets
from IPython.display import display, Math, Latex

# --- Hjelpefunksjoner ---
def parse_number_no(s: str) -> float:
    """Parser norsk tallnotasjon med tusenskille og komma/punktum som desimal."""
    if isinstance(s, (int, float)):
        return float(s)
    if not isinstance(s, str):
        raise ValueError("Forventer tekst")
    s = s.strip()
    s = re.sub(r'[ \u00A0\u202F]', '', s)
    s = s.replace(',', '.')
    try:
        return float(s)
    except ValueError:
        raise ValueError("Ikke gyldig tall")

def parse_percent_no(s: str) -> float:
    """Parser prosentuttrykk som '2 %', '2,5 prosent' osv."""
    if isinstance(s, (int, float)):
        return float(s)
    if not isinstance(s, str):
        raise ValueError("Forventer tekst")
    s = s.strip()
    s = re.sub(r'(?<=\d)[ \u00A0\u202F]', '', s)
    s = s.replace(',', '.')
    s = re.sub(r'(%|prosent)', '', s, flags=re.IGNORECASE).strip()
    try:
        return float(s)
    except ValueError:
        raise ValueError("Ikke gyldig prosentuttrykk")

def format_no(n: float, decimals: Optional[int] = None) -> str:
    """Formaterer tall med norsk tusenskille og komma som desimaltegn."""
    neg = n < 0
    n = abs(n)
    s = f"{n:.{decimals if decimals is not None else 2}f}"
    helt, des = s.split('.')
    helt_rev = helt[::-1]
    grupper = ' '.join(helt_rev[i:i+3] for i in range(0, len(helt_rev), 3))[::-1]
    return ('-' if neg else '') + grupper + ',' + des

# --- Widgets ---
title = widgets.HTML("<h2 style='text-align:center;'>p% av ett tall</h2>")
formula_out = widgets.Output()
with formula_out:
    display(Math(r"\text{Formel: } \quad p\% \text{ av } x = \dfrac{p}{100} \times x"))

prosent_input = widgets.Text(value="25 %", description="Prosent:")
tall_input = widgets.Text(value="200", description="Tallet:")
desimaler = widgets.BoundedIntText(value=2, min=0, max=10, description="Desimaler:")
beregn_btn = widgets.Button(description="Beregn", button_style='success')
result_out = widgets.Output()

# --- Beregn-funksjon ---
def beregn(_):
    result_out.clear_output()
    try:
        prosent = parse_percent_no(prosent_input.value)
        tall = parse_number_no(tall_input.value)
        resultat = prosent / 100 * tall
        with result_out:
            display(Math(rf"\text{{{format_no(prosent)}\% av {format_no(tall)} er }}\ \color{{green}}{{\mathbf{{{resultat:.{desimaler.value}f}}}}}"))
            display(Math(rf"\text{{Formel: }}\ \left( \dfrac{{{prosent}}}{{100}} \right) \times {tall} = {resultat:.{desimaler.value}f}"))
    except ValueError:
        with result_out:
            display(Latex(r"\textcolor{red}{\text{Ugyldig input! Eksempler: } 2\%,\ 2{,}5\%,\ 1\ 000,\ 12\ 345{,}67}"))

beregn_btn.on_click(beregn)

# --- Layout ---
box = widgets.VBox([
    title,
    formula_out,
    widgets.HTML("<hr style='margin:8px 0;'>"),
    widgets.HBox([prosent_input, tall_input]),
    widgets.HBox([desimaler, beregn_btn]),
    result_out
])

display(box)
beregn(None)

In [ ]:
# Hvor mange prosent delen av ett tall er av det hele tallet er, feks 10 er ...% av 30
from dataclasses import dataclass
import ipywidgets as widgets
from IPython.display import display, Markdown, Latex

@dataclass
class Prosentandel:
    delen: float
    hele: float

    def beregn(self) -> float:
        if self.hele == 0:
            raise ValueError("Hele kan ikke være null.")
        return (self.delen / self.hele) * 100

def prosent_interaktiv(b):
    try:
        delen = delen_input.value
        hele = hele_input.value
        prosentandel = Prosentandel(delen=delen, hele=hele)
        prosent = prosentandel.beregn()
        result_output.clear_output()
        with result_output:
            display(Markdown(f"**{delen} er {prosent:.2f}% av {hele}.**"))
            display(Markdown(f"$$ \\text{{Eksempel: }} \\frac{{{delen}}}{{{hele}}} \\times 100\\% = {prosent:.2f}\\% $$"))
    except ValueError as e:
        result_output.clear_output()
        with result_output:
            display(Markdown(f"**Feil:** {e}"))

# Widgets
delen_input = widgets.FloatText(value=10.0, description="Delen:")
hele_input = widgets.FloatText(value=30.0, description="Hele:")
beregn_knapp = widgets.Button(description="Beregn", button_style='success')
beregn_knapp.on_click(prosent_interaktiv)

result_output = widgets.Output()

# Overskrift og formel med MathJax
display(Markdown("## Hvor mange prosent er ett tall av et annet?"))
display(Markdown("$$ \\textbf{Formel:}\\quad \\text{Prosent} = \\left( \\frac{\\text{Delen}}{\\text{Hele}} \\right) \\times 100\\% $$"))

display(widgets.VBox([delen_input, hele_input, beregn_knapp, result_output]))

In [ ]:
# Regel 1. Del av tallet = (Hele tallet ∙ Prosenten) / 100

# Regel 1: Del av tallet = (Hele tallet ∙ Prosenten) / 100
# Interaktiv kalkulator med ipywidgets og LaTeX-formel i overskriften

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# Funksjoner for beregning
def beregn_hele_tallet(prosent, kroner):
    return kroner * 100 / prosent

def beregn_prosent(hele_tallet, kroner):
    return (kroner / hele_tallet) * 100

def beregn_del_av_tallet(hele_tallet, prosent):
    return hele_tallet * prosent / 100

# Interaktivt grensesnitt
valg_dropdown = widgets.Dropdown(
    options=[('Velg beregning', 0), ('Hele tallet', 1), ('Prosent', 2), ('Del av tallet', 3)],
    description='Beregning:',
    style={'description_width': 'initial'}
)

# Input widgets
input1 = widgets.FloatText(description='Verdi 1:', style={'description_width': 'initial'})
input2 = widgets.FloatText(description='Verdi 2:', style={'description_width': 'initial'})
output_area = widgets.Output()

# Dynamisk oppdatering basert på valg
def oppdater_input(change):
    with output_area:
        clear_output()
        if valg_dropdown.value == 1:
            print("Du har valgt: Hele tallet")
            input1.description = "Prosent:"
            input2.description = "Del av tallet:"
        elif valg_dropdown.value == 2:
            print("Du har valgt: Prosent")
            input1.description = "Hele tallet:"
            input2.description = "Del av tallet:"
        elif valg_dropdown.value == 3:
            print("Du har valgt: Del av tallet")
            input1.description = "Hele tallet:"
            input2.description = "Prosent:"
        else:
            print("Velg en beregning fra menyen.")

valg_dropdown.observe(oppdater_input, names='value')

# Beregn-knapp
beregn_knapp = widgets.Button(description="Beregn", button_style='success')

def beregn_resultat(b):
    with output_area:
        clear_output()
        try:
            if valg_dropdown.value == 1:
                resultat = beregn_hele_tallet(input1.value, input2.value)
                print(f"Hele tallet er: {resultat:.2f}")
            elif valg_dropdown.value == 2:
                resultat = beregn_prosent(input1.value, input2.value)
                print(f"Prosentverdien er: {resultat:.2f} %")
            elif valg_dropdown.value == 3:
                resultat = beregn_del_av_tallet(input1.value, input2.value)
                print(f"Del av tallet er: {resultat:.2f}")
            else:
                print("Velg en beregning først.")
        except ZeroDivisionError:
            print("Feil: Du kan ikke dele på null.")
        except Exception as e:
            print(f"Ugyldig input: {e}")

beregn_knapp.on_click(beregn_resultat)

# Vis overskrift med LaTeX-formel og grensesnitt
display(Markdown("### Regel 1"))
display(Markdown(r"$$\text{Del av tallet} = \dfrac{\text{Hele tallet} \times \text{Prosenten}}{100}$$"))
display(valg_dropdown, input1, input2, beregn_knapp, output_area)

In [ ]:
# Finn endringen i prosentpoeng mellom to prosenter
import ipywidgets as widgets
from IPython.display import display, clear_output, Latex

# Vis LaTeX-overskrift
display(Latex(r"\[\text{Endringen i prosentpoeng} = \text{Prosent}_2 - \text{Prosent}_1\]"))

# Lag inputfelter
start_slider = widgets.FloatText(
    value=50.0,
    description='Startprosent:',
    step=0.1
)

slutt_slider = widgets.FloatText(
    value=75.0,
    description='Sluttprosent:',
    step=0.1
)

# Knapp for beregning
beregn_knapp = widgets.Button(description="Beregn endring")

# Output-område
output = widgets.Output()

# Beregningsfunksjon
def beregn_endring(b):
    with output:
        clear_output()
        try:
            startprosent = start_slider.value
            sluttprosent = slutt_slider.value
            
            prosentpoeng = sluttprosent - startprosent
            prosent = (prosentpoeng / startprosent) * 100
            
            # Vis resultat med LaTeX
            display(Latex(
                fr"""
                \[
                \text{{Endringen i prosentpoeng}} = {round(prosentpoeng, 2)} \\
                \text{{Endringen i prosent}} = {round(prosent, 2)}\%
                \]
                """
            ))
        except Exception as e:
            print("Feil:", e)

# Koble knapp til funksjon
beregn_knapp.on_click(beregn_endring)

# Vis komponentene
display(start_slider, slutt_slider, beregn_knapp, output)

In [ ]:
# Regel 2. Endringen i prosent = (Ny verdi – Opprinnelig verdi)/(Opprinnelig verdi) ∙ 100 %
from typing import Optional, Union
from IPython.display import display, Latex, Markdown
import ipywidgets as widgets

# ===========================
# Beregningslogikk (kjerne)
# ===========================
def beregn_prosentendring(
    ny_verdi: Optional[float] = None,
    opprinnelig_verdi: Optional[float] = None,
    endring_i_prosent: Optional[float] = None,
) -> Union[float, str]:
    """
    Beregner én av tre størrelser basert på prosentendringsformelen:
      Endringen i prosent = (Ny verdi – Opprinnelig verdi) / (Opprinnelig verdi) * 100 %

    Parametrene:
    - Oppgi nøyaktig to av tre:
        ny_verdi, opprinnelig_verdi, endring_i_prosent
    - Den tredje blir kalkulert.

    Returnerer:
    - float (hvis vellykket)
    - str med feilmelding (hvis det oppstår valideringsfeil)
    """
    antall_angitt = sum(v is not None for v in (ny_verdi, opprinnelig_verdi, endring_i_prosent))
    if antall_angitt != 2:
        return "Ugyldig input: Oppgi nøyaktig to av de tre verdiene (ny, opprinnelig, endring i %)."

    try:
        if endring_i_prosent is not None and opprinnelig_verdi is not None:
            # Finn ny verdi gitt opprinnelig og prosentendring
            return opprinnelig_verdi * (1 + endring_i_prosent / 100)

        if ny_verdi is not None and opprinnelig_verdi is not None:
            # Finn prosentendring gitt ny og opprinnelig verdi
            if opprinnelig_verdi == 0:
                return "Feil: Opprinnelig verdi kan ikke være lik 0 når du beregner prosentendring."
            return ((ny_verdi - opprinnelig_verdi) / opprinnelig_verdi) * 100

        if ny_verdi is not None and endring_i_prosent is not None:
            # Finn opprinnelig verdi gitt ny og prosentendring
            divisor = (1 + endring_i_prosent / 100)
            if divisor == 0:
                return ("Feil: Endring i prosent = -100 % gir divisor 0. "
                        "Da finnes ingen endelig opprinnelig verdi (matematisk ugyldig).")
            return ny_verdi / divisor

        return "Ugyldig input: logisk gren ikke truffet."

    except Exception as e:
        return f"En uventet feil oppstod: {e}"

# ===========================
# Presentasjon (LaTeX-tittel)
# ===========================
latex_tittel = r"""
\[
\text{Endringen i prosent} = \frac{\text{Ny verdi} - \text{Opprinnelig verdi}}{\text{Opprinnelig verdi}} \cdot 100\%
\]
"""
display(Latex(latex_tittel))
display(Markdown("**Velg hva du vil beregne, fyll inn to felter, og trykk på _Beregn_.**"))

# ===========================
# Widgets (GUI)
# ===========================
valg_widget = widgets.Dropdown(
    options=[
        ("Endringen i prosent", "prosent"),
        ("Ny verdi", "ny"),
        ("Opprinnelig verdi", "opprinnelig"),
    ],
    value="prosent",
    description="Beregn:",
)

ny_widget = widgets.FloatText(
    value=None,
    placeholder="Skriv tall (f.eks. 125.0)",
    description="Ny verdi:",
    disabled=False,
)

opprinnelig_widget = widgets.FloatText(
    value=None,
    placeholder="Skriv tall (f.eks. 100.0)",
    description="Opprinnelig:",
    disabled=False,
)

endring_widget = widgets.FloatText(
    value=None,
    placeholder="Skriv tall (f.eks. 25.0 for 25%)",
    description="Endring %:",
    disabled=False,
)

# Ny: Beregn-knapp
beregn_knapp = widgets.Button(
    description="Beregn",
    button_style="primary",  # 'primary', 'success', 'info', 'warning', 'danger' eller ''
    tooltip="Klikk for å beregne",
    icon="calculator",       # FontAwesome ikon
)

resultat_output = widgets.Output()

# Hjelpetekst
hjelp = widgets.HTML(
    value=(
        "<b>Tips:</b> Oppgi nøyaktig to av feltene. "
        "Det tredje blir beregnet når du trykker <i>Beregn</i>. "
        "Negativ endring (f.eks. -20%) er lov."
    )
)

# ===========================
# Dynamisk synlighet
# ===========================
def oppdater_synlighet(*_):
    """Vis/skjul felter avhengig av hva brukeren ønsker å beregne."""
    beregn = valg_widget.value
    ny_widget.layout.display = "flex"
    opprinnelig_widget.layout.display = "flex"
    endring_widget.layout.display = "flex"

    if beregn == "prosent":
        # Skal beregne prosent => treng ny og opprinnelig
        endring_widget.layout.display = "none"
    elif beregn == "ny":
        # Skal beregne ny verdi => treng opprinnelig og endring %
        ny_widget.layout.display = "none"
    elif beregn == "opprinnelig":
        # Skal beregne opprinnelig verdi => treng ny og endring %
        opprinnelig_widget.layout.display = "none"

oppdater_synlighet()
# Behold observasjon KUN for synlighet (ikke for beregning)
valg_widget.observe(oppdater_synlighet, names="value")

# ===========================
# Beregning og visning (via knapp)
# ===========================
def format_resultat(verdi: Union[float, str], beregn: str) -> str:
    """Formaterer resultatet pent med 2 desimaler når aktuelt."""
    if isinstance(verdi, str):
        return f"**Feil:** {verdi}"
    if beregn == "prosent":
        return f"**Endringen i prosent:** {verdi:.2f} %"
    elif beregn == "ny":
        return f"**Ny verdi:** {verdi:.2f}"
    elif beregn == "opprinnelig":
        return f"**Opprinnelig verdi:** {verdi:.2f}"
    return f"Resultat: {verdi}"

def hent_inndata(beregn: str):
    """
    Returnerer tuple (ny, opprinnelig, endring%) med None der feltet
    ikke skal oppgis for den gjeldende beregningen.
    """
    if beregn == "prosent":
        return (ny_widget.value, opprinnelig_widget.value, None)
    elif beregn == "ny":
        return (None, opprinnelig_widget.value, endring_widget.value)
    elif beregn == "opprinnelig":
        return (ny_widget.value, None, endring_widget.value)
    return (None, None, None)

def oppdater_resultat(*_):
    beregn = valg_widget.value
    ny, opprinnelig, endring = hent_inndata(beregn)

    # Valider: nøyaktig to felter må ha tall (ikke None)
    antall = sum(v is not None for v in (ny, opprinnelig, endring))
    resultat_output.clear_output()

    with resultat_output:
        if antall != 2:
            display(Markdown("Oppgi nøyaktig **to** verdier. Det tredje blir kalkulert."))
            return

        resultat = beregn_prosentendring(
            ny_verdi=ny,
            opprinnelig_verdi=opprinnelig,
            endring_i_prosent=endring
        )
        display(Markdown(format_resultat(resultat, beregn)))

# Knytt beregning til knappen (ikke til felt-endringer)
beregn_knapp.on_click(oppdater_resultat)

# Første visning (uten automatisk beregning)
resultat_output.clear_output()
with resultat_output:
    display(Markdown("Trykk **Beregn** når du har fylt ut to felt."))

# ===========================
# Layout
# ===========================
ui = widgets.VBox([
    valg_widget,
    hjelp,
    widgets.HBox([ny_widget, opprinnelig_widget, endring_widget]),
    beregn_knapp,   # Plassert rett over resultat
    resultat_output
])

display(ui)

In [ ]:
# Regel 3. Ny verdi = Opprinnelig verdi * Vekstfaktor + Beregning av vekstfaktor
from __future__ import annotations
import math
from typing import Optional

# Avhengigheter for notebook-visning:
try:
    import ipywidgets as widgets
    from IPython.display import display, Markdown
except ImportError as e:
    raise ImportError(
        "Mangler ipywidgets eller IPython. Installer med:\n"
        "  %pip install ipywidgets\n"
        "Start notebooken på nytt etter installasjon."
    ) from e


# ---------- Tallhåndtering (norsk) ----------

def parse_tall(tekst: str) -> float:
    """
    Gjør om brukerinput til float og støtter norsk formatering:
    - Desimal: komma (',') eller punktum ('.')
    - Tusenskille: mellomrom, punktum (.) eller smale NBSP-tegn
    Eksempler: '2,5' -> 2.5, '1 234,56' -> 1234.56, '1.234,56' -> 1234.56
    """
    if tekst is None:
        raise ValueError("Mangler tallverdi.")
    s = tekst.strip()
    if not s:
        raise ValueError("Tomt felt. Oppgi et tall.")
    # Normaliser spesialmellomrom og minustegn
    s = (s.replace('\u00A0', ' ')
           .replace('\u202F', ' ')
           .replace('−', '-'))
    # Hvis både komma og punktum finnes, antas norsk format
    if ',' in s and '.' in s:
        s = s.replace('.', '')   # fjern tusenskille
        s = s.replace(',', '.')  # sett desimal
    elif ',' in s:
        s = s.replace(',', '.')  # desimal-komma
    # Fjern mellomrom brukt som tusenskille
    s = s.replace(' ', '')
    return float(s)


def parse_vekstfaktor(tekst: str) -> float:
    """
    Parser vekstfaktor som enten direkte faktor eller prosentendring.
    Tillatte former:
      - Direkte faktor: '1,05', '0,8', '2', '1.03'
      - Prosent: '5%', '+5 %', '-20%', '2,5%'
    Regler:
      - Hvis '%' finnes: faktor = 1 + (verdi_i_prosent / 100)
      - Uten '%': tolkes som direkte faktor (NB: '5' = faktor 5.0, ikke 5 %)
    """
    if tekst is None:
        raise ValueError("Mangler vekstfaktor.")
    s = tekst.strip().lower()
    if not s:
        raise ValueError("Tomt felt. Oppgi vekstfaktor som faktor (f.eks. 1,05) eller prosent (f.eks. 5%).")
    s = (s.replace('\u00A0', ' ')
           .replace('\u202F', ' ')
           .replace('−', '-'))
    if '%' in s:
        s = s.replace('%', '').strip()
        verdi_prosent = parse_tall(s)
        return 1.0 + (verdi_prosent / 100.0)
    else:
        return parse_tall(s)


# ---------- Beregning ----------

def beregn_verdi(
    opprinnelig_verdi: Optional[float] = None,
    vekstfaktor: Optional[float] = None,
    ny_verdi: Optional[float] = None
) -> float:
    """
    Returnerer:
      - Ny verdi hvis opprinnelig_verdi og vekstfaktor er oppgitt.
      - Opprinnelig verdi hvis ny_verdi og vekstfaktor er oppgitt.
      - Vekstfaktor hvis ny_verdi og opprinnelig_verdi er oppgitt.
    Kaster ValueError ved ugyldige delinger eller feil kombinasjoner.
    """
    ant = sum(v is not None for v in (opprinnelig_verdi, vekstfaktor, ny_verdi))
    if ant != 2:
        raise ValueError("Oppgi nøyaktig to av: opprinnelig verdi, vekstfaktor, ny verdi.")

    if opprinnelig_verdi is not None and vekstfaktor is not None:
        return opprinnelig_verdi * vekstfaktor

    if ny_verdi is not None and vekstfaktor is not None:
        if vekstfaktor == 0:
            raise ValueError("Vekstfaktor kan ikke være 0 ved beregning av opprinnelig verdi (−100 %).")
        return ny_verdi / vekstfaktor

    if ny_verdi is not None and opprinnelig_verdi is not None:
        if opprinnelig_verdi == 0:
            raise ValueError("Opprinnelig verdi kan ikke være 0 ved beregning av vekstfaktor.")
        return ny_verdi / opprinnelig_verdi

    raise ValueError("Ugyldig kombinasjon av input.")


# ---------- Norsk formatering av tall ----------

def format_norsk(x: float, desimaler: int = 2) -> str:
    """
    Formaterer tall med norsk stil: tusenskille ' ' og desimalkomma ','.
    Eksempel: 12345.678 -> '12 345,68'
    """
    if x is None or (isinstance(x, float) and (math.isnan(x) or math.isinf(x))):
        return str(x)
    s = f"{x:,.{desimaler}f}"       # f.eks. '12,345.68' (US)
    s = s.replace(',', 'X')          # '12X345.68'
    s = s.replace('.', ',')          # '12X345,68'
    s = s.replace('X', ' ')          # '12 345,68'
    return s


def format_prosent(endring: float, desimaler: int = 1) -> str:
    """
    Formaterer prosentendring (f.eks. +5,2 % eller -20,0 %).
    endring gis som prosent (ikke som faktor), altså 5.2 for +5,2 %.
    """
    tegn = '+' if endring >= 0 else '−'
    return f"{tegn}{format_norsk(abs(endring), desimaler)} %"


# ---------- MathJax-overskrift og formler (bruker $$ ... $$) ----------
# NB: bruker "endring i %" i stedet for Δ % i formelvisningen

display(Markdown(r"""
# **Regel 3**
$$
\textbf{Ny verdi} = \textbf{Opprinnelig verdi} \times \textbf{Vekstfaktor}
$$

$$
\text{Vekstfaktor} = 1 + \frac{\text{endring i \%}}{100}
$$
"""))


# ---------- Interaktivt UI (ipywidgets) ----------

# Valg av hva som skal beregnes
valg = widgets.ToggleButtons(
    options=[
        ("Ny verdi", "ny"),
        ("Opprinnelig verdi", "opprinnelig"),
        ("Vekstfaktor", "faktor"),
    ],
    value="ny",
    description="Hva vil du finne?",
    button_style='',  # 'success', 'info', 'warning', 'danger' eller ''.
)

# Inndatafelter
inp_opprinnelig = widgets.Text(
    value='',
    placeholder="Eks: 1 250,50",
    description='Opprinnelig:',
    disabled=False,
)

inp_ny = widgets.Text(
    value='',
    placeholder="Eks: 1 365,00",
    description='Ny:',
    disabled=False,
)

inp_faktor = widgets.Text(
    value='',
    placeholder="(feks: VF 1,05 eller 5%)",
    description='Vekstfaktor:',
    disabled=False,
)

# Knapper
beregn_btn = widgets.Button(
    description='Beregn',
    button_style='success',
    icon='calculator',
    tooltip='Utfør beregning'
)

rst_btn = widgets.Button(
    description='Tøm',
    button_style='warning',
    icon='eraser',
    tooltip='Tøm alle felt'
)

# Output-område
ut = widgets.Output()

# Dynamisk visning av felter basert på valgt modus
def oppdater_siktbarhet(*args):
    modus = valg.value
    inp_opprinnelig.layout.display = 'flex'
    inp_ny.layout.display = 'flex'
    inp_faktor.layout.display = 'flex'
    if modus == 'ny':
        # Finn ny verdi => (opprinnelig, faktor)
        inp_ny.layout.display = 'none'
    elif modus == 'opprinnelig':
        # Finn opprinnelig => (ny, faktor)
        inp_opprinnelig.layout.display = 'none'
    else:
        # Finn faktor => (opprinnelig, ny)
        inp_faktor.layout.display = 'none'

valg.observe(lambda change: oppdater_siktbarhet(), names='value')
oppdater_siktbarhet()


# Beregning ved klikk (med MathJax/LaTeX i $$ ... $$)
def beregn_klikk(_):
    ut.clear_output()
    modus = valg.value
    try:
        if modus == 'ny':
            opprinnelig = parse_tall(inp_opprinnelig.value)
            faktor = parse_vekstfaktor(inp_faktor.value)
            nyv = beregn_verdi(opprinnelig_verdi=opprinnelig, vekstfaktor=faktor)
            with ut:
                display(Markdown(rf"""
### ✅ Resultat
**Ny verdi** = {format_norsk(nyv, 2)}

#### Forklaring
$$
\text{{Ny}} = \text{{Opprinnelig}} \times \text{{Vekstfaktor}}
= {format_norsk(opprinnelig, 2)} \times {faktor:.3f}
= {format_norsk(nyv, 2)}
$$
"""))
        elif modus == 'opprinnelig':
            nyv = parse_tall(inp_ny.value)
            faktor = parse_vekstfaktor(inp_faktor.value)
            if faktor == 0:
                raise ValueError("Vekstfaktor kan ikke være 0 ved beregning av opprinnelig verdi (−100 %).")
            oppr = beregn_verdi(vekstfaktor=faktor, ny_verdi=nyv)
            with ut:
                display(Markdown(rf"""
### ✅ Resultat
**Opprinnelig verdi** = {format_norsk(oppr, 2)}

#### Forklaring
$$
\text{{Opprinnelig}} = \frac{{\text{{Ny}}}}{{\text{{Vekstfaktor}}}}
= \frac{{{format_norsk(nyv, 2)}}}{{{faktor:.3f}}}
= {format_norsk(oppr, 2)}
$$
"""))
        else:  # modus == 'faktor'
            opprinnelig = parse_tall(inp_opprinnelig.value)
            nyv = parse_tall(inp_ny.value)
            if opprinnelig == 0:
                raise ValueError("Opprinnelig verdi kan ikke være 0 ved beregning av vekstfaktor.")
            faktor = beregn_verdi(opprinnelig_verdi=opprinnelig, ny_verdi=nyv)
            endring_pct = (faktor - 1.0) * 100.0
            tegn = '+' if endring_pct >= 0 else '−'
            med_tegn = f"{tegn}{abs(endring_pct):.1f} %"
            with ut:
                display(Markdown(rf"""
### ✅ Resultat
**Vekstfaktor** = {faktor:.3f}  &nbsp;&nbsp;({med_tegn})

#### Forklaring
$$
\text{{Vekstfaktor}} = \frac{{\text{{Ny}}}}{{\text{{Opprinnelig}}}}
= \frac{{{format_norsk(nyv, 2)}}}{{{format_norsk(opprinnelig, 2)}}}
= {faktor:.3f}
$$

$$
\text{{endring i \%}} = (\text{{Vekstfaktor}} - 1)\cdot 100
= ({faktor:.3f} - 1)\cdot 100
= {endring_pct:+.1f}\%
$$
"""))
    except ValueError as e:
        with ut:
            display(Markdown(f"<span style='color:#b00020'><b>Feil:</b> {str(e)}</span>"))

def reset_klikk(_):
    inp_opprinnelig.value = ''
    inp_ny.value = ''
    inp_faktor.value = ''
    ut.clear_output()

beregn_btn.on_click(beregn_klikk)
rst_btn.on_click(reset_klikk)

# Layout og visning
rad1 = widgets.HBox([valg])
rad2 = widgets.VBox([inp_opprinnelig, inp_ny, inp_faktor])
rad3 = widgets.HBox([beregn_btn, rst_btn])
app = widgets.VBox([rad1, rad2, rad3, ut])

display(app)


# ---------- (Valgfritt) Enkle tester ----------
def _nrtst():
    # parse_tall
    assert parse_tall("2,5") == 2.5
    assert parse_tall("1 234,56") == 1234.56
    assert parse_tall("1.234,56") == 1234.56
    assert parse_tall("-1 000") == -1000.0

    # parse_vekstfaktor
    assert abs(parse_vekstfaktor("5%") - 1.05) < 1e-12
    assert abs(parse_vekstfaktor("-20%") - 0.8) < 1e-12
    assert abs(parse_vekstfaktor("1,05") - 1.05) < 1e-12
    assert abs(parse_vekstfaktor("2") - 2.0) < 1e-12

    # beregn_verdi
    assert abs(beregn_verdi(100, 1.05) - 105) < 1e-12
    assert abs(beregn_verdi(None, 1.25, 125) - 100) < 1e-12
    assert abs(beregn_verdi(80, None, 100) - 1.25) < 1e-12

    # formatering
    assert format_norsk(12345.678, 2) == "12 345,68"
    assert format_prosent(5.25, 2) == "+5,25 %"
    assert format_prosent(-20.0) == "−20,0 %"

    print("Alle tester OK.")

# Kjør ved behov:
# _nrtst()

In [ ]:
#  Regel 4. Ny verdi = Opprinnelig verdi * Vekstfaktor^n hvor n er tiden + løsning av en ukjent i formelen = ett tall 
from math import log
from typing import Optional
from ipywidgets import (
    Dropdown, RadioButtons, FloatText, Button, HBox, VBox, Layout, Output, Checkbox
)
from IPython.display import display, Markdown

# --- UI-komponenter ---
retning = RadioButtons(
    options=[('Økning', 'a'), ('Reduksjon', 'm')],
    value='a',
    description='Retning:',
    layout=Layout(width='300px')
)

oppgavetype = Dropdown(
    options=[
        ('Beregn ny verdi (N)', 'n'),
        ('Beregn opprinnelig verdi (G)', 'g'),
        ('Beregn vekstfaktor (F)', 'v'),
        ('Beregn tid (n)', 't'),
        ('Løs én ukjent (sett ett felt tomt)', 'x')
    ],
    value='n',
    description='Oppgave:',
    layout=Layout(width='400px')
)

inputtype = RadioButtons(
    options=[('Prosent (p)', 'p'), ('Vekstfaktor (F)', 'v')],
    value='p',
    description='Inputtype:',
    layout=Layout(width='400px')
)

felt_N = FloatText(description='Ny verdi (N):', layout=Layout(width='300px'))
felt_G = FloatText(description='Opprinnelig (G):', layout=Layout(width='300px'))
felt_p = FloatText(description='Prosent (p %):', layout=Layout(width='300px'))
felt_F = FloatText(description='Vekstfaktor (F):', layout=Layout(width='300px'))
felt_n = FloatText(description='Tid (n år):', layout=Layout(width='300px'))

btn_beregn = Button(description='Beregn', button_style='primary', layout=Layout(width='150px'))
btn_nullstill = Button(description='Nullstill', layout=Layout(width='120px'))
vis_utledning = Checkbox(value=False, description='Vis utledning (LaTeX)')
out = Output()

def oppdater_siktbarhet(*args):
    if inputtype.value == 'p':
        felt_p.layout.display = ''
        felt_F.layout.display = 'none'
    else:
        felt_p.layout.display = 'none'
        felt_F.layout.display = ''
oppdater_siktbarhet()
inputtype.observe(oppdater_siktbarhet, names='value')

# --- Hjelpefunksjoner ---
def _til_float(verdi) -> Optional[float]:
    return float(verdi) if verdi is not None else None

def _lag_prosent_fra_F(F: float, retning_kode: str) -> float:
    return 100.0 * (F - 1.0) if retning_kode == 'a' else 100.0 * (1.0 - F)

def _lag_F_fra_prosent(p: float, retning_kode: str) -> float:
    return 1.0 + p / 100.0 if retning_kode == 'a' else 1.0 - p / 100.0

def _valider_F(F: float) -> Optional[str]:
    return "Vekstfaktoren F må være > 0." if F <= 0.0 else None

def _valider_prosent_for_reduksjon(p: float, retning_kode: str) -> Optional[str]:
    return "Ved reduksjon må prosent p være mindre enn 100%, ellers blir F ≤ 0." if retning_kode == 'm' and p >= 100.0 else None

def _format_tall(x: float, desimaler: int = 2) -> str:
    return f"{x:.{desimaler}f}"

def _latex_num(x: float) -> str:
    return f"{x:.6g}"

def _vis_resultat(markdown_tekst: str, latex_tekst: Optional[str] = None):
    with out:
        out.clear_output()
        if markdown_tekst:
            display(Markdown(markdown_tekst))
        if latex_tekst:
            display(Markdown(f"$$ {latex_tekst} $$"))  # Bruk blokkmath med $$

# --- Beregningslogikk ---
def beregn(_):
    with out:
        out.clear_output()
    ret = retning.value
    modus = oppgavetype.value
    bruk_prosent = (inputtype.value == 'p')

    N = _til_float(felt_N.value)
    G = _til_float(felt_G.value)
    p = _til_float(felt_p.value) if bruk_prosent else None
    F = _til_float(felt_F.value) if not bruk_prosent else None
    n = _til_float(felt_n.value)

    try:
        # --- Vanlige moduser ---
        if modus in ['n', 'g', 'v', 't']:
            if modus == 'n':
                if bruk_prosent:
                    err = _valider_prosent_for_reduksjon(p, ret)
                    if err: _vis_resultat(f"**Feil:** {err}"); return
                    F = _lag_F_fra_prosent(p, ret)
                errF = _valider_F(F)
                if errF: _vis_resultat(f"**Feil:** {errF}"); return
                if G == 0.0 or n == 0.0: _vis_resultat("**Feil:** Du må fylle inn G og n."); return
                N_calc = G * (F ** n)
                md = f"**Ny verdi:** N = { _format_tall(N_calc) }"
                latex = f"N = G \\cdot F^n = { _latex_num(G) } \\cdot { _latex_num(F) }^{{{ _latex_num(n) }}}"
                _vis_resultat(md, latex if vis_utledning.value else None)

            elif modus == 'g':
                if bruk_prosent:
                    err = _valider_prosent_for_reduksjon(p, ret)
                    if err: _vis_resultat(f"**Feil:** {err}"); return
                    F = _lag_F_fra_prosent(p, ret)
                errF = _valider_F(F)
                if errF: _vis_resultat(f"**Feil:** {errF}"); return
                if N == 0.0 or n == 0.0: _vis_resultat("**Feil:** Du må fylle inn N og n."); return
                G_calc = N / (F ** n)
                md = f"**Opprinnelig verdi:** G = { _format_tall(G_calc) }"
                latex = f"G = \\frac{{N}}{{F^n}} = \\frac{{{ _latex_num(N) }}}{{{ _latex_num(F) }^{{{ _latex_num(n) }}}}}"
                _vis_resultat(md, latex if vis_utledning.value else None)

            elif modus == 'v':
                if N == 0.0 or G == 0.0 or n == 0.0: _vis_resultat("**Feil:** Du må fylle inn N, G og n."); return
                F_calc = (N / G) ** (1.0 / n)
                p_calc = _lag_prosent_fra_F(F_calc, ret)
                md = f"**Vekstfaktor:** F = { _format_tall(F_calc, 4) }  \n**Tilsvarende prosent:** p = { _format_tall(p_calc) } %"
                latex = f"F = \\left(\\frac{{N}}{{G}}\\right)^{{1/n}} = \\left(\\frac{{{ _latex_num(N) }}}{{{ _latex_num(G) }}}\\right)^{{1/{ _latex_num(n) }}}"
                _vis_resultat(md, latex if vis_utledning.value else None)

            elif modus == 't':
                if bruk_prosent:
                    err = _valider_prosent_for_reduksjon(p, ret)
                    if err: _vis_resultat(f"**Feil:** {err}"); return
                    F = _lag_F_fra_prosent(p, ret)
                errF = _valider_F(F)
                if errF: _vis_resultat(f"**Feil:** {errF}"); return
                if F == 1.0: _vis_resultat("**Feil:** F = 1 gir ingen endring."); return
                if N == 0.0 or G == 0.0: _vis_resultat("**Feil:** Du må fylle inn N og G."); return
                n_calc = log(N / G) / log(F)
                md = f"**Tid:** n = { _format_tall(n_calc) } år"
                latex = f"n = \\frac{{\\ln(N/G)}}{{\\ln(F)}} = \\frac{{\\ln({ _latex_num(N) }/{ _latex_num(G) })}}{{\\ln({ _latex_num(F) })}}"
                _vis_resultat(md, latex if vis_utledning.value else None)

        # --- x-modus ---
        elif modus == 'x':
            # Identifiser mangler
            mangler = []
            if N == 0.0: mangler.append('N')
            if G == 0.0: mangler.append('G')
            if n == 0.0: mangler.append('n')
            if bruk_prosent and p == 0.0: mangler.append('F/p')
            if not bruk_prosent and F == 0.0: mangler.append('F/p')

            if len(mangler) != 1:
                _vis_resultat(f"**Feil:** I x-modus må nøyaktig ett felt være ukjent. Nå mangler: {', '.join(mangler) or 'ingen'}.")
                return

            ukjent = mangler[0]
            Fcalc = None
            if bruk_prosent and p != 0.0:
                Fcalc = _lag_F_fra_prosent(p, ret)
            elif not bruk_prosent and F != 0.0:
                Fcalc = F

            if ukjent == 'N':
                if G == 0.0 or n == 0.0 or Fcalc is None: _vis_resultat("**Feil:** For å finne N må G, n og F/p være utfylt."); return
                N_calc = G * (Fcalc ** n)
                md = f"**Ny verdi:** N = { _format_tall(N_calc) }"
                latex = f"N = G \\cdot F^n = { _latex_num(G) } \\cdot { _latex_num(Fcalc) }^{{{ _latex_num(n) }}}"
                _vis_resultat(md, latex if vis_utledning.value else None)

            elif ukjent == 'G':
                if N == 0.0 or n == 0.0 or Fcalc is None: _vis_resultat("**Feil:** For å finne G må N, n og F/p være utfylt."); return
                G_calc = N / (Fcalc ** n)
                md = f"**Opprinnelig verdi:** G = { _format_tall(G_calc) }"
                latex = f"G = \\frac{{N}}{{F^n}} = \\frac{{{ _latex_num(N) }}}{{{ _latex_num(Fcalc) }^{{{ _latex_num(n) }}}}}"
                _vis_resultat(md, latex if vis_utledning.value else None)

            elif ukjent == 'n':
                if N == 0.0 or G == 0.0 or Fcalc is None: _vis_resultat("**Feil:** For å finne n må N, G og F/p være utfylt."); return
                if Fcalc == 1.0: _vis_resultat("**Feil:** F = 1 gir ingen endring."); return
                n_calc = log(N / G) / log(Fcalc)
                md = f"**Tid:** n = { _format_tall(n_calc) } år"
                latex = f"n = \\frac{{\\ln(N/G)}}{{\\ln(F)}} = \\frac{{\\ln({ _latex_num(N) }/{ _latex_num(G) })}}{{\\ln({ _latex_num(Fcalc) })}}"
                _vis_resultat(md, latex if vis_utledning.value else None)

            elif ukjent == 'F/p':
                if N == 0.0 or G == 0.0 or n == 0.0: _vis_resultat("**Feil:** For å finne F/p må N, G og n være utfylt."); return
                F_calc = (N / G) ** (1.0 / n)
                p_calc = _lag_prosent_fra_F(F_calc, ret)
                md = f"**Vekstfaktor:** F = { _format_tall(F_calc, 4) }  \n**Tilsvarende prosent:** p = { _format_tall(p_calc) } %"
                latex = f"F = \\left(\\frac{{N}}{{G}}\\right)^{{1/n}} = \\left(\\frac{{{ _latex_num(N) }}}{{{ _latex_num(G) }}}\\right)^{{1/{ _latex_num(n) }}}"
                _vis_resultat(md, latex if vis_utledning.value else None)

        else:
            _vis_resultat("**Feil:** Ugyldig oppgavevalg.")

    except Exception as e:
        _vis_resultat(f"**Det oppstod en feil:** {e}")

def nullstill(_):
    felt_N.value = 0.0
    felt_G.value = 0.0
    felt_p.value = 0.0
    felt_F.value = 0.0
    felt_n.value = 0.0
    out.clear_output()

btn_beregn.on_click(beregn)
btn_nullstill.on_click(nullstill)

# --- UI med dollartegn ---
toppinfo = Markdown(r"""
### Interaktivt verktøy – Regel 4:

$$
N = G \cdot F^{n}
$$

Fyll inn feltene under og trykk **Beregn**.  
Velg om du oppgir **prosent (p)** eller **vekstfaktor (F)**.  
Aktiver **Vis utledning (LaTeX)** for å se regnetrinnene.
""")

rad_1 = HBox([retning, oppgavetype])
rad_2 = HBox([inputtype, vis_utledning])
rad_3 = HBox([felt_N, felt_G])
rad_4 = HBox([felt_p, felt_F])
rad_5 = HBox([felt_n])
rad_6 = HBox([btn_beregn, btn_nullstill])

display(toppinfo, rad_1, rad_2, rad_3, rad_4, rad_5, rad_6, out)

# $\color{green}{\text{Kapittel 2 - Potenser, Standardform og Røtter}}$

In [1]:
# Standardform-kalkulator
import math
from typing import List, Optional, Tuple

from IPython.display import display, Math, HTML
import ipywidgets as widgets

# -----------------------------
# Litt enkel stil for oversikt
# -----------------------------
display(HTML("""
<style>
  .widget-label { min-width: 120px; }
  .calc-section h3 { margin-top: 0.6rem; }
  .calc-section hr { margin: 0.8rem 0; }
  .note { color: #555; font-size: 90%; }
  .error { color: #b00020; }
</style>
"""))

# -----------------------------
# Hjelpefunksjoner: parsing, formatering og beregning
# -----------------------------

def parse_scientific_input(input_str: str) -> Optional[float]:
    """
    Tolker input som kan være:
      - vanlig tall: 5200, -0.0035
      - e-notasjon: 5.2e3, -3.5e-4
      - a * 10^n: 5.2 * 10^3, -3.5*10^-4
    Tillater komma som desimaltegn.
    Returnerer float eller None ved ugyldig format.
    """
    if input_str is None:
        return None
    s = input_str.strip().replace(" ", "").lower()
    s = s.replace(",", ".")  # norsk komma

    if "*10^" in s:
        try:
            mantisse_str, eksponent_str = s.split("*10^", 1)
            mantisse = float(mantisse_str)
            eksponent = int(eksponent_str)
            return mantisse * (10 ** eksponent)
        except (ValueError, IndexError):
            return None
    else:
        # Prøv standard float (inkl. e-notasjon)
        try:
            return float(s)
        except ValueError:
            return None


def til_standardform(tall: float, desimaler: int = 2) -> Tuple[str, float, int]:
    """
    Konverterer et tall til LaTeX-standardformen a × 10^n.
    Returnerer (latex_str, a, n).

    Regler:
      - 0 vises som 0.00 × 10^0 (med valgt desimaler).
      - NaN/inf håndteres pent med 0 × 10^0.
      - a rundes til valgt antall desimaler.
      - Justerer ved kanttilfelle der avrunding gir 10.00 -> skifter eksponent.
    """
    if not math.isfinite(tall) or tall == 0:
        a_fmt = f"{0:.{desimaler}f}"
        return f"{a_fmt}\\times 10^{{0}}", 0.0, 0

    # Finn eksponent n slik at 1 <= |a| < 10
    n = int(math.floor(math.log10(abs(tall))))
    a = tall / (10 ** n)

    # Avrunding av mantisse
    a_fmt = f"{a:.{desimaler}f}"

    # Kanttilfelle: mantisse runder til 10.00
    if float(a_fmt) >= 10:
        a = float(a_fmt) / 10.0
        n += 1
        a_fmt = f"{a:.{desimaler}f}"

    latex_str = f"{a_fmt}\\times 10^{{{n}}}"
    return latex_str, float(a_fmt), n


def latex_wrapper(latex_eq: str) -> Math:
    """
    Pakk en LaTeX-likning som Math-objekt med \\displaystyle for tydelig visning.
    """
    return Math(f"\\displaystyle {latex_eq}")


def formater_beregning_str(operasjon: str, tall_liste: List[float]) -> str:
    """
    Lager en pen LaTeX-streng som viser selve beregningen.
    """
    if operasjon == "Addisjon":
        deler = [f"{t}" for t in tall_liste]
        return " + ".join(deler)
    elif operasjon == "Subtraksjon":
        deler = [f"{t}" for t in tall_liste]
        return " - ".join(deler)
    elif operasjon == "Multiplikasjon":
        deler = [f"{t}" for t in tall_liste]
        return " \\times ".join(deler)
    elif operasjon == "Divisjon":
        deler = [f"{t}" for t in tall_liste]
        return " \\div ".join(deler)
    else:
        return "?"


def safe_prod(tall: List[float]) -> float:
    """
    Robust produkt. Tom liste gir 1.0 (matematisk nøytral element).
    """
    if not tall:
        return 1.0
    p = 1.0
    for x in tall:
        p *= x
    return p


# -----------------------------
# UI: Enkel modus (listeoperasjoner)
# -----------------------------

# Kontroller
operasjon_dd = widgets.Dropdown(
    options=["Addisjon", "Subtraksjon", "Multiplikasjon", "Divisjon"],
    value="Addisjon",
    description="Operasjon:",
)

antall_slider = widgets.IntSlider(
    value=2, min=2, max=10, step=1, description="Antall tall:", continuous_update=False
)

desimaler_slider = widgets.IntSlider(
    value=2, min=0, max=10, step=1, description="Desimaler:"
)

inputs_box = widgets.VBox([])
beregn_btn = widgets.Button(description="Beregn", button_style="success")
nullstill_btn = widgets.Button(description="Nullstill", button_style="warning")
output_area = widgets.Output()
historikk = widgets.HTML("<b>Historikk:</b><br>")
hjelp_html = widgets.HTML(
    "<p class='note'>Bruk f.eks. <code>5200</code>, <code>5.2e3</code> eller <code>5.2 * 10^3</code>. "
    "Komma som desimaltegn er tillatt.</p>"
)

def lag_inputfelt(n: int):
    felt = []
    for i in range(n):
        tb = widgets.Text(
            description=f"tall {i+1}:",
            placeholder="Eks: 5200, 5.2e3 eller 5.2 * 10^3",
            layout=widgets.Layout(width="60%")
        )
        felt.append(tb)
    inputs_box.children = felt

def oppdater_antall(change=None):
    lag_inputfelt(antall_slider.value)

antall_slider.observe(oppdater_antall, names="value")
oppdater_antall()

def beregn(_):
    output_area.clear_output()
    tall_liste = []
    feil = []
    for tb in inputs_box.children:
        v = parse_scientific_input(tb.value)
        if v is None:
            feil.append(tb.value)
        else:
            tall_liste.append(v)

    with output_area:
        if feil:
            display(HTML(f"<p class='error'>Ugyldig input: {', '.join([repr(x) for x in feil])}. "
                         "Bruk f.eks. <code>5200</code>, <code>5.2e3</code> eller <code>5.2 * 10^3</code>.</p>"))
            return

        if len(tall_liste) < 2:
            display(HTML("<p class='error'>Du må oppgi minst to tall.</p>"))
            return

        # Utregning
        op = operasjon_dd.value
        if op == "Addisjon":
            resultat = sum(tall_liste)
        elif op == "Subtraksjon":
            resultat = tall_liste[0] - sum(tall_liste[1:])
        elif op == "Multiplikasjon":
            resultat = safe_prod(tall_liste)
        elif op == "Divisjon":
            resultat = tall_liste[0]
            for x in tall_liste[1:]:
                if x == 0:
                    display(HTML("<p class='error'>FEIL: Kan ikke dele på 0.</p>"))
                    return
                resultat /= x
        else:
            display(HTML("<p class='error'>Ukjent operasjon.</p>"))
            return

        # Formatering
        des = desimaler_slider.value
        latex_std, a, n = til_standardform(resultat, desimaler=des)
        beregning_str = formater_beregning_str(op, tall_liste)

        display(HTML("<div class='calc-section'><hr><h3>Resultat</h3></div>"))
        display(latex_wrapper(f"{beregning_str} = {latex_std}"))
        display(HTML(f"<p>(Dette betyr <code>{a}</code> ganget med <code>10</code> opphøyd i <code>{n}</code>.)</p>"))

        # Historikk
        histlinje = f"{beregning_str} = {latex_std}"
        historikk.value += f"{histlinje}<br>"

def nullstill(_):
    for tb in inputs_box.children:
        tb.value = ""
    output_area.clear_output()

beregn_btn.on_click(beregn)
nullstill_btn.on_click(nullstill)

ui_enkel = widgets.VBox([
    widgets.HBox([operasjon_dd, antall_slider, desimaler_slider]),
    hjelp_html,
    inputs_box,
    widgets.HBox([beregn_btn, nullstill_btn]),
    output_area,
    historikk
])

# -----------------------------
# UI: Brøkmodus (teller/nevner)
# -----------------------------

antall_teller = widgets.IntSlider(value=2, min=1, max=10, step=1, description="Antall i teller:", continuous_update=False)
antall_nevner = widgets.IntSlider(value=1, min=1, max=10, step=1, description="Antall i nevner:", continuous_update=False)
desimaler_slider2 = widgets.IntSlider(value=2, min=0, max=10, step=1, description="Desimaler:")

teller_box = widgets.VBox([])
nevner_box = widgets.VBox([])
beregn_btn2 = widgets.Button(description="Beregn brøk", button_style="success")
nullstill_btn2 = widgets.Button(description="Nullstill", button_style="warning")
output_area2 = widgets.Output()
historikk2 = widgets.HTML("<b>Historikk (brøk):</b><br>")
hjelp_html2 = widgets.HTML(
    "<p class='note'>Teller/nevner kan inneholde uttrykk som <code>3e2</code>, "
    "<code>2.5 * 10^3</code> osv. Komma som desimaltegn er tillatt.</p>"
)

def lag_felt_teller(n):
    felt = []
    for i in range(n):
        tb = widgets.Text(
            description=f"teller {i+1}:",
            placeholder="Eks: 3.2e2, 5 * 10^3",
            layout=widgets.Layout(width="60%")
        )
        felt.append(tb)
    teller_box.children = felt

def lag_felt_nevner(n):
    felt = []
    for i in range(n):
        tb = widgets.Text(
            description=f"nevner {i+1}:",
            placeholder="Eks: 4e-1, 2.5 * 10^2",
            layout=widgets.Layout(width="60%")
        )
        felt.append(tb)
    nevner_box.children = felt

def oppdater_teller(change=None):
    lag_felt_teller(antall_teller.value)

def oppdater_nevner(change=None):
    lag_felt_nevner(antall_nevner.value)

antall_teller.observe(oppdater_teller, names="value")
antall_nevner.observe(oppdater_nevner, names="value")
oppdater_teller(); oppdater_nevner()

def beregn_brok(_):
    output_area2.clear_output()
    teller_tall, nevner_tall = [], []
    feil = []

    for tb in teller_box.children:
        v = parse_scientific_input(tb.value)
        if v is None: feil.append(tb.value)
        else: teller_tall.append(v)

    for tb in nevner_box.children:
        v = parse_scientific_input(tb.value)
        if v is None: feil.append(tb.value)
        else: nevner_tall.append(v)

    with output_area2:
        if feil:
            display(HTML(f"<p class='error'>Ugyldig input: {', '.join([repr(x) for x in feil])}.</p>"))
            return
        if not teller_tall or not nevner_tall:
            display(HTML("<p class='error'>Både teller og nevner må ha minst ett tall.</p>"))
            return

        teller_prod = safe_prod(teller_tall)
        nevner_prod = safe_prod(nevner_tall)

        if nevner_prod == 0:
            display(HTML("<p class='error'>FEIL: Nevneren blir 0. Kan ikke dele på 0.</p>"))
            return

        resultat = teller_prod / nevner_prod

        des = desimaler_slider2.value
        latex_std, a, n = til_standardform(resultat, desimaler=des)

        teller_str = " \\times ".join([f"{x}" for x in teller_tall])
        nevner_str = " \\times ".join([f"{x}" for x in nevner_tall])

        latex_beregning = f"\\frac{{{teller_str}}}{{{nevner_str}}} = {latex_std}"

        display(HTML("<div class='calc-section'><hr><h3>Resultat (brøk)</h3></div>"))
        display(latex_wrapper(latex_beregning))
        display(HTML(f"<p>(Dette betyr <code>{a}</code> ganget med <code>10</code> opphøyd i <code>{n}</code>.)</p>"))

        historikk2.value += f"{latex_beregning}<br>"

def nullstill_brok(_):
    for tb in teller_box.children:
        tb.value = ""
    for tb in nevner_box.children:
        tb.value = ""
    output_area2.clear_output()

beregn_btn2.on_click(beregn_brok)
nullstill_btn2.on_click(nullstill_brok)

ui_brok = widgets.VBox([
    widgets.HBox([antall_teller, antall_nevner, desimaler_slider2]),
    hjelp_html2,
    widgets.HTML("<b>Teller:</b>"),
    teller_box,
    widgets.HTML("<b>Nevner:</b>"),
    nevner_box,
    widgets.HBox([beregn_btn2, nullstill_btn2]),
    output_area2,
    historikk2
])

# -----------------------------
# Samlet visning i faner
# -----------------------------
tabs = widgets.Tab(children=[ui_enkel, ui_brok])
tabs.set_title(0, "Enkel modus")
tabs.set_title(1, "Brøkmodus")

display(HTML("<h2>Standardform-kalkulator (LaTeX/MathJax)</h2>"))
display(tabs)

In [ ]:
# Generell kvadratrotgrad ved bruker input
tall_str = input("Skriv inn et positivt tall: ")
rot_str = input("Skriv inn rotgraden (f.eks. 2 for kvadratrot, 3 for kubikkrot): ")

tall = float(tall_str)
rot = float(rot_str)

svar = tall**(1/rot)

print(f"{rot}. roten av {tall} er: {svar}")

In [ ]:
# Angi tallet og rotgraden du vil beregne
tall = 625.0
rot = 4.0

# Beregn resultatet
svar = tall**(1/rot)

# Skriv ut svaret
print(f"{rot}. roten av {tall} er: {svar}")

In [ ]:
# Positive og negative eksponenter
print(10**4)
print(10**3)
print(10**2)
print(10**1)

print(10**0)

print(10**-1)
print(10**-2)
print(10**-3)
print(10**-4)

In [ ]:
# Regner ut en likning på høyre og venstre side av = tegnet
import sympy
from sympy import sympify, solve, symbols, Eq
import math # For å sjekke om input er et tall

def solve_equation_with_precision():
    """
    Ber brukeren om en ligning, løser den ved hjelp av SymPy,
    og skriver ut løsningene. Numeriske løsninger kan vises med spesifisert presisjon.
    Støtter flere variabler og løkker for flere forsøk.
    """
    print("\n--- Løs Ligninger ---")
    print("Skriv inn ligningen du vil løse (f.eks. 'x**2 - 3 = 2*x' eller 'a*x + b = 0').")
    print("For numeriske svar, kan du skrive inn ønsket desimalpresisjon etter ligningen, f.eks. 'x^2 = 2, 2' for 2 desimaler.")
    print("Skriv 'avslutt' for å avslutte programmet.")

    while True:
        user_input = input("\n> Skriv inn ligningen (evt. med desimalpresisjon): ").strip()

        if user_input.lower() == 'avslutt':
            print("Avslutter ligningsløseren. Ha en fin dag!")
            break

        # Sjekk for desimalpresisjon spesifisert med komma (f.eks. "x^2 = 2, 3")
        precision = None
        equation_str_to_solve = user_input
        if ',' in user_input:
            parts = user_input.split(',')
            try:
                # Sjekk om den siste delen er et gyldig heltall for presisjon
                potential_precision = int(parts[-1].strip())
                if potential_precision >= 0:
                    precision = potential_precision
                    equation_str_to_solve = ','.join(parts[:-1]).strip() # Ligningen er alt før siste komma
                else:
                    print("⚠️ Ugyldig presisjon: Presisjon må være et positivt heltall eller null.")
                    continue
            except ValueError:
                # Hvis det ikke er et tall etter komma, behandle det som en del av ligningen
                pass

        if '=' not in equation_str_to_solve:
            print("⚠️ Feil: Ligningen må inneholde et '=' tegn.")
            continue

        try:
            left_str, right_str = equation_str_to_solve.split('=', 1)

            # Prøv å sympify uttrykkene for å finne alle symboler
            all_symbols = set()
            try:
                # Bruk en midlertidig sympify for å hente symboler uten å løse ennå
                left_expr_temp = sympify(left_str)
                right_expr_temp = sympify(right_str)
                all_symbols.update(left_expr_temp.free_symbols)
                all_symbols.update(right_expr_temp.free_symbols)
            except sympy.SympifyError:
                raise ValueError("Kunne ikke tolke uttrykket. Sjekk syntaksen din.")

            if not all_symbols:
                # Hvis ingen variabler, sjekk om det er en ren numerisk ligning (f.eks. "2+2=4")
                try:
                    left_val = sympify(left_str).evalf()
                    right_val = sympify(right_str).evalf()
                    if left_val == right_val:
                        print(f"✅ Ligningen '{equation_str_to_solve}' er sann (f.eks. {left_val} = {right_val}).")
                    else:
                        print(f"🚫 Ligningen '{equation_str_to_solve}' er usann (f.eks. {left_val} ≠ {right_val}).")
                except Exception: # Håndter tilfeller som "2 + a = 4" der 'a' ikke er definert
                    print("⚠️ Feil: Ingen variabler funnet i ligningen, og det er ikke en gyldig numerisk sammenligning. Kanskje en variabel ble stavet feil?")
                continue

            # Hvis det er flere variabler, la brukeren velge for å løse
            var_to_solve_for = None
            if len(all_symbols) == 1:
                var_to_solve_for = all_symbols.pop()
            else:
                # Spør brukeren hvilken variabel de vil løse for
                print(f"Flere variabler funnet: {', '.join(str(s) for s in all_symbols)}.")
                while var_to_solve_for is None:
                    choice = input(f"For hvilken variabel vil du løse? (f.eks. '{list(all_symbols)[0]}'): ").strip()
                    if choice in [str(s) for s in all_symbols]:
                        var_to_solve_for = symbols(choice)
                    else:
                        print(f"Ugyldig valg. Velg en av {', '.join(str(s) for s in all_symbols)}.")


            # Opprett en dictionary med symboler for sympify (viktig for å gjenkjenne egendefinerte symboler)
            symbol_dict = {str(s): s for s in all_symbols}
            
            left_expr = sympify(left_str, locals=symbol_dict)
            right_expr = sympify(right_str, locals=symbol_dict)

            equation = Eq(left_expr, right_expr)

            solutions = solve(equation, var_to_solve_for)

            # --- Formater og skriv ut løsningene ---
            if solutions is None:
                print(f"🚫 Ingen løsninger funnet for '{equation_str_to_solve}'. Ligningen er inkonsistent.")
            elif isinstance(solutions, list):
                if not solutions:
                    print(f"🚫 Ingen løsninger funnet for '{equation_str_to_solve}'.")
                else:
                    print(f"✅ Løsning(er) for '{equation_str_to_solve}' for '{var_to_solve_for}':")
                    for sol in solutions:
                        if precision is not None:
                            try:
                                # evalf() konverterer til numerisk, n=presisjon angir signifikante siffer, ikke desimaler direkte.
                                # For å få eksakte desimaler bruker vi f-string formatering etterpå.
                                numerical_sol = sol.evalf()
                                print(f"   {var_to_solve_for} ≈ {numerical_sol:.{precision}f} (numerisk med {precision} desimaler)")
                            except Exception: # Kan feile hvis sol ikke er et tall (f.eks. Infinity)
                                print(f"   {var_to_solve_for} = {sol} (kan ikke vises numerisk med {precision} desimaler)")
                        else:
                            print(f"   {var_to_solve_for} = {sol}")
            elif solutions is True:
                print(f"♾️ Uendelig mange løsninger. Ligningen er en identitet.")
            else:
                print(f"✅ Løsning(er) for '{equation_str_to_solve}' for '{var_to_solve_for}': {solutions}")

        except sympy.SympifyError as e:
            print(f"⚠️ Feil ved parsing av uttrykket: {e}. Sjekk syntaksen (f.eks. bruk '*' for multiplikasjon, '**' for potens).")
        except ValueError as e:
            print(f"⚠️ Feil: {e}")
        except Exception as e:
            print(f"🚫 En uventet feil oppstod: {e}. Rapporter dette gjerne til utvikleren.")

# Kall funksjonen for å starte programmet
if __name__ == "__main__":
    solve_equation_with_precision()

In [ ]:
# Definer en funksjon
import sympy
from sympy import sympify, symbols, solve, diff, Eq
import math # For is_number_like check

def is_number_like(s):
    """Sjekker om en streng kan tolkes som et tall."""
    try:
        float(s)
        return True
    except ValueError:
        return False

def function_analyzer():
    """
    En interaktiv funksjonsløser som lar brukeren definere en funksjon,
    evaluere den, derivere den, og finne nullpunkter, med mulighet for numerisk presisjon.
    """
    print("\n--- Funksjonsanalysator ---")
    print("Skriv inn funksjonsuttrykket (f.eks. 'x**2 - 3*x + 2').")
    print("Standard variabel er 'x'. Andre variabler kan brukes, f.eks. 't**2' for f(t).")
    print("Skriv 'avslutt' for å avslutte. Skriv 'ny funksjon' for å endre funksjonen.")

    current_function_str = None
    f_expr = None
    variable = symbols('x') # Standard variabel

    while True:
        if current_function_str is None:
            func_input = input("\n> Definer din funksjon f(x) (f.eks. 'x**2 - 3'): ").strip()
            if func_input.lower() == 'avslutt':
                print("Avslutter funksjonsanalysatoren. Ha en fin dag!")
                break
            
            try:
                # Sympyify for å finne frie symboler og definere funksjonen
                f_expr = sympify(func_input)
                
                # Finn variabler. Hvis det er flere, spør brukeren hvilken som er hovedvariabelen
                free_syms = f_expr.free_symbols
                if len(free_syms) == 0:
                    print("⚠️ Feil: Ingen variabler funnet i funksjonen. Dette er en konstant funksjon.")
                    # Vi kan likevel fortsette med en konstant, bare med standard 'x'
                    variable = symbols('x') 
                elif len(free_syms) == 1:
                    variable = free_syms.pop()
                else:
                    print(f"Flere variabler funnet: {', '.join(str(s) for s in free_syms)}.")
                    while True:
                        choice = input(f"Hvilken er hovedvariabelen (f.eks. '{list(free_syms)[0]}')? ").strip()
                        if choice in [str(s) for s in free_syms]:
                            variable = symbols(choice)
                            break
                        else:
                            print(f"Ugyldig valg. Velg en av {', '.join(str(s) for s in free_syms)}.")
                
                current_function_str = func_input
                print(f"✅ Funksjonen din er nå f({variable}) = {f_expr}")

            except sympy.SympifyError as e:
                print(f"⚠️ Feil ved parsing av funksjonen: {e}. Sjekk syntaksen.")
                continue
            except Exception as e:
                print(f"🚫 En uventet feil ved definisjon av funksjon: {e}")
                continue

        print(f"\nFunksjon: f({variable}) = {f_expr}")
        print("Hva vil du gjøre? (evaluer [verdi], derivert, nullpunkter, ny funksjon, avslutt)")
        print("For numeriske svar, legg til ', [desimaler]' etter kommandoen (f.eks. 'evaluer 2, 3')")
        
        command_input = input("\n> Kommando: ").strip().lower()
        
        # Sjekk for desimalpresisjon
        precision = None
        command_parts = command_input.split(',')
        if len(command_parts) > 1:
            try:
                potential_precision = int(command_parts[-1].strip())
                if potential_precision >= 0:
                    precision = potential_precision
                    command_input = command_parts[0].strip()
                else:
                    print("⚠️ Ugyldig presisjon: Presisjon må være et positivt heltall eller null.")
                    continue
            except ValueError:
                pass # Ikke en presisjon, fortsett med hele kommandoen

        if command_input == 'avslutt':
            print("Avslutter funksjonsanalysatoren. Ha en fin dag!")
            break
        elif command_input == 'ny funksjon':
            current_function_str = None # Tilbakestill for å be om ny funksjon
            f_expr = None
            continue
        elif command_input.startswith('evaluer'):
            try:
                _, value_str = command_input.split(' ', 1)
                value = sympify(value_str) # Bruk sympify for å tolke verdien (f.eks. 'sqrt(2)' eller 'pi')
                result = f_expr.subs(variable, value) # Subs erstatter variabelen med verdien
                
                if precision is not None:
                    numerical_result = result.evalf()
                    print(f"✅ f({value}) ≈ {numerical_result:.{precision}f} (numerisk med {precision} desimaler)")
                else:
                    print(f"✅ f({value}) = {result} (symbolsk)")
            except (ValueError, IndexError):
                print("⚠️ Feil: Bruk 'evaluer [verdi]' (f.eks. 'evaluer 5' eller 'evaluer pi').")
            except sympy.SympifyError as e:
                print(f"⚠️ Feil ved evaluering av verdien: {e}. Sjekk syntaksen for verdien.")
            except Exception as e:
                print(f"🚫 En feil oppstod under evaluering: {e}")

        elif command_input == 'derivert':
            try:
                derivative = diff(f_expr, variable)
                print(f"✅ f'({variable}) = {derivative} (symbolsk derivert)")
                if precision is not None:
                    # Hvis ønsket presisjon, evaluer den deriverte ved et punkt (trenger et punkt)
                    print("⚠️ For numerisk derivert, bruk 'evaluer [verdi]' med den deriverte funksjonen som f(x).")
            except Exception as e:
                print(f"🚫 En feil oppstod under derivasjon: {e}")

        elif command_input == 'nullpunkter':
            try:
                solutions = solve(Eq(f_expr, 0), variable) # Løs f(x) = 0
                if solutions is None:
                    print(f"🚫 Ingen nullpunkter funnet for f({variable}) = {f_expr}.")
                elif isinstance(solutions, list):
                    if not solutions:
                        print(f"🚫 Ingen nullpunkter funnet for f({variable}) = {f_expr}.")
                    else:
                        print(f"✅ Nullpunkt(er) for f({variable}) = {f_expr}:")
                        for sol in solutions:
                            if precision is not None:
                                try:
                                    numerical_sol = sol.evalf()
                                    print(f"   {variable} ≈ {numerical_sol:.{precision}f} (numerisk med {precision} desimaler)")
                                except Exception:
                                    print(f"   {variable} = {sol} (kan ikke vises numerisk med {precision} desimaler)")
                            else:
                                print(f"   {variable} = {sol}")
                elif solutions is True:
                     print(f"♾️ Uendelig mange nullpunkter (funksjonen er identisk null).")
                else:
                    print(f"✅ Nullpunkt(er) for f({variable}) = {f_expr}: {solutions}")

            except Exception as e:
                print(f"🚫 En feil oppstod under søk etter nullpunkter: {e}")

        else:
            print("⚠️ Ukjent kommando. Prøv 'evaluer', 'derivert', 'nullpunkter', 'ny funksjon' eller 'avslutt'.")

# Start funksjonsanalysatoren
if __name__ == "__main__":
    function_analyzer()

In [ ]:
# Fra en funksjon regn ut overtidslønn og lønn
def L(x,y):
    return 160*x+240*y

print("Når Anna jobber 52 timer med vanlig timelønn og 50 timer med overtidslønn, tjener Anna totalt L(52,50) =", L(52,50),"kr denne uken")

In [ ]:
# Tabell med (x,y) verdier

x = 5
y = 4*x + 3 # Funksjonen 

print("Når x-verdien er", x,", er y-verdien lik",y)

print("x \t y") 

for x in range(11):
    y = 4*x + 3  # Funksjonen
    print(x, "\t", y)

In [ ]:
# En annen måte å lage en tabell med (x,y) verdier

def f(x):
   y = 4*x + 3
   return y

print("Når x-verdien er", 5,", er f(x) - verdien:", f(5)) 

# Next, a loop
for x in range(11):
    print("Når x-verdien er", x, "er f(x) - verdien:\t", f(x)) 

In [ ]:
# Funksjon for å konvertere et tall til standardform - Lysfart
def til_standardform(tall):
    return "{:.2e}".format(tall)

# Gitte verdier
frekvens = 4.0 * 10**14  # Hz
bolgelengde = 7.5 * 10**-7  # m

# Beregn lysfarten
lysfart = frekvens * bolgelengde

# Konverter lysfarten til standardform
standardform_lysfart = til_standardform(lysfart)

print(f"Lysfarten er {standardform_lysfart} m/s.")

In [ ]:
# Funksjon for å konvertere et tall til standardform - Avstand til galakser
def til_standardform(tall):
    return "{:.2e}".format(tall)

# Gitte verdier
fart = 1.2 * 10**6  # m/s
hubbles_konstant = 2.27 * 10**-18  # s^-1

# Beregn avstanden
avstand = fart / hubbles_konstant

# Konverter avstanden til standardform
standardform_avstand = til_standardform(avstand)

print(f"Avstanden til galaksen er {standardform_avstand} meter ifølge Hubbles lov.")

In [ ]:
# Lager en rett linje i ett koordinatsystem
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = -10
ymax = 10
points = 2*(xmax-xmin) 
x = np.linspace(xmin, xmax, points)

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis
plt.grid()

y = 2*x +1
plt.plot(x,y, 'red')

plt.show()

In [ ]:
# Forholdet mellom en kule og en terning
import math

# Funksjon for å beregne volumet av ei kule med diameter a
def volum_av_kule(diameter):
    radius = diameter / 2
    volum = (4/3) * math.pi * (radius**3)
    return volum

# Funksjon for å beregne volumet av en terning med sidekanter a
def volum_av_terning(sidekant):
    volum = sidekant**3
    return volum

# Funksjon for å beregne forholdet mellom volumet av ei kule og volumet av en terning
def beregn_forhold(diameter):
    volum_kule = volum_av_kule(diameter)
    volum_terning = volum_av_terning(diameter)
    forhold = volum_kule / volum_terning
    return forhold

# Gitt diameter a
a = 1  # Du kan endre denne verdien til et hvilket som helst positivt tall

# Beregn forholdet
forhold = beregn_forhold(a)

print(f"Forholdet mellom volumet av ei kule med diameter {a} og volumet av en terning med sidekanter {a} er {forhold:.2f}.")

# $\color{orange}{\text{Kapittel 3 - Figurer / Formler og likninger/ Proporsjonale størrelser}}$

In [ ]:
# Program som regner ut fart, avstand og tid ved å spørre brukeren om 2 av disse
from decimal import Decimal, getcontext
from math import gcd

getcontext().prec = 10  # Sett presisjon for desimaler

def konverter_til_sekunder(tid, enhet):
    enheter = {
        "år": 365 * 24 * 3600,
        "måneder": 30 * 24 * 3600,
        "uker": 7 * 24 * 3600,
        "dager": 24 * 3600,
        "timer": 3600,
        "minutter": 60,
        "sekunder": 1
    }
    return tid * enheter.get(enhet, None)

def konverter_til_meter(avstand, enhet):
    enheter = {
        "km": 1000,
        "m": 1,
        "cm": 0.01,
        "mm": 0.001,
        "mil": 1609.34,
        "nautiske mil": 1852
    }
    return avstand * enheter.get(enhet, None)

def konverter_til_hastighet(hastighet, enhet):
    enheter = {
        "km/t": 1000 / 3600,
        "m/s": 1,
        "mil/t": 1609.34 / 3600,
        "knop": 1852 / 3600
    }
    return hastighet * enheter.get(enhet, None)

def beregn_manglende_verdi(avstand=None, tid=None, hastighet=None):
    if avstand is not None and tid is not None:
        return avstand / tid  # Hastighet
    elif avstand is not None and hastighet is not None:
        return avstand / hastighet  # Tid
    elif tid is not None and hastighet is not None:
        return tid * hastighet  # Avstand
    return None

def be_om_tall(prompt):
    while True:
        try:
            return float(input(prompt).strip())
        except ValueError:
            print("Ugyldig inndata. Vennligst skriv inn et tall.")

def be_om_enhet(prompt, gyldige_enheter):
    while True:
        enhet = input(prompt).strip().lower()
        if enhet in gyldige_enheter:
            return enhet
        print(f"Ugyldig enhet. Gyldige alternativer: {', '.join(gyldige_enheter)}")

def be_om_input():
    avstand, tid, hastighet = None, None, None
    
    choice = input("Hva vil du beregne? (1: strekning, 2: fart, 3: tid, q: avslutt): ").strip().lower()
    if choice == 'q':
        print("Programmet avsluttes.")
        return None, None, None
    
    if choice == '1':
        tid = be_om_tall("Oppgi tiden: ")
        tid_enhet = be_om_enhet("Er tiden i år, måneder, uker, dager, timer, minutter eller sekunder? ", ["år", "måneder", "uker", "dager", "timer", "minutter", "sekunder"])
        tid = konverter_til_sekunder(tid, tid_enhet)
        
        hastighet = be_om_tall("Oppgi hastigheten: ")
        hastighet_enhet = be_om_enhet("Er hastigheten i km/t, m/s, mil/t eller knop? ", ["km/t", "m/s", "mil/t", "knop"])
        hastighet = konverter_til_hastighet(hastighet, hastighet_enhet)
        
    elif choice == '2':
        avstand = be_om_tall("Oppgi strekningen: ")
        avstand_enhet = be_om_enhet("Er strekningen i km, m, cm, mm, mil eller nautiske mil? ", ["km", "m", "cm", "mm", "mil", "nautiske mil"])
        avstand = konverter_til_meter(avstand, avstand_enhet)
        
        tid = be_om_tall("Oppgi tiden: ")
        tid_enhet = be_om_enhet("Er tiden i år, måneder, uker, dager, timer, minutter eller sekunder? ", ["år", "måneder", "uker", "dager", "timer", "minutter", "sekunder"])
        tid = konverter_til_sekunder(tid, tid_enhet)
        
    elif choice == '3':
        avstand = be_om_tall("Oppgi strekningen: ")
        avstand_enhet = be_om_enhet("Er strekningen i km, m, cm, mm, mil eller nautiske mil? ", ["km", "m", "cm", "mm", "mil", "nautiske mil"])
        avstand = konverter_til_meter(avstand, avstand_enhet)
        
        hastighet = be_om_tall("Oppgi hastigheten: ")
        hastighet_enhet = be_om_enhet("Er hastigheten i km/t, m/s, mil/t eller knop? ", ["km/t", "m/s", "mil/t", "knop"])
        hastighet = konverter_til_hastighet(hastighet, hastighet_enhet)
        
    return avstand, tid, hastighet

def hovedprogram():
    while True:
        avstand, tid, hastighet = be_om_input()
        if avstand is None and tid is None and hastighet is None:
            break
        
        if [avstand, tid, hastighet].count(None) > 1:
            print("Du må oppgi minst to av verdiene: avstand, tid eller hastighet.")
            continue
        
        resultat = beregn_manglende_verdi(avstand, tid, hastighet)
        if resultat is None:
            print("Noe gikk galt. Sjekk inndataene dine og prøv igjen.")
            continue
        
        if avstand is None:
            print(f"Strekningen er {resultat:.2f} meter.")
        elif tid is None:
            print(f"Tiden er {resultat:.2f} sekunder.")
        elif hastighet is None:
            print(f"Hastigheten er {resultat:.2f} m/s, {(resultat * 3600 / 1000):.2f} km/t, {(resultat * 3600 / 1609.34):.2f} mil/t eller {(resultat * 3600 / 1852):.2f} knop.")

if __name__ == "__main__":
    hovedprogram()

In [ ]:
# Omgjøring mellom Celsius, Farenheit og Kelvin
def celsius_to_fahrenheit(tempC):
    return 1.8 * tempC + 32

def celsius_to_kelvin(tempC):
    return tempC + 273.15

def kelvin_to_celsius(tempK):
    return tempK - 273.15

def fahrenheit_to_celsius(tempF):
    return (tempF - 32) / 1.8

def konverter_temperatur():
    while True:
        print("Velkommen til temperaturkonvertering!")
        print("Velg en enhet for konvertering:")
        print("1. Celsius (C)")
        print("2. Kelvin (K)")
        print("3. Fahrenheit (F)")
        print("Trykk 'q' for å avslutte programmet.\n")

        choice = input("Velg et alternativ (1/2/3/q): ").strip().lower()
        if choice == 'q':
            print("Programmet avsluttes.")
            break

        try:
            temperatur = float(input("Hva er temperaturen? "))
        except ValueError:
            print("Ugyldig temperatur. Vennligst oppgi et tall.")
            continue

        if choice == '1':
            tempF = celsius_to_fahrenheit(temperatur)
            tempK = celsius_to_kelvin(temperatur)
            resultat = f"{temperatur} °C er lik {round(tempF, 1)} °F og {round(tempK, 2)} K"
        elif choice == '2':
            tempC = kelvin_to_celsius(temperatur)
            tempF = celsius_to_fahrenheit(tempC)
            resultat = f"{temperatur} K er lik {round(tempC, 2)} °C og {round(tempF, 1)} °F"
        elif choice == '3':
            tempC = fahrenheit_to_celsius(temperatur)
            tempK = celsius_to_kelvin(tempC)
            resultat = f"{temperatur} °F er lik {round(tempC, 2)} °C og {round(tempK, 2)} K"
        else:
            resultat = "Ugyldig valg. Vennligst oppgi 1, 2 eller 3."

        print(resultat)

if __name__ == "__main__":
    konverter_temperatur()

In [ ]:
# Oppgave 3.32 Summen av de første partallene
partall = 2
n = 1
sum = 0

while n <= 50:
  sum += partall
  print("Summen av de", n, "første partallene er", sum)
  n += 1
  partall += 2

In [ ]:
# Eksempel s. 68-69 med funksjon
def main():
    x1 = 5
    print(f"Her om dagen så jeg {x1} svarte katter krysse veien")

    def f(x):
        return 6 - 0.2 * x

    print(f"f(30) = {f(30):.2f}")

if __name__ == "__main__":
    main()

In [ ]:
# Oppgave 10 Våren 2025 heldagsprøve
def K(V, L):
    return (V * 100) / (L**3)

def determine_form(K_factor):
    if K_factor < 0.9:
        return "Tynn fisk og dårlig form"
    elif 0.9 <= K_factor < 1:
        return "Normal kondisjon"
    elif 1 <= K_factor < 1.2:
        return "Meget fin kondisjon"
    else:
        return "Helt eksepsjonelt"

def main():
    V = 295  # weight in grams
    L = 33   # length in centimeters
    Kondisjon_faktor = K(V, L)
    form = determine_form(Kondisjon_faktor)
    
    print(f" a) Når ørreten veier {V} gram og lengden er {L} centimeter, er K-faktoren {Kondisjon_faktor:.2f} og formen er '{form}'.")

def calculate_weight(K_factor, length):
    return (K_factor * (length ** 3)) / 100

# Gitte verdier
K_factor = 0.90
length = 42

# Beregn vekten
weight = calculate_weight(K_factor, length)

print(f" b) Når ørreten har en K-verdi på {K_factor} og lengden er {length} centimeter, er vekten på fisken {weight:.2f} gram.")

def calculate_length(weight, K_factor):
    return ((weight * 100) / K_factor) ** (1/3)

# Gitte verdier
weight = 1600  # vekt i gram (1,6 kg)
K_factor_min = 1.0
K_factor_max = 1.2

# Beregn de mulige lengdene
length_min = calculate_length(weight, K_factor_max)
length_max = calculate_length(weight, K_factor_min)

print(f" c) Når ørreten veier {weight} gram og K-verdien er mellom {K_factor_min} og {K_factor_max}, kan lengden være mellom {length_min:.1f} cm og {length_max:.1f} cm.")

if __name__ == "__main__":
    main()

In [ ]:
# Eksempel s. 69
def L(x, y):
    return 160 * x + 240 * y

def main():
    timer_vanlig = 52
    timer_overtid = 50
    total_lonn = L(timer_vanlig, timer_overtid)
    print(f"Når Anna jobber {timer_vanlig} timer med vanlig timelønn og {timer_overtid} timer med overtidslønn, tjener Anna totalt {total_lonn:.2f} kr denne uken")

if __name__ == "__main__":
    main()

In [ ]:
# Formelregning
from sympy import symbols, Eq, parse_expr, Symbol, simplify, S, Number as SympyNumber
from sympy.core.relational import Relational
from sympy.solvers import solve
from sympy.solvers.inequalities import solve_univariate_inequality
from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application

# Kompatibilitet for RelationalOp
try:
    from sympy.core.relational import RelationalOp
except ImportError:
    RelationalOp = Relational # For eldre SymPy-versjoner

# Konfigurasjon for parser
transformations = standard_transformations + (implicit_multiplication_application,)

# Hjelpeordbok for symboler som kan kollidere med SymPy-konstanter
_RESERVED_NAMES_AS_SYMBOLS = {name: Symbol(name) for name in ["E", "I", "N", "O", "Q", "S"]}

# -------------------- Hjelpefunksjoner for Parsing --------------------
def custom_parse_expr(expr_str, local_dict_override=None, **kwargs):
    """
    Parser et uttrykk og sikrer at visse navn (E, I, N, O, Q, S)
    behandles som symboler, med mindre annet er spesifisert.
    Bruker standard 'transformations'.
    """
    effective_local_dict = _RESERVED_NAMES_AS_SYMBOLS.copy()
    if local_dict_override: # For tilfeller der vi IKKE vil overstyre (f.eks. verdiparsing)
        effective_local_dict.update(local_dict_override)
    
    if 'transformations' not in kwargs:
        kwargs['transformations'] = transformations
        
    return parse_expr(expr_str, local_dict=effective_local_dict, **kwargs)

# -------------------- Kjernefunksjoner --------------------

def parse_ligning(expr_str):
    """Parser et uttrykk pa formen 'venstre = hoyre' til en sympy-ligning."""
    venstre, hoyre = expr_str.split('=', maxsplit=1)
    return Eq(custom_parse_expr(venstre.strip()),
              custom_parse_expr(hoyre.strip()))

def løs_uttrykk(uttrykk_str):
    """Loser en ligning, ulikhet eller system av ligninger."""
    try:
        if ';' in uttrykk_str:
            ligninger = [parse_ligning(eq.strip()) for eq in uttrykk_str.split(';')]
            return solve(ligninger, dict=True)

        is_potential_inequality = any(op in uttrykk_str for op in ['<', '>', '<=', '>='])
        is_assignment_like = '=' in uttrykk_str and not any(op in uttrykk_str for op in ['<=', '>=', '!=', '=='])


        if is_potential_inequality and not is_assignment_like:
            if " & " in uttrykk_str or " | " in uttrykk_str:
                 return "X Sammensatte ulikheter med '&' eller '|' stottes ikke direkte. Prov en ulikhet."
            
            if '==' in uttrykk_str: 
                pass
            else:
                ulikhet = custom_parse_expr(uttrykk_str) 
                if not isinstance(ulikhet, (Relational, RelationalOp)):
                    return f"X Uttrykket '{uttrykk_str}' er ikke en gyldig ulikhetsstruktur."
                variabler = sorted(list(ulikhet.free_symbols), key=lambda s: s.name)
                if not variabler:
                    simplified_truth_value = simplify(ulikhet)
                    if simplified_truth_value == S.true: return S.Reals
                    if simplified_truth_value == S.false: return S.EmptySet
                    return f"Symbolsk konstant ulikhet: {ulikhet}"
                hoved_var = variabler[0]
                return solve_univariate_inequality(ulikhet, hoved_var, relational=False)

        if '==' in uttrykk_str:
             lhs, rhs = uttrykk_str.split('==', 1)
             ligning = Eq(custom_parse_expr(lhs.strip()), custom_parse_expr(rhs.strip()))
        elif '=' not in uttrykk_str: 
            parsed_lhs = custom_parse_expr(uttrykk_str)
            ligning = Eq(parsed_lhs, 0)
        else: 
            ligning = parse_ligning(uttrykk_str)
        
        return solve(ligning, dict=True)
    except Exception as e:
        return f"X Feil under losning: {e}"

def evaluer_uttrykk(uttrykk_str, kjente_verdier, symbolsk=True):
    """Evaluerer et uttrykk (eller hoyreside av en likning) med gitte verdier."""
    try:
        expr_to_parse = uttrykk_str
        if '=' in uttrykk_str and '==' not in uttrykk_str :
            parts = uttrykk_str.split('=', maxsplit=1)
            if not (parts[0].endswith('<') or parts[0].endswith('>') or parts[0].endswith('!')):
                 _, hoyre_side_str = parts
                 expr_to_parse = hoyre_side_str.strip()

        parsed_uttrykk = custom_parse_expr(expr_to_parse)
        
        subs_dict = {}
        for s in parsed_uttrykk.free_symbols:
            if s.name in kjente_verdier:
                subs_dict[s] = kjente_verdier[s.name]

        evaluert_uttrykk = parsed_uttrykk.subs(subs_dict)

        is_numeric_evaluable = hasattr(evaluert_uttrykk, 'is_Number') and evaluert_uttrykk.is_Number
        if not is_numeric_evaluable: 
             is_numeric_evaluable = hasattr(evaluert_uttrykk, 'is_number') and evaluert_uttrykk.is_number
        if not is_numeric_evaluable:
            is_numeric_evaluable = isinstance(evaluert_uttrykk, SympyNumber) or not evaluert_uttrykk.free_symbols

        if not symbolsk and is_numeric_evaluable:
            resultat = evaluert_uttrykk.evalf()
        else:
            resultat = evaluert_uttrykk
        
        return resultat
    except Exception as e:
        return f"X Feil under evaluering: {e}"

def løs_for_variabel(uttrykk_str, mål_variabel_navn, kjente_verdier):
    """Loser en ukjent gitt kjente verdier."""
    try:
        eq_str = uttrykk_str.split(';')[0].strip()
        
        if '==' in eq_str:
            lhs, rhs = eq_str.split('==', 1)
            ligning = Eq(custom_parse_expr(lhs.strip()), custom_parse_expr(rhs.strip()))
        elif '=' in eq_str:
            ligning = parse_ligning(eq_str) 
        else:
            ligning = Eq(custom_parse_expr(eq_str.strip()),0)

        mål_symbol = Symbol(mål_variabel_navn)
        
        subs_for_eq = {}
        for s in ligning.free_symbols:
            if s.name in kjente_verdier and s.name != mål_variabel_navn:
                 subs_for_eq[s] = kjente_verdier[s.name]

        substituert_ligning = ligning.subs(subs_for_eq)
        
        if mål_symbol not in substituert_ligning.free_symbols:
            if hasattr(substituert_ligning, 'lhs') and hasattr(substituert_ligning, 'rhs'):
                simplified_eq_check = simplify(substituert_ligning.lhs - substituert_ligning.rhs)
                if simplified_eq_check == 0: 
                    return f"Ligningen er alltid sann for de gitte verdiene. '{mål_variabel_navn}' kan vaere hva som helst (eller ikke relevant)."
                elif not substituert_ligning.free_symbols: 
                     return "Ligningen er usann/en selvmotsigelse for de gitte verdiene. Ingen losning."
            return f"Variabelen '{mål_variabel_navn}' finnes ikke i ligningen etter substitusjon, eller ligningen er ikke avhengig av den."

        return solve(substituert_ligning, mål_symbol)
    except Exception as e:
        return f"X Feil under isolering: {e}"

def hent_kjente_verdier():
    """Spor brukeren om variableverdier i formatet x=3, y=pi/2."""
    raw_input_str = input("Skriv inn kjente verdier (f.eks. x=3, y=pi/2, z=sqrt(2)):\n> ")

    try:
        verdier = {}
        if raw_input_str.strip() == "": return verdier
        for item in raw_input_str.split(','):
            key_val_pair = item.strip().split('=', maxsplit=1)
            if len(key_val_pair) != 2:
                print(f"Advarsel: Ugyldig format for '{item.strip()}'. Hopper over.")
                continue
            key, val_str = key_val_pair
            key = key.strip()
            val_str = val_str.strip()

            try:
                parsed_val = parse_expr(val_str, transformations=transformations, evaluate=True, local_dict={})
            except SyntaxError: 
                try:
                    parsed_val = float(val_str) 
                except ValueError:
                    try:
                        parsed_val = int(val_str) 
                    except ValueError:
                        print(f"Advarsel: Kunne ikke parse verdien '{val_str}' for '{key}'. Hopper over.")
                        continue
            
            is_num_type = isinstance(parsed_val, (int, float))
            is_sympy_num_obj = hasattr(parsed_val, 'is_Number') and parsed_val.is_Number
            if not is_sympy_num_obj: 
                is_sympy_num_obj = hasattr(parsed_val, 'is_number') and parsed_val.is_number
            
            if is_num_type and not is_sympy_num_obj : 
                 verdier[key] = SympyNumber(parsed_val)
            else: 
                verdier[key] = parsed_val
        return verdier
    except Exception as e:
        print(f"Advarsel: Ugyldig format for kjente verdier ({e}). Prov igjen med f.eks. x=3, y=pi/2.")
        return hent_kjente_verdier()

# -------------------- Hovedprogram --------------------
def main():
    print("Formel- og uttrykksloser med SymPy")
    print("Skriv 'q' nar som helst for a avslutte programmet.\n")

    if not hasattr(SympyNumber, 'is_Number') and hasattr(SympyNumber, 'is_number'):
        SympyNumber.is_Number = property(lambda self: self.is_number)
    elif not hasattr(SympyNumber, 'is_Number'): 
         SympyNumber.is_Number = property(lambda self: isinstance(self, SympyNumber))


    while True:
        uttrykk_str_input = input("Skriv inn et uttrykk, en ligning, ulikhet, eller system (separert med ';'):\n> ")
        if uttrykk_str_input.lower() == 'q':
            print("Avslutter programmet. Ha en fin dag!")
            break

        handling = input("Velg handling:\n1 = Evaluer uttrykk\n2 = Los ligning(er)/ulikhet\n3 = Isoler en variabel\n(q for a avslutte)\n> ")
        if handling.lower() == 'q':
            print("Avslutter programmet. Ha en fin dag!")
            break

        if handling == '1':
            verdier_input = hent_kjente_verdier()
            resultat = evaluer_uttrykk(uttrykk_str_input, verdier_input, symbolsk=False)
            
            if isinstance(resultat, str) and resultat.startswith("X"): print(resultat)
            elif hasattr(resultat, 'evalf'): 
                try:
                    num_val = resultat.evalf() 
                    if hasattr(num_val, 'is_Integer') and num_val.is_Integer:
                        print(f"Resultat: {int(num_val)}")
                    elif (hasattr(num_val, 'is_Float') and num_val.is_Float) or \
                         (hasattr(num_val, 'is_Rational') and num_val.is_Rational) or \
                         (hasattr(num_val, 'as_real_imag')): 
                        try:
                            py_float_val = float(num_val)
                            if py_float_val == int(py_float_val): 
                                print(f"Resultat: {int(py_float_val)}")
                            else:
                                print(f"Resultat: {py_float_val:.2f}") 
                        except (TypeError, ValueError, OverflowError): 
                             print(f"Resultat: {num_val}") 
                    else: 
                        print(f"Resultat: {num_val}")

                except (TypeError, AttributeError, ValueError): 
                    print(f"Resultat: {resultat}") 
            else: 
                print(f"Resultat: {resultat}")


        elif handling == '2':
            resultat = løs_uttrykk(uttrykk_str_input)
            print("Losning(er):", resultat)

        elif handling == '3':
            verdier_input = hent_kjente_verdier()
            mål_input = input("Hvilken variabel onsker du a isolere/lose for?\n> ")
            resultat = løs_for_variabel(uttrykk_str_input, mål_input, verdier_input)
            
            if isinstance(resultat, str) and resultat.startswith("X"): print(resultat)
            elif isinstance(resultat, str): print(f"Info: {resultat}") 
            elif isinstance(resultat, list):
                if not resultat: print(f"Ingen losning funnet for {mål_input}.")
                elif len(resultat) == 1: print(f"Isolert losning for {mål_input}: {resultat[0]}")
                else: print(f"Isolerte losninger for {mål_input}: {resultat}")
            else: print(f"Uventet resultat: {resultat}")
        else:
            print("Advarsel: Ugyldig valg. Prov igjen.")
        print("\n----------------------------\n")

if __name__ == "__main__":
    main()

In [2]:
# Avansert likningsløser
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np

def solve_equation(left_side, right_side, symbols_to_solve_for):
    """Løser en ligning for de gitte symbolene."""
    try:
        left_expr = sp.sympify(left_side)
        right_expr = sp.sympify(right_side)
    except sp.SympifyError:
        print("Kunne ikke tolke ett eller begge uttrykkene. Kontroller parenteser og skrivemåte.")
        return None, None, None, None

    equation = sp.Eq(left_expr, right_expr)

    try:
        if not symbols_to_solve_for:
            return equation, left_expr, right_expr, []

        solution = sp.solve(equation, *symbols_to_solve_for, dict=True)
        return equation, left_expr, right_expr, solution
    except NotImplementedError:
        print(f"SymPy kunne ikke finne en algebraisk løsning for denne typen ligning.")
        return equation, left_expr, right_expr, None
    except Exception as e:
        print(f"En feil oppstod under løsning: {e}")
        return None, None, None, None


def plot_equation(left_expr, right_expr, var, solution_points=None):
    """Tegner grafen for venstre og høyre side av ligningen og markerer løsningspunkter."""
    x_vals = np.linspace(-10, 10, 1000)

    try:
        f_left = sp.lambdify(var, left_expr, modules=['numpy', {'sqrt': np.lib.scimath.sqrt}])
        f_right = sp.lambdify(var, right_expr, modules=['numpy', {'sqrt': np.lib.scimath.sqrt}])
    except Exception as e:
        print(f"Kunne ikke lage plottbare funksjoner: {e}")
        return

    # Evaluer y-verdier
    y_left_complex = f_left(x_vals)
    y_right_complex = f_right(x_vals)

    # ### FIKS: Sikrer at y-verdiene er arrays, selv om uttrykket er en konstant.
    if not isinstance(y_left_complex, np.ndarray):
        y_left_complex = np.full_like(x_vals, y_left_complex, dtype=float)
    if not isinstance(y_right_complex, np.ndarray):
        y_right_complex = np.full_like(x_vals, y_right_complex, dtype=float)

    # Plotter kun den reelle delen for å unngå feil med komplekse tall.
    y_left = np.real(y_left_complex)
    y_right = np.real(y_right_complex)
    
    if np.any(np.iscomplex(y_left_complex)) or np.any(np.iscomplex(y_right_complex)):
        print("Advarsel: Komplekse tall ble generert under plotting. Viser kun den reelle delen.")

    plt.figure(figsize=(12, 7))
    plt.plot(x_vals, y_left, label=f"Venstre side: ${sp.latex(left_expr)}$", color='blue')
    plt.plot(x_vals, y_right, label=f"Høyre side: ${sp.latex(right_expr)}$", color='red')

    if solution_points:
        # Filtrer ut ikke-reelle løsninger før plotting
        real_solutions = [sol for sol in solution_points if all(not val.has(sp.I) for val in sol.values())]
        for sol_dict in real_solutions:
            if var in sol_dict:
                try:
                    x_val_sol = float(sol_dict[var])
                    y_val_sol = float(np.real(f_left(x_val_sol)))
                    plt.plot(x_val_sol, y_val_sol, 'ko', markersize=8, label=f"Løsning: x ≈ {x_val_sol:.3f}")
                    plt.annotate(f"({x_val_sol:.3f}, {y_val_sol:.3f})", (x_val_sol, y_val_sol), textcoords="offset points", xytext=(5,5))
                except (TypeError, ValueError, AttributeError) as e:
                    print(f"Kunne ikke plotte løsningspunkt {sol_dict[var]}: {e}")

    plt.title(f"Grafisk fremstilling: ${sp.latex(left_expr)} = {sp.latex(right_expr)}$", fontsize=16)
    plt.xlabel(str(var), fontsize=12)
    plt.ylabel("Verdi", fontsize=12)
    plt.grid(True, which='both', linestyle='--')
    plt.axhline(0, color='black', linewidth=0.7)
    plt.axvline(0, color='black', linewidth=0.7)

    all_finite_y = np.concatenate((y_left[np.isfinite(y_left)], y_right[np.isfinite(y_right)]))
    if all_finite_y.size > 0:
        y_min = np.nanpercentile(all_finite_y, 5)
        y_max = np.nanpercentile(all_finite_y, 95)
        padding = (y_max - y_min) * 0.15
        if padding < 1: padding = 1
        plt.ylim(y_min - padding, y_max + padding)

    plt.legend()
    plt.show()


def prompt_float(prompt_msg):
    """Henter et flyttall fra brukeren på en robust måte."""
    while True:
        value_str = input(prompt_msg).strip()
        if value_str.lower() == 'q':
            return None
        try:
            return float(value_str)
        except ValueError:
            print("Ugyldig tallverdi. Prøv igjen eller skriv 'q' for å avslutte.")


def main():
    """Hovedprogrammet for ligningsløseren."""
    print("=== Avansert Ligningsløser, Evaluator og Graftegner ===")
    while True:
        left_side_str = input("\nSkriv inn venstre side av likningen (eller 'q' for å avslutte): ").strip()
        if left_side_str.lower() == 'q':
            break

        right_side_str = input("Skriv inn høyre side av likningen: ").strip()
        if right_side_str.lower() == 'q':
            break

        try:
            equation_symbols = tuple(sp.sympify(left_side_str).free_symbols.union(sp.sympify(right_side_str).free_symbols))
            if not equation_symbols:
                print("Likningen inneholder ingen variabler.")
            else:
                print(f"Variabler gjenkjent: {', '.join(map(str, equation_symbols))}")
        except sp.SympifyError:
            print("Kunne ikke tolke likningen. Prøv igjen.")
            continue

        eq, left_expr, right_expr, solutions = solve_equation(left_side_str, right_side_str, equation_symbols)

        if eq is None:
            continue

        print(f"\nLikning: {sp.pretty(eq)}")

        if not solutions and len(equation_symbols) == 1:
            var_sym = equation_symbols[0]
            print("Ingen algebraisk løsning funnet.")
            if input(f"Vil du prøve å finne en numerisk løsning for '{var_sym}'? (ja/nei): ").lower() == 'ja':
                start_point = prompt_float("Oppgi et startpunkt/gjett for løsningen (f.eks. 1): ")
                if start_point is not None:
                    try:
                        numerical_eq = left_expr - right_expr
                        nsol = sp.nsolve(numerical_eq, var_sym, start_point)
                        print(f"Numerisk løsning funnet nær {start_point}: {var_sym} = {nsol.evalf(5)}")
                        solutions = [{var_sym: nsol}]
                    except Exception as e:
                        print(f"Kunne ikke finne en numerisk løsning: {e}")

        elif solutions:
            print("Algebraiske løsning(er):")
            for i, sol_dict in enumerate(solutions, 1):
                formatted_sol = []
                for var, val in sol_dict.items():
                    is_complex = any(i.is_imaginary for i in val.atoms(sp.I))
                    if not val.free_symbols:
                        val_str = f"{val.evalf(5)}"
                        if is_complex:
                             val_str += " (kompleks)"
                    else:
                        val_str = str(val)
                    formatted_sol.append(f"{var} = {val_str}")
                print(f"  Løsningssett {i}: {', '.join(formatted_sol)}")
        else:
             print("Ingen løsning funnet for denne ligningen.")

        if equation_symbols:
            main_var_for_plot = None
            if len(equation_symbols) == 1:
                main_var_for_plot = equation_symbols[0]
            else:
                plot_choice = input(f"Vil du plotte ligningen med hensyn på én variabel? (ja/nei): ").lower()
                if plot_choice == 'ja':
                    var_str = input(f"Hvilken variabel vil du ha på x-aksen? ({', '.join(map(str, equation_symbols))}): ").strip()
                    if var_str in map(str, equation_symbols):
                        main_var_for_plot = sp.Symbol(var_str)
                    else:
                        print("Ugyldig variabel for plotting.")
            
            if main_var_for_plot:
                vals_for_plot = {}
                other_vars = [s for s in equation_symbols if s != main_var_for_plot]
                if other_vars:
                    print(f"For å plotte med hensyn på '{main_var_for_plot}', må de andre variablene få en verdi.")
                    for ov in other_vars:
                        val = prompt_float(f"  Verdi for '{ov}': ")
                        if val is None: break
                        vals_for_plot[ov] = val
                    if len(vals_for_plot) != len(other_vars):
                         print("Kan ikke plotte uten verdier for alle andre variabler.")
                    else:
                        plot_left = left_expr.subs(vals_for_plot)
                        plot_right = right_expr.subs(vals_for_plot)
                        plot_equation(plot_left, plot_right, main_var_for_plot, solutions)
                else:
                    plot_equation(left_expr, right_expr, main_var_for_plot, solutions)

        if equation_symbols and input("\nVil du sette inn verdier i likningen for å løse/evaluere? (ja/nei): ").lower() == 'ja':
            values_for_eq = {}
            for sym in equation_symbols:
                val = input(f"  Verdi for '{sym}' (trykk Enter for å hoppe over): ").strip()
                if val:
                    try:
                        values_for_eq[sym] = float(val)
                    except ValueError:
                        print(f"  Ugyldig verdi for {sym}, hopper over.")
            
            remaining_symbols = equation_symbols - set(values_for_eq.keys())
            
            try:
                sub_eq = eq.subs(values_for_eq)
                print(f"Likning etter substitusjon: {sp.pretty(sub_eq)}")
                
                if len(remaining_symbols) == 1:
                    final_sol = sp.solve(sub_eq, remaining_symbols)
                    print(f"Løsning for '{list(remaining_symbols)[0]}': {final_sol}")
                elif not remaining_symbols:
                    is_true = sub_eq.doit()
                    print(f"Likningen er {is_true} med de gitte verdiene.")
                else:
                    print("Fortsatt for mange ukjente til å finne en unik løsning.")
            except Exception as e:
                print(f"En feil oppstod under evaluering: {e}")

    print("\nAvslutter... Ha en fin dag! :)")

if __name__ == "__main__":
    main()

=== Avansert Ligningsløser, Evaluator og Graftegner ===



Skriv inn venstre side av likningen (eller 'q' for å avslutte):  q



Avslutter... Ha en fin dag! :)


In [ ]:
# Søyle, sektor og linjediagram, Farger : https://matplotlib.org/stable/gallery/color/named_colors.html
import matplotlib.pyplot as plt

# Liste med navn og frekvenser
navn = ["1963", "1973", "1983", "1993", "2003", "2013", "2023"]                  # Bytt ut med dine verdier/tall
frekvenser = [24, 23.4, 24.7, 26, 27.9, 28.6, 30.3]                             # Bytt ut med dine verdier/tall

# Fargevalg
farger = ['yellow','coral','green','blue','red','orange']

# Beregn totalfrekvensen
total = sum(frekvenser)

# Beregn andelene (i prosent) for hver sektor
andelene = [100 * frek / total for frek in frekvenser]

# Kakediagram
plt.figure()
plt.pie(andelene, labels=navn, autopct='%1.1f%%', colors=farger)
plt.title("Frekvenser av navn")                                                     # Endre overskrift
plt.axis('equal')                                                                  
plt.show()

# Søylediagram
plt.figure()
plt.bar(navn, frekvenser, color=farger)
plt.title("Frekvenser av navn")                                                     # Endre overskrift
plt.xlabel("Navn")                                                                 # x-akse navn
plt.ylabel("Frekvens")                                                            # y-akse navn
plt.ylim(ymin=0)
plt.show()

# Linjediagram
plt.figure()
plt.plot(navn, frekvenser, marker='o', color='#ff7f0e')
plt.title("Mors fødealder fra 1963 til 2023")                                   # Endre overskrift
plt.xlabel("Årstall")                                                           # x-akse navn
plt.ylabel("Mors fødealder")                                                    # y-akse navn
plt.ylim(ymin=0)
plt.show()

In [ ]:
# Figurtall - bare legg inn antall figurnummerene i koden under her og trykk kjør koden med shift + enter eller play knappen :)import numpy as np
import numpy as np
from scipy.optimize import curve_fit
import warnings
from scipy.optimize import OptimizeWarning

# Eksempel på noen ulike figurtall du kan endre verdiene på
figurtall = [1, 3, 6, 10]  # Endre disse tallene så det matcher antallet for de ulike figurnummerne :)

def lineær_funksjon(x, a, b):
    return a * x + b

def kvadratisk_funksjon(x, a, b, c):
    return a * x**2 + b * x + c

def fjerdegrads_funksjon(x, a, b, c, d, e):
    return a * x**4 + b * x**3 + c * x**2 + d * x + e

def femtegrads_funksjon(x, a, b, c, d, e, f):
    return a * x**5 + b * x**4 + c * x**3 + d * x**2 + e * x + f

def sjettegrads_funksjon(x, a, b, c, d, e, f, g):
    return a * x**6 + b * x**5 + c * x**4 + d * x**3 + e * x**2 + f * x + g

def potens_funksjon(x, a, b):
    return a * x**b

def eksponential_funksjon(x, a, b):
    return a * np.exp(b * x)

def best_fitting_function(figurtall):
    x_data = np.arange(1, len(figurtall) + 1)
    y_data = np.array(figurtall)

    models = [
        (lineær_funksjon, "f(n) = {:.2f}n + {:.2f}", 2),
        (kvadratisk_funksjon, "f(n) = {:.2f}n^2 + {:.2f}n + {:.2f}", 3),
        (fjerdegrads_funksjon, "f(n) = {:.2f}n^4 + {:.2f}n^3 + {:.2f}n^2 + {:.2f}n + {:.2f}", 5),
        (femtegrads_funksjon, "f(n) = {:.2f}n^5 + {:.2f}n^4 + {:.2f}n^3 + {:.2f}n^2 + {:.2f}n + {:.2f}", 6),
        (sjettegrads_funksjon, "f(n) = {:.2f}n^6 + {:.2f}n^5 + {:.2f}n^4 + {:.2f}n^3 + {:.2f}n^2 + {:.2f}n + {:.2f}", 7),
        (potens_funksjon, "f(n) = {:.2f}n^{:.2f}", 2),
        (eksponential_funksjon, "f(n) = {:.2f}e^{:.2f}n", 2)
    ]

    best_model = None
    best_rss = np.inf
    best_params = None
    best_equation = ""

    for func, equation, num_params in models:
        if num_params > len(figurtall):
            continue  # Hopper over modeller med for mange parametere

        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=OptimizeWarning)
                popt, _ = curve_fit(func, x_data, y_data)
                y_fit = func(x_data, *popt)
                rss = np.sum((y_data - y_fit) ** 2)

                if np.all(np.diff(y_fit) >= 0):  # Sjekker om funksjonen er voksende
                    if rss < best_rss:
                        best_rss = rss
                        best_model = func
                        best_params = popt
                        best_equation = equation.format(*popt)

        except Exception:
            continue  # Hopper over modeller som gir feil

    return best_model, best_params, best_equation

# Finn den beste tilpassede funksjonen
funksjon, params, funksjon_str = best_fitting_function(figurtall)

if funksjon:
    print(f"Den beste tilpassede funksjonen er: {funksjon_str}")
    n_values = [5, 6, 20]  # Endre disse tallene for figurnummerene du ønsker å finne
    for figurnummer in n_values:
        figurverdi = funksjon(figurnummer, *params)
        print(f"Figurtallet for figur {figurnummer} er: {int(round(figurverdi))}")
else:
    print("Kunne ikke gjenkjenne en passende funksjon.")

In [ ]:
# Proporsjonale størrelser med ukjente x-verdier og y-verdier
import matplotlib.pyplot as plt
import numpy as np

def are_proportional(list_x, list_y, rtol=1e-9, atol=1e-12):
    """
    Sjekker om to lister er proporsjonale i formen y = k * x.
    Returnerer (True, k) hvis de er det, og (False, None) hvis ikke.
    k er proporsjonalitetskonstanten (k = y/x).

    Regler:
    - (x, y) = (0, 0) er OK og hopper ikke ut (det ligger på linja y = kx for alle k).
    - Hvis x = 0 og y != 0 -> ikke proporsjonalt (kan ikke finne et k som fungerer).
    - Vi finner k fra første par med x != 0.
    """
    # Grunnleggende sjekker
    if len(list_x) != len(list_y) or len(list_x) == 0:
        return False, None

    # Konverter til numpy og sjekk at alt er endelig (ikke NaN/inf)
    x = np.asarray(list_x, dtype=float)
    y = np.asarray(list_y, dtype=float)
    if not (np.isfinite(x).all() and np.isfinite(y).all()):
        return False, None
    # Finn første indeks der x != 0 for å beregne k
    nz = np.where(~np.isclose(x, 0.0, atol=atol, rtol=0))[0]
    if nz.size == 0:
        # Alle x er 0. Da må alle y være 0 for at det skal være proporsjonalt.
        return (np.allclose(y, 0.0, atol=atol, rtol=0), 0.0 if np.allclose(y, 0.0, atol=atol, rtol=0) else None)

    k = y[nz[0]] / x[nz[0]]

    # Verifiser at alle par følger samme forhold
    for xi, yi in zip(x, y):
        if np.isclose(xi, 0.0, atol=atol, rtol=0):
            if not np.isclose(yi, 0.0, atol=atol, rtol=0):
                return False, None  # x=0 men y!=0
            continue  # (0,0) er ok
        if not np.isclose(yi / xi, k, rtol=rtol, atol=atol):
            return False, None

    return True, float(k)

# --- Bytt ut tallene under med dine tall ---
# x = uavhengig variabel (f.eks. mengde i kg epler), y = avhengig variabel (f.eks. pris i kroner)
list_x = [0.5, 1.5, 3, 5]      # x-verdier
list_y = [11, 33, 66, 110]     # y-verdier

result, k = are_proportional(list_x, list_y)

if result:
    print(f"Listene er proporsjonale: y = k * x, med k = {k:g}")

    # --- Lag grafen ---
    plt.figure(figsize=(7, 5))

    # Plotter datapunktene
    plt.plot(list_x, list_y, 'o', markersize=8, label='Datapunkter')

    # Tegn linjen y = kx. For proporsjonalitet går linjen gjennom origo.
    x_min, x_max = min(list_x), max(list_x)
    # Velg linjeområde som inkluderer 0 for pen visualisering
    x_left = min(0.0, x_min)
    x_right = max(0.0, x_max)
    x_line = np.linspace(x_left, x_right, 400)
    y_line = k * x_line
    plt.plot(x_line, y_line, label=f'y = {k:.4g}·x')

    # Pynter på grafen
    plt.title('Proporsjonale størrelser')
    plt.xlabel('Mengde (kg epler)')   # Endre etiketter til din kontekst
    plt.ylabel('Pris (kroner)')
    plt.grid(True)
    plt.legend()

    # Hvis alle x og y er ikke-negative, start aksene på 0
    if min(list_x) >= 0 and min(list_y) >= 0:
        plt.xlim(left=0)
        plt.ylim(bottom=0)

    plt.tight_layout()
    plt.show()
else:
    print("Listene er ikke proporsjonale.")

# --- Forutsigelser (bare hvis proporsjonalt) ---
if result:
    # Finn y når x = ...
    x_val = 1.5   # <- endre til ønsket verdi
    y_val = k * x_val
    print("\nForutsigelse:")
    print(f"Når x = {x_val:g}, blir y ≈ {y_val:.4g}")

    # Finn x når y = ...
    y_target = 66  # <- endre til ønsket verdi
    if np.isclose(k, 0.0):
        if np.isclose(y_target, 0.0):
            print("Når k = 0 og y = 0 finnes uendelig mange x (alle x gir y=0).")
        else:
            print("Når k = 0 og y ≠ 0 finnes ingen løsning for x.")
    else:
        x_needed = y_target / k
        print(f"For å oppnå y = {y_target:g}, må x ≈ {x_needed:.4g}")
else:
    print("\nKan ikke gjøre forutsigelser fordi listene ikke er proporsjonale.")

In [ ]:
# Omvendt proporsjonale størrelser med ukjente y-verdier og x-verdier
import matplotlib.pyplot as plt
import numpy as np

def are_inverse_proportional(list_a, list_b, rtol=1e-9, atol=1e-12):
    """
    Sjekker om to lister er omvendt proporsjonale (y = k/x).
    Returnerer (True, k) hvis de er det, og (False, None) hvis ikke.
    k er proporsjonalitetskonstanten.

    Merk: x-verdier (list_a) kan ikke inneholde 0, siden y = k/x er udefinert for x = 0.
    """
    if len(list_a) != len(list_b) or len(list_a) == 0:
        return False, None

    # I omvendt proporsjonalitet kan x ikke være 0
    if any(a == 0 for a in list_a):
        return False, None

    # Beregn k som produktet a*b (burde være konstant hvis forholdet er perfekt)
    k = list_a[0] * list_b[0]

    for a, b in zip(list_a, list_b):
        if not np.isclose(a * b, k, rtol=rtol, atol=atol):
            return False, None

    return True, k

# --- Bytt ut tallene under med dine tall ---
list_a = [3, 4, 6, 8]   # x-verdier (kan ikke inneholde 0)
list_b = [1000, 750, 500, 375]   # y-verdier

result, k = are_inverse_proportional(list_a, list_b)

if result:
    print(f"Listene er omvendt proporsjonale med proporsjonalitetskonstanten k = {k:g}")

    # --- Lag grafen ---
    plt.figure(figsize=(7, 5))

    # Plotter datapunktene
    plt.plot(list_a, list_b, 'o', markersize=8, label='Datapunkter')

    # Definer området for en jevn kurve
    x_min, x_max = min(list_a), max(list_a)
    eps = 1e-9

    def plot_curve(x0, x1, label=None):
        xs = np.linspace(x0, x1, 400)
        xs = xs[np.abs(xs) > eps]  # fjern 0 for å unngå deling på 0
        ys = k / xs
        plt.plot(xs, ys, label=label)

    # Dersom x-området krysser 0, tegn to grener (venstre og høyre for 0)
    if x_min < 0 < x_max:
        plot_curve(x_min, -eps, label=f'y = {k:g}/x')
        plot_curve(eps, x_max)
    else:
        # Hele området på én side av 0
        # Pass på at vi ikke inkluderer 0 i xs
        left, right = (x_min, x_max) if x_max < 0 or x_min > 0 else (eps, x_max)
        plot_curve(left, right, label=f'y = {k:g}/x')

    plt.title('Omvendt proporsjonale størrelser')          # Bytt ut med overskrift
    plt.xlabel('Antall deltakere')                         # Bytt ut med aksenavn
    plt.ylabel('Pris per deltaker')                        # Bytt ut med aksenavn
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

else:
    print("Listene er ikke omvendt proporsjonale.")

# --- 5. Bruk funksjonen til å forutsi verdier basert på y = k/x ---
# Eksempel: Finn y når x = 1.5
x_val = 5                                                 # Bytt ut tallet med det du ønsker å finne

if result:
    if np.isclose(x_val, 0.0):
        print("\nForutsigelse: x kan ikke være 0 i modellen y = k/x.")
    else:
        y_val = k / x_val
        print(f"\nForutsigelse:")
        print(f"Når x = {x_val:g}, blir y ≈ {y_val:.4g}")

    # Eksempel: Finn x når y = 6
    y_target = 250                                    # Bytt ut tallet med det du ønsker å finne
    if np.isclose(y_target, 0.0):
        if np.isclose(k, 0.0):
            print("Når y = 0 og k = 0, finnes uendelig mange løsninger for x ≠ 0.")
        else:
            print("Når y = 0 men k ≠ 0, finnes ingen x som oppfyller y = k/x.")
    else:
        x_needed = k / y_target
        print(f"For å oppnå y = {y_target:g}, må x ≈ {x_needed:.4g}")
else:
    print("\nKan ikke gjøre forutsigelser fordi listene ikke er omvendt proporsjonale.")

# $\color{grey}{\text{Kapittel 4 - Statistikk}}$

In [ ]:
# Pyton kode: Kopier teksten under her og lim inn i Jupyter, prøv å endre tallene og se hva som skjer :) 
# 1 sett med frekvenser - Søyle, sektor og linjediagram 
import matplotlib.pyplot as plt

# Liste med navn og frekvenser
navn = ["A", "B", "C", "D", "E"]                                 # Forandre tilpasset din oppgave
frekvenser = [20, 30, 10, 25, 15]                                # Forandre tilpasset din oppgave

# Fargevalg
farger = ['red','#66b3ff','#99ff99','#ffcc99','#c2c2f0']         # Forandre fargevalg

# Beregn totalfrekvensen
total = sum(frekvenser)

# Beregn andelene (i prosent) for hver sektor
andelene = [100 * frek / total for frek in frekvenser]

# Kakediagram
plt.pie(andelene, labels=navn, autopct='%1.1f%%', colors=farger)
plt.title("Frekvenser av navn")                                     # Forandre tittel på kakediagram
plt.axis('equal')
plt.show()

# Søylediagram
plt.bar(navn, frekvenser, color=farger)
plt.title("Frekvenser av navn")                                     # Forandre tittel på søylediagram
plt.xlabel("Navn")                                                  # Forandre x-aksenavnet på søylediagram
plt.ylabel("Frekvens")                                              # Forandre y-aksenavnet på søylediagram
plt.ylim(ymin=0)
plt.show()

# Linjediagram
plt.plot(navn, frekvenser, marker='o', color='#ff7f0e')
plt.title("Frekvenser av navn")                                     # Forandre tittel på linjediagram
plt.xlabel("Navn")                                                  # Forandre x-aksenavnet på linjediagram
plt.ylabel("Frekvens")                                              # Forandre y-aksenavnet på linjediagram
plt.ylim(ymin=0)
plt.show()

In [ ]:
# Pyton kode: Kopier teksten under her og lim inn i Jupyter, prøv å endre tallene og se hva som skjer :)
# 2 sett med frekvenser ved siden av hverandre - Søyle, sektor og linjediagram 

import matplotlib.pyplot as plt
import numpy as np

# Liste med navn og frekvenser
navn = ["Mandag", "Tirsdag", "Onsdag", "Torsdag", "Fredag"]        # Ukedag
frekvenser1 = [10, 15, 11, 12, 15]                                 # Syklister
frekvenser2 = [7, 9, 6, 7, 12]                                     # Syklister med hjelm

# Fargevalg
farger = ['lemonchiffon','coral','#99ff99','#ffcc99','#c2c2f0']
farger1 = ['blue']
farger2 = ['red']

# Beregn totalfrekvensen for begge sett
total1 = sum(frekvenser1)
total2 = sum(frekvenser2)

# Beregn andelene (i prosent) for hver sektor for begge sett
andelene1 = [100 * frek / total1 for frek in frekvenser1]
andelene2 = [100 * frek / total2 for frek in frekvenser2]

# Kakediagram for første sett
plt.pie(andelene1, labels=navn, autopct='%1.1f%%', colors=farger)
plt.title("Frekvenser av navn - Sett 1")
plt.axis('equal')
plt.show()

# Kakediagram for andre sett
plt.pie(andelene2, labels=navn, autopct='%1.1f%%', colors=farger)
plt.title("Frekvenser av navn - Sett 2")
plt.axis('equal')
plt.show()

# Søylediagram
bar_width = 0.3  # Reduser bredden på søylene
index = np.arange(len(navn)) * 1.5  # Øk avstanden mellom søylene

fig, ax = plt.subplots()
bar1 = ax.bar(index, frekvenser1, bar_width, label='Syklister totalt', color=farger1)
bar2 = ax.bar(index + bar_width + 0.05, frekvenser2, bar_width, label='Syklister med hjelm', color=farger2)  # Legg til mellomrom mellom søylene

ax.set_xlabel('Ukedag')
ax.set_ylabel('Frekvens')
ax.set_title('Syklister totalt og syklister med hjelm')
ax.set_xticks(index + bar_width / 2 + 0.025)
ax.set_xticklabels(navn)
ax.legend()

# Juster y-aksen slik at det høyeste antall frekvenser matcher y-aksen sitt høyeste
max_frekvens = max(max(frekvenser1), max(frekvenser2))
ax.set_ylim(0, max_frekvens + 1)

plt.show()

# Linjediagram for første sett
plt.plot(navn, frekvenser1, marker='o', color='#ff7f0e', label='Sett 1')
plt.plot(navn, frekvenser2, marker='x', color='#1f77b4', label='Sett 2')
plt.title("Frekvenser av navn")                                               # Endre overskrift
plt.xlabel("Navn")                                                            # Endre x-akse navn
plt.ylabel("Frekvens")                                                        # Endre y-akse navn
plt.ylim(ymin=0)
plt.legend()
plt.show()

In [ ]:
#Pyton kode: Kopier teksten under her og lim inn i Jupyter, prøv å endre tallene og se hva som skjer :)
# 2 sett med frekvenser over hverandre - Søyle, sektor og linjediagram + frekvenstabell
import matplotlib.pyplot as plt
import numpy as np

# Liste med navn og frekvenser
navn = ["test", "B", "C", "D", "E"]
frekvenser_1 = [3, 6, 5, 5, 3]
frekvenser_2 = [7, 9, 6, 7, 12]

# Fargevalg
farger = ['red', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0']

# Beregn summen av frekvensene
total_frekvens_1 = sum(frekvenser_1)
total_frekvens_2 = sum(frekvenser_2)

# Kakediagram
plt.pie([total_frekvens_1, total_frekvens_2], labels=['Frekvenser 1', 'Frekvenser 2'], autopct='%1.1f%%', colors=farger)
plt.title("Sum av frekvenser 1 og frekvenser 2")
plt.axis('equal')
plt.show()

# Søylediagram
bredde = 0.4  # Bredde på hver søyle
x_pos = np.arange(len(navn))  # x-koordinater for hver søyle

plt.bar(x_pos, frekvenser_2, width=bredde, color='blue', label='Frekvenser 2')
plt.bar(x_pos, frekvenser_1, width=bredde, bottom=frekvenser_2, color='red', label='Frekvenser 1')

plt.title("Frekvenser av navn")
plt.xlabel("Navn")
plt.ylabel("Frekvens")
plt.xticks(x_pos, navn)
plt.legend()
plt.ylim(0, 16)  # Setter y-aksen fra 0 til 16
plt.show()

# Linjediagram
plt.plot(navn, frekvenser_1, marker='o', color='#ff7f0e', label='Frekvenser 1')
plt.plot(navn, frekvenser_2, marker='o', color='#1f77b4', label='Frekvenser 2')

plt.title("Frekvenser av navn")
plt.xlabel("Navn")
plt.ylabel("Frekvens")
plt.legend()
plt.ylim(ymin=0)
plt.show()

# Calculate relative frequencies
relative_frekvenser_1 = [f / total_frekvens_1 for f in frekvenser_1]
relative_frekvenser_2 = [f / total_frekvens_2 for f in frekvenser_2]

# Calculate cumulative frequencies
cumulative_frekvenser_1 = np.cumsum(relative_frekvenser_1)
cumulative_frekvenser_2 = np.cumsum(relative_frekvenser_2)

# Create a table
table_data = list(zip(navn, [str(f) for f in frekvenser_1], [str(f) for f in frekvenser_2],
                      [f'{f:.2f}%' for f in relative_frekvenser_1], [f'{f:.2f}%' for f in relative_frekvenser_2],
                      [f'{f:.2f}%' for f in cumulative_frekvenser_1], [f'{f:.2f}%' for f in cumulative_frekvenser_2]))
table_data.append(('', '', '', f'{sum(relative_frekvenser_1):.2f}%', f'{sum(relative_frekvenser_2):.2f}%',
                  f'{sum(cumulative_frekvenser_1):.2f}%', f'{sum(cumulative_frekvenser_2):.2f}%'))  # Add the row for sum

table_columns = ['Navn', 'Frekvenser 1', 'Frekvenser 2', 'Relative Frekvenser 1', 'Relative Frekvenser 2',
                  'Kumulativ Frekvenser 1', 'Kumulativ Frekvenser 2']

# Create the plot
plt.figure(figsize=(10, 6))  # Set the size of the figure

# Plot the table
table = plt.table(cellText=table_data, colLabels=table_columns, loc='center', cellLoc='center', fontsize=14)
table.scale(1, 1.2)  # Scale the table to increase the row height

# Remove the axis
plt.axis('off')

# Set a title for the table
plt.title("Tabell over frekvenser og relative frekvenser", fontsize=16, y=0.7)  # Adjust the y position of the title


# Show the plot
plt.show()

In [ ]:
# Finne median, gjennomsnitt, modus (altså typetall), variasjonsbredden og antallet tall i listen (altså frekvensen) 
# og standardavviket til en liste med tall
from statistics import median, mean, mode, stdev, StatisticsError

# Liste med tallverdier
liste = [5,4,5,3,4,5,5,4,5,3]               # Endre tallene i frekvenstabellen

# Beregner medianen og gjennomsnittet
medianen = round(median(liste), 2)
gjennomsnittet = round(mean(liste), 2)

# Forsøker å beregne typetallet og håndterer tilfeller der det ikke finnes et unikt typetall
try:
    typetallet = round(mode(liste), 2)
except StatisticsError:
    typetallet = "Ingen unik modus"

# Beregner variasjonsbredden
variasjonsbredden = max(liste) - min(liste)

# Beregner standardavviket
standardavviket = round(stdev(liste), 0)

# Beregner antallet tall i listen
antall_tall = len(liste)

# Skriver ut resultatene
print("Medianen er", medianen)
print("Gjennomsnittet er", gjennomsnittet)
print("Typetallet er", typetallet)
print("Variasjonsbredden er", variasjonsbredden)
print("Standardavviket er", standardavviket)
print("Antallet tall i listen er", antall_tall)

In [ ]:
# Gjennomsnitt og typetall i en frekvenstabell
from statistics import mean, mode
# matplotlib.pyplot for å lage plott og grafer
import matplotlib.pyplot as plt

# 1. DEFINER FREKVENSTABELLEN
# Dette er den eneste delen du trenger å endre for din oppgave.
# Format: [(observasjonsverdi, frekvens), ...]
frekvenstabell = [(0, 8), (1, 5), (2, 6), (3, 2), (4, 3), (5, 2), (6, 1)]

# 2. UTVIDE FREKVENSTABELLEN TIL EN FULLSTENDIG DATALISTE
# Vi lager en liste der hver observasjon legges til så mange ganger som frekvensen tilsier.
# Eksempel: (0, 8) blir til åtte 0-ere i listen.
data_liste = []
for observasjon, frekvens in frekvenstabell:
    data_liste += frekvens * [observasjon]

# (Valgfritt) Skriv ut den fulle listen for å se resultatet
# print("Fullstendig dataliste:", data_liste)

# 3. BEREGNE STATISTISKE MÅL
gjennomsnitt = mean(data_liste)
typetall = mode(data_liste)

# 4. SKRIVE UT RESULTATENE TIL KONSOLLEN
print("--- Resultater fra analysen ---")
print(f"Gjennomsnittet er: {round(gjennomsnitt, 2)}")
print(f"Typetallet er: {typetall}")
print("--------------------------------")

# 5. VISUALISERE DATAENE MED ET SØYLEDIAGRAM
# Henter ut observasjonene (x-verdiene) og frekvensene (y-verdiene) fra tabellen
observasjoner = [item[0] for item in frekvenstabell]
frekvenser = [item[1] for item in frekvenstabell]

# Lager selve søylediagrammet
plt.figure(figsize=(10, 6)) # Angir en fin størrelse på plottet
plt.bar(observasjoner, frekvenser, color='skyblue', edgecolor='black', label='Frekvens')

# Legger til en vertikal linje for å vise gjennomsnittet
plt.axvline(gjennomsnitt, color='red', linestyle='--', linewidth=2, label=f'Gjennomsnitt ({gjennomsnitt:.2f})')

# Legger til titler og etiketter for å gjøre plottet lett å forstå
plt.title('Frekvensfordeling av observasjoner', fontsize=16)
plt.xlabel('Observasjonsverdi', fontsize=12)
plt.ylabel('Frekvens (antall)', fontsize=12)
plt.xticks(observasjoner) # Sørger for at alle observasjonsverdier vises på x-aksen
plt.legend() # Viser etikettene (f.eks. "Gjennomsnitt")
plt.grid(axis='y', linestyle='--', alpha=0.7) # Legger til et rutenett på y-aksen

# Viser det ferdige plottet
plt.show()

In [ ]:
# Gjennomsnitt, typetall og median i en frekvenstabell
from statistics import mean, mode, median
# matplotlib.pyplot for å lage plott og grafer
import matplotlib.pyplot as plt

# 1. DEFINER FREKVENSTABELLEN
# Dette er den eneste delen du trenger å endre for din oppgave.
# Format: [(observasjonsverdi, frekvens), ...]
frekvenstabell = [(1, 306), (2, 2000), (3, 2216), (4, 2090), (5, 1730), (6, 667)]

# 2. UTVIDE FREKVENSTABELLEN TIL EN FULLSTENDIG DATALISTE
# Vi lager en liste der hver observasjon legges til så mange ganger som frekvensen tilsier.
data_liste = []
for observasjon, frekvens in frekvenstabell:
    data_liste += frekvens * [observasjon]

# (Valgfritt) Skriv ut den fulle listen for å se resultatet
# print("Fullstendig dataliste:", data_liste)

# 3. BEREGNE SENTRALMÅL (GJENNOMSNITT, TYPETALL OG MEDIAN)
gjennomsnitt = mean(data_liste)
typetall = mode(data_liste)
median_verdi = median(data_liste)

# 4. SKRIVE UT RESULTATENE TIL KONSOLLEN
print("--- Sentralmål fra frekvenstabellen ---")
print(f"Gjennomsnitt: {round(gjennomsnitt, 2)}")
print(f"Typetall:     {typetall}")
print(f"Median:       {median_verdi}")
print("---------------------------------------")

# 5. VISUALISERE DATAENE MED ET SØYLEDIAGRAM
# Henter ut observasjonene (x-verdiene) og frekvensene (y-verdiene)
observasjoner = [item[0] for item in frekvenstabell]
frekvenser = [item[1] for item in frekvenstabell]

# Lager selve søylediagrammet
plt.figure(figsize=(10, 6)) # Angir en fin størrelse på plottet
plt.bar(observasjoner, frekvenser, color='cornflowerblue', edgecolor='black', label='Frekvens')

# Legger til vertikale linjer for å vise sentralmålene
plt.axvline(gjennomsnitt, color='red', linestyle='--', linewidth=2, label=f'Gjennomsnitt ({gjennomsnitt:.2f})')
plt.axvline(median_verdi, color='green', linestyle=':', linewidth=2.5, label=f'Median ({median_verdi})')

# Legger til titler og etiketter for å gjøre plottet lett å forstå
plt.title('Frekvensfordeling med sentralmål', fontsize=16)
plt.xlabel('Observasjonsverdi', fontsize=12)
plt.ylabel('Frekvens (antall)', fontsize=12)
plt.xticks(observasjoner) # Sørger for at alle observasjonsverdier vises på x-aksen
plt.legend() # Viser etikettene (forklaringen av linjer og søyler)
plt.grid(axis='y', linestyle='--', alpha=0.7) # Legger til et rutenett på y-aksen

# Viser det ferdige plottet
plt.show()

In [ ]:
# Variasjonsbredde, gjennomsnitt og standardavvik for en liste med tall
from statistics import mean, pstdev

# Liste med tallverdier
liste = [73, 85, 71, 75, 74, 79, 86, 70, 74, 62, 69]                   # Endre disse tallene

# Beregner variasjonsbredden
variasjonsbredde = max(liste) - min(liste)

# Beregner gjennomsnittet og standardavviket
gjennomsnitt = mean(liste)
standardavvik = pstdev(liste)

# Skriver ut resultatene
print("Variasjonsbredden er", round(variasjonsbredde,2))
print("Gjennomsnittet er", round(gjennomsnitt, 2))
print("Standardavviket er", round(standardavvik, 2))

In [ ]:
# 
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, shapiro

# Eksempeldata for høyder i cm
hoyder = [199.2, 188.7, 199.5, 198.2, 198.9, 188.5, 199.3]              # Forandre disse tallene

def beregn_statistikk(data):
    gjennomsnitt = np.mean(data)
    standardavvik = np.std(data)
    variasjonsbredde = max(data) - min(data)
    return gjennomsnitt, standardavvik, variasjonsbredde
def plott_normalfordeling(data, gjennomsnitt, standardavvik):
    x = np.linspace(gjennomsnitt - 3 * standardavvik, gjennomsnitt + 3 * standardavvik, 1000)
    y = norm.pdf(x, gjennomsnitt, standardavvik)

    plt.figure(figsize=(10, 6))
    plt.plot(x, y, label='Normalfordeling', color='black', lw=2)

    # Skravering for 1, 2 og 3 standardavvik
    colors = ['green', 'orange', 'blue']
    for i in range(1, 4):
        plt.fill_between(x, 0, y, where=(x >= gjennomsnitt - i * standardavvik) & (x <= gjennomsnitt + i * standardavvik),
                         color=colors[i-1], alpha=0.3, label=f'{i} Standardavvik')

    # Vertikale linjer for gjennomsnitt og standardavvik
    plt.axvline(gjennomsnitt, color='black', linestyle='-', label=f'Gjennomsnitt ({gjennomsnitt:.2f} cm)', lw=2)
    for i in range(1, 4):
        plt.axvline(gjennomsnitt + i * standardavvik, color='red', linestyle='--', label=f'{i} Standardavvik ({gjennomsnitt + i * standardavvik:.2f} cm)')
        plt.axvline(gjennomsnitt - i * standardavvik, color='red', linestyle='--')

    plt.title("Høydefordeling i Land A")
    plt.xlabel("Høyde (cm)")
    plt.ylabel("Tetthet")
    plt.legend(loc='upper right', frameon=False, title="Standardavvik", fontsize=10)
    plt.grid(True)
    plt.show()

def skriv_ut_statistikk(gjennomsnitt, standardavvik, variasjonsbredde):
    print(f"Variasjonsbredden er {variasjonsbredde:.2f}")
    print(f"Gjennomsnittet er {gjennomsnitt:.2f}")
    print(f"Standardavviket er {standardavvik:.2f}")
    print("\nI en normalfordeling vil:")
    print("68.27 % av datasettet være under ett standardavvik fra gjennomsnittet.")
    print("95.45 % av datasettet være under to standardavvik fra gjennomsnittet.")
    print("99.73 % av datasettet være under tre standardavvik fra gjennomsnittet.")
    print("99.994 % av datasettet være under fire standardavvik fra gjennomsnittet.")

def test_normalfordeling(data):
    stat, p_value = shapiro(data)
    if p_value > 0.05:
        print("Dataene er normalfordelte (p-verdi > 0.05).")
    else:
        print("Dataene er ikke normalfordelte (p-verdi <= 0.05).")

# Beregn statistikk
gjennomsnitt, standardavvik, variasjonsbredde = beregn_statistikk(hoyder)

# Plott normalfordeling
plott_normalfordeling(hoyder, gjennomsnitt, standardavvik)

# Skriv ut statistikk
skriv_ut_statistikk(gjennomsnitt, standardavvik, variasjonsbredde)

# Test om dataene er normalfordelte
test_normalfordeling(hoyder)

Hvis dataene ikke er normalfordelte, betyr det at de ikke følger den typiske "klokkeformen" til en normalfordeling. Dette kan ha flere konsekvenser, avhengig av hva du prøver å oppnå:

Statistiske tester: Mange statistiske tester, som t-test og ANOVA, antar at dataene er normalfordelte. Hvis denne antagelsen ikke holder, kan resultatene fra disse testene være misvisende.

Konfidensintervaller: Konfidensintervaller basert på normalfordeling kan være unøyaktige hvis dataene ikke er normalfordelte.

Regresjonsanalyse: Lineær regresjon antar ofte normalfordelte residualer. Hvis residualene ikke er normalfordelte, kan modellens prediksjoner være upålitelige.

Beskrivende statistikk: Gjennomsnitt og standardavvik er mest meningsfulle for normalfordelte data. For ikke-normalfordelte data kan median og interkvartilavstand være bedre mål.

Transformasjoner: Du kan prøve å transformere dataene (f.eks. log-transformasjon) for å oppnå normalfordeling.

Alternative metoder: Bruk ikke-parametriske tester som ikke krever normalfordeling, som Mann-Whitney U-test eller Kruskal-Wallis test.

In [ ]:
# Beregner standardavviket for en liste med tall med litt andre farger for 2 sett med verdier
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Eksempeldata for høyder i cm for to forskjellige lag
hoyder_lag_a = [199.2, 188.7, 199.5, 198.2, 198.9, 188.5, 199.3]                    # Forandre disse tallene
hoyder_lag_b = [185.2, 190.1, 192.3, 187.5, 193.2, 189.8, 191.4]                    # Forandre disse tallene

# Beregner gjennomsnittet og standardavviket for begge lag
gjennomsnitt_a = np.mean(hoyder_lag_a)
standardavvik_a = np.std(hoyder_lag_a)
gjennomsnitt_b = np.mean(hoyder_lag_b)
standardavvik_b = np.std(hoyder_lag_b)

# Lager X-verdier som går fra -3 til +3 standardavvik fra gjennomsnittet for begge lag
x_a = np.linspace(gjennomsnitt_a - 3 * standardavvik_a, gjennomsnitt_a + 3 * standardavvik_a, 1000)
x_b = np.linspace(gjennomsnitt_b - 3 * standardavvik_b, gjennomsnitt_b + 3 * standardavvik_b, 1000)

# Beregner Y-verdiene for normalfordelingen for begge lag
y_a = norm.pdf(x_a, gjennomsnitt_a, standardavvik_a)
y_b = norm.pdf(x_b, gjennomsnitt_b, standardavvik_b)

# Plotter normalfordelingskurvene
plt.figure(figsize=(12, 8))
plt.plot(x_a, y_a, label='Normalfordeling Lag A', color='black', lw=2)
plt.plot(x_b, y_b, label='Normalfordeling Lag B', color='blue', lw=2)

# Skravering for 1, 2 og 3 standardavvik for Lag A (mer kontrastfylte farger)
plt.fill_between(x_a, 0, y_a, where=(x_a >= gjennomsnitt_a - 1 * standardavvik_a) & (x_a <= gjennomsnitt_a + 1 * standardavvik_a),
                 color='red', alpha=0.3, label='1 Standardavvik Lag A')
plt.fill_between(x_a, 0, y_a, where=(x_a >= gjennomsnitt_a - 2 * standardavvik_a) & (x_a <= gjennomsnitt_a + 2 * standardavvik_a),
                 color='purple', alpha=0.3, label='2 Standardavvik Lag A')
plt.fill_between(x_a, 0, y_a, where=(x_a >= gjennomsnitt_a - 3 * standardavvik_a) & (x_a <= gjennomsnitt_a + 3 * standardavvik_a),
                 color='blue', alpha=0.3, label='3 Standardavvik Lag A')

# Skravering for 1, 2 og 3 standardavvik for Lag B (mer kontrastfylte farger)
plt.fill_between(x_b, 0, y_b, where=(x_b >= gjennomsnitt_b - 1 * standardavvik_b) & (x_b <= gjennomsnitt_b + 1 * standardavvik_b),
                 color='green', alpha=0.3, label='1 Standardavvik Lag B')
plt.fill_between(x_b, 0, y_b, where=(x_b >= gjennomsnitt_b - 2 * standardavvik_b) & (x_b <= gjennomsnitt_b + 2 * standardavvik_b),
                 color='orange', alpha=0.3, label='2 Standardavvik Lag B')
plt.fill_between(x_b, 0, y_b, where=(x_b >= gjennomsnitt_b - 3 * standardavvik_b) & (x_b <= gjennomsnitt_b + 3 * standardavvik_b),
                 color='yellow', alpha=0.3, label='3 Standardavvik Lag B')

# Legger til vertikale linjer for gjennomsnitt og standardavvikene for begge lag
plt.axvline(gjennomsnitt_a, color='black', linestyle='-', label=f'Gjennomsnitt Lag A ({gjennomsnitt_a:.2f} cm)', lw=2)
plt.axvline(gjennomsnitt_b, color='blue', linestyle='-', label=f'Gjennomsnitt Lag B ({gjennomsnitt_b:.2f} cm)', lw=2)
for i in range(1, 4):
    # Linjer til høyre for gjennomsnittet
    plt.axvline(gjennomsnitt_a + i * standardavvik_a, color='red', linestyle='--', label=f'{i} Standardavvik Lag A ({gjennomsnitt_a + i * standardavvik_a:.2f} cm)')
    plt.axvline(gjennomsnitt_b + i * standardavvik_b, color='green', linestyle='--', label=f'{i} Standardavvik Lag B ({gjennomsnitt_b + i * standardavvik_b:.2f} cm)')
    # Linjer til venstre for gjennomsnittet
    plt.axvline(gjennomsnitt_a - i * standardavvik_a, color='red', linestyle='--')
    plt.axvline(gjennomsnitt_b - i * standardavvik_b, color='green', linestyle='--')

# Legger til tittel og etiketter
plt.title("Høydefordeling i Lag A og Lag B")                       # Endre overskriften
plt.xlabel("Høyde (cm)")                                           # Endre x-akse navnet
plt.ylabel("Tetthet")                                              # Endre y-akse navnet

# Justering av legenden for å sikre god lesbarhet
plt.legend(loc='upper right', frameon=False, title="Standardavvik", fontsize=10)

# Beregner variasjonsbredden for begge lag
variasjonsbredde_a = max(hoyder_lag_a) - min(hoyder_lag_a)
variasjonsbredde_b = max(hoyder_lag_b) - min(hoyder_lag_b)

# Skriver ut resultatene
print("Variasjonsbredden for Lag A er", round(variasjonsbredde_a, 2))
print("Gjennomsnittet for Lag A er", round(gjennomsnitt_a, 2))
print("Standardavviket for Lag A er", round(standardavvik_a, 2))
print("Variasjonsbredden for Lag B er", round(variasjonsbredde_b, 2))
print("Gjennomsnittet for Lag B er", round(gjennomsnitt_b, 2))
print("Standardavviket for Lag B er", round(standardavvik_b, 2))

# Viser grafen
plt.grid(True)
plt.show()

# Legger til prosentverdiene under bildet
print("\nI en normalfordeling vil:")
print("68.27 % av datasettet være under ett standardavvik fra gjennomsnittet.")
print("95.45 % av datasettet være under to standardavvik fra gjennomsnittet.")
print("99.73 % av datasettet være under tre standardavvik fra gjennomsnittet.")
print("99.994 % av datasettet være under fire standardavvik fra gjennomsnittet.")

In [ ]:
# Gjennomsnitt, standardvvik og variasjonbredde for 2 sett med sekvenser og samme normalfordeling
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import matplotlib.colors as mcolors

def plot_team_distribution(ax, data, name, line_color, fill_colors):
    """
    Beregner og plotter normalfordelingen for ett enkelt lag på en gitt akse.

    Args:
        ax (matplotlib.axes.Axes): Aksen som skal plottes på.
        data (list): Listen med datapunkter (f.eks. høyder).
        name (str): Navnet på laget (f.eks. 'Lag A').
        line_color (str): Fargen på selve normalkurven.
        fill_colors (list): En liste med tre farger for 1, 2 og 3 standardavvik.
    """
    # 1. Beregn statistikk
    mean = np.mean(data)
    std = np.std(data)
    variance_range = np.ptp(data)  # np.ptp() er en snarvei for max(data) - min(data)

    # 2. Forbered data for plotting
    x = np.linspace(mean - 4 * std, mean + 4 * std, 1000)
    y = norm.pdf(x, mean, std)

    # 3. Plot selve normalkurven
    ax.plot(x, y, label=f'Normalfordeling {name}', color=line_color, lw=2)

    # 4. Fyll områder for standardavvik
    # Vi plotter det største området først (3 std) med lavest alpha, deretter mindre oppå.
    for i in range(3, 0, -1):
        ax.fill_between(x, 0, y,
                        where=(x >= mean - i * std) & (x <= mean + i * std),
                        color=fill_colors[i-1],
                        alpha=0.3,
                        label=f'{i} Standardavvik {name}')

    # 5. Plot vertikale linjer
    ax.axvline(mean, color=line_color, linestyle='-', label=f'Gjennomsnitt {name} ({mean:.2f} cm)', lw=2)
    for i in range(1, 4):
        # Linjer for standardavvik (uten egen label for å holde legenden ren)
        ax.axvline(mean + i * std, color=line_color, linestyle='--', alpha=0.7)
        ax.axvline(mean - i * std, color=line_color, linestyle='--', alpha=0.7)

    return {
        "gjennomsnitt": mean,
        "standardavvik": std,
        "variasjonsbredde": variance_range
    }

# --- HOVEDSKRIPT ---

# 1. Definer data og utseende for lagene på ett sted
teams_data = {
    'Lag A': {
        'data': [199.2, 188.7, 199.5, 198.2, 198.9, 188.5, 199.3],
        'line_color': 'black',
        'fill_colors': ['red', 'purple', 'blue']  # Farger for 1, 2, og 3 std
    },
    'Lag B': {
        'data': [185.2, 190.1, 192.3, 187.5, 193.2, 189.8, 191.4],
        'line_color': 'dodgerblue',
        'fill_colors': ['green', 'orange', 'yellow'] # Farger for 1, 2, og 3 std
    }
}

# 2. Sett opp figuren
fig, ax = plt.subplots(figsize=(12, 8))

# 3. Loop gjennom lagene og plot dem
results = {}
for name, properties in teams_data.items():
    results[name] = plot_team_distribution(
        ax=ax,
        data=properties['data'],
        name=name,
        line_color=properties['line_color'],
        fill_colors=properties['fill_colors']
    )

# 4. Tilpass og vis plottet
ax.set_title("Høydefordeling i Lag A og Lag B")
ax.set_xlabel("Høyde (cm)")
ax.set_ylabel("Tetthet")
ax.legend(loc='upper right', frameon=False, title="Forklaring")
ax.grid(True)
plt.show()

# 5. Skriv ut en pen oppsummering av resultatene
print("\n--- Statistisk Oppsummering ---")
for name, stats in results.items():
    print(f"\nResultater for {name}:")
    print(f"  Variasjonsbredde: {stats['variasjonsbredde']:.2f}")
    print(f"  Gjennomsnitt:     {stats['gjennomsnitt']:.2f}")
    print(f"  Standardavvik:    {stats['standardavvik']:.2f}")

# 6. Legg til generell informasjon
print("\nI en normalfordeling vil ca.:")
print("  68.3 % av dataene ligge innenfor ett standardavvik fra gjennomsnittet.")
print("  95.5 % av dataene ligge innenfor to standardavvik fra gjennomsnittet.")
print("  99.7 % av dataene ligge innenfor tre standardavvik fra gjennomsnittet.")

In [ ]:
# Gjennomsnitt, standardavvik, variansen fra en frekvenstabell
from statistics import mean, pstdev, variance

# Frekvenstabell med verdier og deres frekvenser
frekvenstabell = [(0,1), (1,1), (2,16), (3,8), (4,1), (5,0), (6,1), (7,0), (8,0), (9,1)]  # Endre disse tallene å de matcher frekvenstabellen gitt i din oppgave

# Tom liste for å lagre verdiene
liste = []

# Fyller listen med verdier i henhold til deres frekvens
for (x, f) in frekvenstabell:
    liste += f * [x]

# Beregner gjennomsnittet, standardavviket og variansen
gjennomsnitt = mean(liste)
standardavvik = pstdev(liste)
varians = variance(liste)

# Skriver ut resultatene
print("Gjennomsnittet er", round(gjennomsnitt, 2))
print("Standardavviket er", round(standardavvik, 2))
print("Variansen er", round(varians, 2))

In [ ]:
# Gjennomsnitt, standardavvik, variansen ifra en frekvenstabell
from statistics import mean, pstdev, variance

# Frekvenstabell med verdier og deres frekvenser
frekvenstabell = [(4,2), (5,4), (5.5,2), (6,3), (7,11), (7.5,3), (8,2), (9, 1)]  # Endre disse tallene å de matcher frekvenstabellen gitt i din oppgave

# Tom liste for å lagre verdiene
liste = []

# Fyller listen med verdier i henhold til deres frekvens
for (x, f) in frekvenstabell:
    liste += f * [x]

# Beregner gjennomsnittet, standardavviket og variansen
gjennomsnitt = mean(liste)
standardavvik = pstdev(liste)
varians = variance(liste)

# Skriver ut resultatene
print("Gjennomsnittet er", round(gjennomsnitt, 2))
print("Standardavviket er", round(standardavvik, 2))
print("Variansen er", round(varians, 2))

In [ ]:
# Lage histogram
import matplotlib.pyplot as plt

# Definer dataene
intervaller = [150, 160, 165, 170, 175, 180, 185, 190, 200]                   # Intervallgrensene altså [0,10> gir 0 og 10 som intervallgrense
frekvenser = [28, 18, 43, 35, 48, 23, 15, 8]                                 # Frekvensene

# Regn ut søylehøyder
soyle_hoyder = []
for i in range(len(frekvenser)):
    intervall_bredde = intervaller[i+1] - intervaller[i]
    soyle_hoyder.append(frekvenser[i] / intervall_bredde)

# Print ut summen av frekvensene
total_frekvenser = sum(frekvenser)
print(f"Totalt antall frekvenser: {total_frekvenser}")

# Plott histogrammet
plt.bar(intervaller[:-1], soyle_hoyder, width=[intervaller[i+1] - intervaller[i] for i in range(len(frekvenser))], align='edge', edgecolor='black', color='rosybrown')

# Sett label på akser og tittel
plt.xlabel('Intervallgrenser - Høyden til elevene på skolen')                        # Endre x-akse navnet
plt.ylabel('Søylehøyde = Frekvens/Intervallbredde')                            # Alltid Søylehøyde = frekvens/intervallbredde
plt.title('Skole - elevene sin høyde')                                              # Endre overkskrift

# Vis plottet
plt.show()

In [ ]:
# Finne median og gjennomsnitt i ett klassedelt materiale ofte i forbindelse med ett histogram
import matplotlib.pyplot as plt

# Intervallgrenser og frekvenser
intervaller = [0, 10, 15, 20, 30, 50, 70]               # Intervallgrensene altså [0,10> gir 0 og 10 som intervallgrense
frekvenser = [20, 30, 150, 125, 75, 100]                # Frekvensene

# Finn midtpunktene
midtpunkter = [(intervaller[i] + intervaller[i+1]) / 2 for i in range(len(intervaller)-1)]

# Print ut summen av frekvensene
total_frekvenser = sum(frekvenser)
print(f"Totalt antall frekvenser: {total_frekvenser}")

# Regn ut gjennomsnittet
total = sum([midtpunkter[i] * frekvenser[i] for i in range(len(midtpunkter))])
gjennomsnitt = total / sum(frekvenser)
print("Gjennomsnittet er:", round(gjennomsnitt, 1))

# Regn ut medianen
n = sum(frekvenser)
midten = n / 2
cumulative_freq = 0
median = None
for i in range(len(intervaller)-1):
    cumulative_freq += frekvenser[i]
    if cumulative_freq >= midten:
        # Interpoler for å finne medianen nøyaktig
        if cumulative_freq == midten:
            median = (midtpunkter[i] + midtpunkter[i+1]) / 2
        else:
            median = midtpunkter[i]
        break

# Plot de relative kumulative frekvensene mot intervallgrensene
cumulative_freqs = []
cumulative_freq = 0
for i in range(len(intervaller)-1):
    cumulative_freq += frekvenser[i]
    cumulative_freqs.append(cumulative_freq)

rel_cumulative_freq = [0] + [cf / sum(frekvenser) for cf in cumulative_freqs]
plt.plot(intervaller, rel_cumulative_freq, label='Kumulative frekvenser', color='hotpink', linestyle='solid')
plt.axhline(y=0.5, color='red', linestyle='dashed', label='y = 0.5')

# Finn hvor medianen krysser den blå linjen og marker det punktet
for i in range(len(rel_cumulative_freq)-1):
    if rel_cumulative_freq[i] <= 0.5 and rel_cumulative_freq[i+1] >= 0.5:
        x1, y1 = intervaller[i], rel_cumulative_freq[i]
        x2, y2 = intervaller[i+1], rel_cumulative_freq[i+1]
        m = (y2 - y1) / (x2 - x1)
        x = x1 + (0.5 - y1) / m
        plt.plot(x, 0.5, marker='o', color='red', label='Medianpunkt')
        break

# Legg til en overskrift
plt.title("Tur med idrettslaget")                                    # Endre overskriften
plt.xlabel("Intervallgrenser - alderen til medlemmene")              # Endre x-akse navnet
plt.ylabel("Relative kumulative frekvenser")                         # Endre y-akse navnet
plt.legend()
plt.grid(False)
print("Medianen er:", round(x, 1))
plt.show()

In [ ]:
# Variasjonsbredde, standardavvik, median, gjennomsnitt i ett gruppet materiale
import numpy as np
import matplotlib.pyplot as plt

# Guttene sin spilletid i minutter
guttene_spilletid = np.array([18, 21, 26, 33, 33, 41, 48, 55, 64, 70, 112, 125, 146])    # Endre disse tallene til din oppgave

# a) Variasjonsbredde
variasjonsbredde = np.max(guttene_spilletid) - np.min(guttene_spilletid)
print(f"Variasjonsbredde: {variasjonsbredde} minutter")

# b) Standardavvik
gjennomsnitt = np.average(guttene_spilletid)
standardavvik = np.sqrt(np.average((guttene_spilletid - gjennomsnitt) ** 2))
print(f"Standardavvik: {standardavvik:.2f} minutter")

# c) Sammenligning med jentegjengen
jentegjeng_standardavvik = 60
if standardavvik > jentegjeng_standardavvik:
    sammenligning = "Guttene har større variasjon i spilletiden."
else:
    sammenligning = "Guttene og jentene har omtrent samme variasjon i spilletiden."

print(sammenligning)

# d) Antall elever i undersøkelsen
frekvenser = np.array([6, 41, 223, 113, 47, 8])
antall_elever = np.sum(frekvenser)
print(f"Antall elever: {antall_elever}")

# e) Medianens klasse
klasse_midtpunkter = np.array([10, 30, 50, 70, 90, 125])
median_klasse = klasse_midtpunkter[np.argmax(frekvenser)]
print(f"Medianens klasse: [{median_klasse-10}, {median_klasse+10})")

# f) Gjennomsnitt i det grupperte materialet
gjennomsnitt_gruppert = np.sum(klasse_midtpunkter * frekvenser) / antall_elever
print(f"Gjennomsnitt i det grupperte materialet: {gjennomsnitt_gruppert:.2f} minutter")


In [ ]:
# Søylediagram for 2 sett av prosenter og tabell - dataramme
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

# Lag en data ramme med de gitte dataene
data = {
    'Ukedag': ['Mandag', 'Tirsdag', 'Onsdag', 'Torsdag', 'Fredag'],                # Endre tallene så de passer til din oppgave
    'Syklister': [10, 15, 11, 12, 15],
    'Syklister med hjelm': [7, 9, 6, 7, 12]
}

df = pd.DataFrame(data)

# Legg sammen syklister og syklister med hjelm
sum_syklister = df['Syklister'].sum()
sum_syklister_med_hjelm = df['Syklister med hjelm'].sum()

# Legg til en ny rad med totalt, ekskluder NA-verdier
total_row = pd.DataFrame({'Ukedag': ['Totalt'], 'Syklister': [sum_syklister], 'Syklister med hjelm': [sum_syklister_med_hjelm]})
df = pd.concat([df, total_row], ignore_index=True)

# Fyll NaN-verdier med tomme strenger
df = df.fillna('')

# Beregn prosentene av de som sykler med og uten hjelm
df['Prosent med hjelm'] = ((df['Syklister med hjelm'] / df['Syklister']) * 100).round(2)
df['Prosent uten hjelm'] = (((df['Syklister'] - df['Syklister med hjelm']) / df['Syklister']) * 100).round(2)

# Fjern tallene som oppstår til venstre
df.index = [''] * len(df)

# Sentrer alle tallene i dataframen og vis hele dataframen på en oversiktlig måte
styled_df = df.style.set_table_styles(
    [{'selector': 'td', 'props': [('text-align', 'center')]}]
)

display(styled_df)

# Søylediagram
navn = df['Ukedag'][:-1]
prosent_med_hjelm = df['Prosent med hjelm'][:-1]
prosent_uten_hjelm = df['Prosent uten hjelm'][:-1]

bar_width = 0.3  # Reduser bredden på søylene
index = np.arange(len(navn)) * 1.5  # Øk avstanden mellom søylene

fig, ax = plt.subplots()
bar1 = ax.bar(index, prosent_med_hjelm, bar_width, label='Prosent med hjelm', color='blue')
bar2 = ax.bar(index + bar_width + 0.05, prosent_uten_hjelm, bar_width, label='Prosent uten hjelm', color='red')  # Legg til mellomrom mellom søylene

ax.set_xlabel('Ukedag')
ax.set_ylabel('Prosent')
ax.set_title('Antall syklister som bruker hjelm/ikke bruker hjelm i prosent')
ax.set_xticks(index + bar_width / 2 + 0.025)
ax.set_xticklabels(navn)
ax.legend()

# Juster y-aksen slik at den høyeste prosentandelen matcher y-aksen sitt høyeste
max_prosent_med_hjelm = prosent_med_hjelm.max()
max_prosent_uten_hjelm = prosent_uten_hjelm.max()
max_prosent = max(max_prosent_med_hjelm, max_prosent_uten_hjelm)
ax.set_ylim(0, max_prosent + 5)  # Legg til litt margin på toppen

plt.show()

In [ ]:
# Oppgave 2 - Del 2 Eksempelsett Våren 2023
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Tallene er hentet fra diagrammet over
total_2017 = 145
total_2019 = 125
total_2020 = 65
innenlands_2019 = 35
innenlands_2020 = 50
utenlands_2019 = total_2019 - innenlands_2019
utenlands_2020 = total_2020 - innenlands_2020

# Beregning av prosentvis endring
innenlands_endring = (innenlands_2020 - innenlands_2019) / innenlands_2019 * 100
utenlands_endring = (utenlands_2019 - utenlands_2020) / utenlands_2019 * 100

# Verdier for sirkeldiagrammer
utenlands_2017 = total_2017 - 40
innenlands_2017 = 40
utenlands_2021 = 80 - 65
innenlands_2021 = 65

# Utskrift av løsningsforslaget med 2 desimaler
print(f"1. resultat: 2017 var et toppår når det gjelder reiser totalt, med en kostnad på {total_2017:.2f} milliarder kroner.")
print(f"2. resultat: Fra 2019 til 2020 sank summen brukt på reiser totalt fra cirka {total_2019:.2f} milliarder kroner til {total_2020:.2f} milliarder, altså nesten en halvering.")
print(f"3. resultat: I samme periode økte summen brukt på innenlandsreiser, fra cirka {innenlands_2019:.2f} milliarder kroner til {innenlands_2020:.2f} milliarder, en økning på {innenlands_endring:.2f} prosent.")
print(f"4. resultat: Summen brukt på utenlandsreiser falt i samme periode fra {utenlands_2019:.2f} milliarder til {utenlands_2020:.2f} milliarder, en reduksjon på {utenlands_endring:.2f} prosent.")

# Lag en data ramme med de gitte dataene
data = {
    'År': [2017, 2021],
    'Utenlands': [utenlands_2017, utenlands_2021],
    'Innenlands': [innenlands_2017, innenlands_2021]
}

df = pd.DataFrame(data)

# Sentrer alle tallene i dataframen og vis hele dataframen på en oversiktlig måte
styled_df = df.style.set_table_styles(
    [{'selector': 'td', 'props': [('text-align', 'center')]}]
)

# Fjern tallene som oppstår til venstre
df.index = [''] * len(df)

display(styled_df)

print(f"5. resultat: Nedenfor er det laget sirkeldiagrammer over fordelingen av reisetypene i 2017 og 2021.")

# Lag sirkeldiagrammer over fordelingen av reisetypene i 2017 og 2021
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Sirkeldiagram for 2017
axs[0].pie(df.iloc[0, 1:], labels=['Utenlands', 'Innenlands'], autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])
axs[0].set_title('Fordeling av reisetypene i 2017')

# Sirkeldiagram for 2021
axs[1].pie(df.iloc[1, 1:], labels=['Utenlands', 'Innenlands'], autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])
axs[1].set_title('Fordeling av reisetypene i 2021')

plt.show()

# $\color{lightblue}{\text{Kapittel 5 - Lineære funksjoner}}$

In [ ]:
# Stigningstallet til den rette linjen dersom du får vite koordinatene (x1,y1) og (x2, y2)
x1 = 0
y1 = -2
x2 = 10
y2 = 0

slope = (y2 - y1) / (x2 - x1)

print("stigningstall = ", slope)

In [ ]:
# Stigningstallet til den rette linjen dersom du får vite koordinatene (x1,y1) og (x2, y2), interaktivt program med graf
import matplotlib.pyplot as plt
import numpy as np
import sys

def hent_koordinat(prompt_tekst):
    """
    Ber brukeren om en koordinat.
    Returnerer tallet som en float.
    Hvis brukeren skriver 'q', returneres None, som er et signal om å avslutte.
    """
    while True:
        bruker_input = input(prompt_tekst)
        if bruker_input.strip().lower() == 'q':
            return None
        try:
            return float(bruker_input)
        except ValueError:
            print("Ugyldig input! Vennligst skriv et tall, eller 'q' for å avslutte.")

def beregn_og_visualiser_stigningstall():
    """
    Hovedfunksjonen som kjører i en løkke for å beregne og visualisere stigningstall.
    """
    while True:
        print("\n--- Ny Beregning ---")
        print("Skriv inn koordinater. Tast 'q' og Enter når som helst for å avslutte.")

        x1 = hent_koordinat("Skriv inn x-koordinaten for første punkt (x1): ")
        if x1 is None: break

        y1 = hent_koordinat("Skriv inn y-koordinaten for første punkt (y1): ")
        if y1 is None: break

        x2 = hent_koordinat("Skriv inn x-koordinaten for andre punkt (x2): ")
        if x2 is None: break

        y2 = hent_koordinat("Skriv inn y-koordinaten for andre punkt (y2): ")
        if y2 is None: break

        try:
            # Endring 1: Gjør figurvinduet større
            fig, ax = plt.subplots(figsize=(12, 8))

            if x1 == x2:
                print("\nDette er en vertikal linje (x = {}).".format(x1))
                print("Stigningstallet er udefinert.")
                ax.axvline(x=x1, color='blue', linestyle='-', label=f'Linje x = {x1}')
                ax.plot([x1, x2], [y1, y2], 'ro', markersize=8, label=f'Punkter ({x1:.2f},{y1:.2f}) og ({x2:.2f},{y2:.2f})')
                ax.set_title('Vertikal Linje (Udefinert Stigningstall)')
            else:
                stigningstall = (y2 - y1) / (x2 - x1)
                print(f"\nStigningstallet (m) er: {stigningstall:.2f}")

                x_min, x_max = min(x1, x2), max(x1, x2)
                x_range = max(1, x_max - x_min)
                
                x_verdier = np.linspace(x_min - x_range * 0.2, x_max + x_range * 0.2, 100)
                y_verdier = stigningstall * (x_verdier - x1) + y1
                
                ax.plot(x_verdier, y_verdier, 'b-', label=f'Linje (m = Endring i y aksen / Endring i x aksen = {stigningstall:.2f})')
                ax.plot([x1, x2], [y1, y2], 'ro', markersize=8, label=f'Punkter ({x1:.2f},{y1:.2f}) og ({x2:.2f},{y2:.2f})')
                ax.plot([x1, x2], [y1, y1], 'g--', label=f'Endring i x aksen = {x2 - x1:.2f}')
                ax.plot([x2, x2], [y1, y2], 'm--', label=f'Endring i y aksen = {y2 - y1:.2f}')
                ax.set_title('Visualisering av Stigningstall')

            ax.set_xlabel('x-akse')
            ax.set_ylabel('y-akse')
            ax.grid(True)
            ax.axhline(0, color='black', linewidth=0.5)
            ax.axvline(0, color='black', linewidth=0.5)
            ax.set_aspect('equal', adjustable='box')

            # Endring 2: Flytt labelen (legend) utenfor grafen
            # Dette plasserer legend-boksen til høyre for selve graf-området.
            fig.subplots_adjust(right=0.7) # Gir plass til boksen på høyre side
            ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), borderaxespad=0.)

            plt.show()

        except Exception as e:
            print(f"\nEn uventet feil oppstod under plotting: {e}")
        
        fortsette = input("\nGrafen er vist. Trykk Enter for en ny beregning, eller 'q' for å avslutte: ")
        if fortsette.strip().lower() == 'q':
            break

    print("Programmet avsluttes.")

if __name__ == "__main__":
    beregn_og_visualiser_stigningstall()

In [ ]:
# Interaktiv analyse av stigningstall med widgets
# Koden må kjøres i Jupyter Notebook eller JupyterLab
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Funksjon for å beregne stigningstall
def calculate_slope(x1, y1, x2, y2):
    if x2 - x1 == 0:
        return "Udefinert (vertikal linje)"
    return (y2 - y1) / (x2 - x1)

# Funksjon som oppdaterer visningen basert på brukerens valg
def update(x1, y1, x2, y2, show_slope, show_plot, show_labels):
    slope = calculate_slope(x1, y1, x2, y2)
    
    if show_slope:
        print(f"Stigningstall = {slope}")

    if show_plot:
        plt.figure(figsize=(5, 3))
        plt.plot([x1, x2], [y1, y2], marker='o')
        plt.title(f"Linje fra ({x1}, {y1}) til ({x2}, {y2})")
        plt.grid(True)
        plt.axhline(0, color='black', linewidth=0.5)
        plt.axvline(0, color='black', linewidth=0.5)
        
        if show_labels:
            plt.text(x1, y1, f"({x1}, {y1})", fontsize=12, ha='right')
            plt.text(x2, y2, f"({x2}, {y2})", fontsize=12, ha='right')
        
        plt.show()

# Slidere for koordinater
x1_slider = widgets.IntSlider(value=0, min=-10, max=10, description='x1')
y1_slider = widgets.IntSlider(value=-2, min=-10, max=10, description='y1')
x2_slider = widgets.IntSlider(value=1, min=-10, max=10, description='x2')
y2_slider = widgets.IntSlider(value=0, min=-10, max=10, description='y2')

# Avkrysningsbokser for visningsvalg
show_slope_checkbox = widgets.Checkbox(value=True, description='Vis stigningstall')
show_plot_checkbox = widgets.Checkbox(value=True, description='Vis plott')
show_labels_checkbox = widgets.Checkbox(value=True, description='Vis koordinater')

# Kombiner widgets og koble til funksjonen
ui = widgets.VBox([
    x1_slider, y1_slider, x2_slider, y2_slider,
    show_slope_checkbox, show_plot_checkbox, show_labels_checkbox
])

out = widgets.interactive_output(update, {
    'x1': x1_slider,
    'y1': y1_slider,
    'x2': x2_slider,
    'y2': y2_slider,
    'show_slope': show_slope_checkbox,
    'show_plot': show_plot_checkbox,
    'show_labels': show_labels_checkbox
})

# Vis brukergrensesnittet
display(ui, out)

In [ ]:
#Tabell med (x,y) verdier

x = 5
y = 1*x -2 # Funksjonen 

print("Når x-verdien er", x,", er y-verdien lik",y)

print("x \t y") 

for x in range(11):
    y = 1*x -2  # Funksjonen
    print(x, "\t", y)

In [ ]:
# En annen måte å gjøre det samme på som forrige program
def f(x):
   y = 4*x + 3
   return y

print("Når x-verdien er", 5,", er f(x) - verdien:", f(5)) 

# Next, a loop
for x in range(11):
    print("Når x-verdien er", x, "er f(x) - verdien:\t", f(x)) 

In [ ]:
# Øvingsoppgave 5.159 s.325
xstart = 0
xslutt = 10
print("x \t y")
for x in range(xstart, xslutt):
  y = 4*x - 5
  print(x, "\t", y)

In [ ]:
# Lage den rette linjen og lage en graf
import matplotlib.pyplot as plt

# Skriv inn datapunktene (x1,y1) og (x2,y2) du får oppgitt
x1 = 0 # Bytt denne verdien
y1 = 4 # Bytt denne verdien
x2 = 2 # Bytt denne verdien
y2 = 0 # Bytt denne verdien

# Develop the equation y = mx + b
m = (y2 - y1) / (x2 - x1) # Stigningtsallet m
b = y1 - m * x1 # Konstantleddet b
print("y = ", m, "x + ", b)

# For the graph
xmin = -10
xmax = 10
ymin = -10
ymax = 10

# For the line on the graph
y3 = m * xmin + b
y4 = m * xmax + b

# Basic setup for the graph
fig, ax = plt.subplots()
plt.axis([xmin, xmax, ymin, ymax])  # Vindu størrelse
plt.plot([xmin, xmax], [0, 0], 'k')  # svar x akse
plt.plot([0, 0], [ymin, ymax], 'k')  # svart y akse

ax.set_xlabel("x verdier")
ax.set_ylabel("y verdier")
ax.set_title("Grafen til den rette linjen")
ax.grid(True)

# Plot the linear function as a red line
plt.plot([xmin, xmax], [y3, y4], 'b', label=f'y = {m}x + {b}')
plt.legend()
plt.show()

In [ ]:
# Program for å lage rett linje
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

def beregn_koeffisienter(funksjon):
    m, b = funksjon.split('x')
    return float(m), float(b)

def beregn_skjæringspunkt(funksjon1, funksjon2):
    m1, b1 = beregn_koeffisienter(funksjon1)
    m2, b2 = beregn_koeffisienter(funksjon2)
    if m1 == m2:
        return None  # Funksjonene er parallelle og har ingen skjæringspunkt
    x = (b2 - b1) / (m1 - m2)
    y = m1 * x + b1
    return x, y

def plot_funksjoner(funksjoner, data=None):
    x = np.linspace(-10, 10, 400)
    fig, ax = plt.subplots()

    funksjonsnavn = ['f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    for i, funksjon in enumerate(funksjoner):
        m, b = beregn_koeffisienter(funksjon)
        y = m * x + b
        ax.plot(x, y, label=f'{funksjonsnavn[i]}(x) = {m}x + {b}')
        print(f"Skjæringspunktet med y-aksen for funksjon {funksjonsnavn[i]}(x) = {m}x + {b} er (0, {b})")
        if m != 0:
            print(f"Skjæringspunktet med x-aksen for funksjon {funksjonsnavn[i]}(x) = {m}x + {b} er (-{b/m}, 0)")

    if data is not None:
        ax.scatter(*zip(*data), color='red')
        X = np.array([p[0] for p in data]).reshape(-1, 1)
        Y = np.array([p[1] for p in data])
        reg = LinearRegression().fit(X, Y)
        m = reg.coef_[0]
        b = reg.intercept_
        y = m * x + b
        ax.plot(x, y, label=f'r(x) = {m}x + {b}', linestyle='dashed')
        print(f"Regresjonslinjen for dummy dataene er r(x) = {m}x + {b}")

    ax.axhline(0, color='black',linewidth=0.5)
    ax.axvline(0, color='black',linewidth=0.5)
    plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)
    ax.set_xlabel(input("Hva vil du kalle x-aksen? "))
    ax.set_ylabel(input("Hva vil du kalle y-aksen? "))
    ax.set_title(input("Hva vil du kalle plottet? "))
    ax.legend()
    plt.show()

def main():
    funksjonsnavn = ['f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    antall_funksjoner = int(input("Hvor mange funksjoner vil du plotte? "))
    funksjoner = [input(f"Tast inn funksjon {i+1} (i formatet mx+b): ") for i in range(antall_funksjoner)]

    dummy_data = input("Har du noen dummy data du vil legge til? (ja/nei) ")
    if dummy_data.lower() == 'ja':
        data = []
        antall_punkter = int(input("Hvor mange datapunkter vil du legge til? "))
        for i in range(antall_punkter):
            punkt = tuple(map(float, input(f"Tast inn punkt {i+1} (i formatet x,y): ").split(',')))
            data.append(punkt)
    else:
        data = None

    for i in range(len(funksjoner)):
        for j in range(i+1, len(funksjoner)):
            punkt = beregn_skjæringspunkt(funksjoner[i], funksjoner[j])
            if punkt is not None:
                print(f"Skjæringspunktet mellom funksjon {funksjonsnavn[i]}(x) og funksjon {funksjonsnavn[j]}(x) er {punkt}")

    plot_funksjoner(funksjoner, data)

if __name__ == "__main__":
    main()

In [ ]:
# Kode for flere rette linjer i samme koordinatsystem
import matplotlib.pyplot as plt
 
# Funksjon for å beregne ligningen til en linje
def linje_koeffisienter(x1, y1, x2, y2):
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return m, b
 
# Definer grensene for grafen
xmin, xmax = -10, 10
ymin, ymax = -10, 10
 
# Liste over linjepunkter (Legg inn eller fjern)
linjer = [
    (-3, 0, 0, 3),  # Første linje
    (0, -2, 1, 0),  # Andre linje
    (0, 4, 2, 0),  # Tredje linje
 
]
 
# Opprett figur og akse
fig, ax = plt.subplots()
plt.axis([xmin, xmax, ymin, ymax])  # Vinduets størrelse
plt.plot([xmin, xmax], [0, 0], 'k')  # Svart x-akse
plt.plot([0, 0], [ymin, ymax], 'k')  # Svart y-akse
ax.set_xlabel("x verdier")
ax.set_ylabel("y verdier")
ax.set_title("Grafen til rette linjer")
ax.grid(True)
 
# Legg til linjene i plottet
for i, (x1, y1, x2, y2) in enumerate(linjer):
    m, b = linje_koeffisienter(x1, y1, x2, y2)
    print(f"Linje {i+1}: y = {m}x + {b}")
    y3 = m * xmin + b
    y4 = m * xmax + b
    plt.plot([xmin, xmax], [y3, y4], label=f'Linje {i+1}: y = {m:.2f}x + {b:.2f}')
 
# Vis legenden og plottet
plt.legend()
plt.show()

In [ ]:
# Lineær regresjon med dummy data
import numpy as np
import matplotlib.pyplot as plt

# Datasett
x = [0, 2, 4, 6, 8]                          # Bytt ut tallene slik at de passer til din oppgave
y = [132, 148, 172, 197, 223]                # Bytt ut tallene slik at de passer til din oppgave

# Konverter listene til numpy arrays
x = np.array(x)
y = np.array(y)

# Beregn gjennomsnittet av x- og y-verdiene
x_mean = np.mean(x)
y_mean = np.mean(y)

# Beregn variansen til x-verdiene
var_x = np.sum((x - x_mean) ** 2)

# Beregn kovariansen mellom x- og y-verdiene
cov_xy = np.sum((x - x_mean) * (y - y_mean))

# Beregn koeffisientene a og b
a = cov_xy / var_x
b = y_mean - a * x_mean

print ("a verdien er", a)
print ("b verdien er", b)
# Lineær funksjon
def linear_func(x, a, b):
    return a * x + b

# Skriv ut lineær funksjon
print ("y(x) = ax + b")
print("Lineær funksjon: y(x) = {:.2f} * x + {:.1f}".format(a, b))

# Plot datasettet og lineær funksjon
plt.scatter(x, y, label='Data')
plt.plot(x, linear_func(x, a, b), label='Lineær funksjon',color ="r")
plt.xlabel('x')            # Navn på x-aksen
plt.ylabel('y')            # Navn på y-aksen
plt.grid(True)
plt.legend()

# legg til en overskrift
plt.title("Lineær regresjon")

plt.show()

# Finn y-verdien når x = ...
x_val = 15
y_val = b + (a*x_val)
print(f"y-verdi for x={x_val}: {y_val:.2f}")

# Finn x-verdien når y = … 
y_val = 15
x_val = (y_val - b)/a
print(f"x-verdi for y={y_val}: {x_val:.2f}")


In [ ]:
# Graftegning (Rette linjer) ved hjelp av Python og skjæringspunkter
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

def beregn_koeffisienter(funksjon):
    m, b = funksjon.split('x')
    return float(m), float(b)

def beregn_skjæringspunkt(funksjon1, funksjon2):
    m1, b1 = beregn_koeffisienter(funksjon1)
    m2, b2 = beregn_koeffisienter(funksjon2)
    if m1 == m2:
        return None  # Funksjonene er parallelle og har ingen skjæringspunkt
    x = (b2 - b1) / (m1 - m2)
    y = m1 * x + b1
    return x, y

def plot_funksjoner(funksjoner, data=None):
    x = np.linspace(-10, 10, 400)
    fig, ax = plt.subplots()

    funksjonsnavn = ['f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    for i, funksjon in enumerate(funksjoner):
        m, b = beregn_koeffisienter(funksjon)
        y = m * x + b
        ax.plot(x, y, label=f'{funksjonsnavn[i]}(x) = {m}x + {b}')
        print(f"Skjæringspunktet med y-aksen for funksjon {funksjonsnavn[i]}(x) = {m}x + {b} er (0, {b})")
        if m != 0:
            print(f"Skjæringspunktet med x-aksen for funksjon {funksjonsnavn[i]}(x) = {m}x + {b} er (-{b/m}, 0)")

    if data is not None:
        ax.scatter(*zip(*data), color='red')
        X = np.array([p[0] for p in data]).reshape(-1, 1)
        Y = np.array([p[1] for p in data])
        reg = LinearRegression().fit(X, Y)
        m = reg.coef_[0]
        b = reg.intercept_
        y = m * x + b
        ax.plot(x, y, label=f'r(x) = {m}x + {b}', linestyle='dashed')
        print(f"Regresjonslinjen for dummy dataene er r(x) = {m}x + {b}")

    ax.axhline(0, color='black',linewidth=0.5)
    ax.axvline(0, color='black',linewidth=0.5)
    plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)
    ax.set_xlabel(input("Hva vil du kalle x-aksen? "))
    ax.set_ylabel(input("Hva vil du kalle y-aksen? "))
    ax.set_title(input("Hva vil du kalle plottet? "))
    ax.legend()
    plt.show()

def main():
    funksjonsnavn = ['f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    antall_funksjoner = int(input("Hvor mange funksjoner vil du plotte? "))
    funksjoner = [input(f"Tast inn funksjon {i+1} (i formatet mx+b): ") for i in range(antall_funksjoner)]

    dummy_data = input("Har du noen dummy data du vil legge til? (ja/nei) ")
    if dummy_data.lower() == 'ja':
        data = []
        antall_punkter = int(input("Hvor mange datapunkter vil du legge til? "))
        for i in range(antall_punkter):
            punkt = tuple(map(float, input(f"Tast inn punkt {i+1} (i formatet x,y): ").split(',')))
            data.append(punkt)
    else:
        data = None

    for i in range(len(funksjoner)):
        for j in range(i+1, len(funksjoner)):
            punkt = beregn_skjæringspunkt(funksjoner[i], funksjoner[j])
            if punkt is not None:
                print(f"Skjæringspunktet mellom funksjon {funksjonsnavn[i]}(x) og funksjon {funksjonsnavn[j]}(x) er {punkt}")

    plot_funksjoner(funksjoner, data)

if __name__ == "__main__":
    main()


In [ ]:
# Lage den rette linjen og lage en graf
import matplotlib.pyplot as plt

x1 = 2
y1 = 3
x2 = 6
y2 = 8

# Develop the equation y = mx + b
m = (y2 - y1) / (x2 - x1)
b = y1 - m * x1
print("y = ", m, "x + ", b)

# For the graph
xmin = -10
xmax = 10
ymin = -10
ymax = 10

# For the line on the graph
y3 = m * xmin + b
y4 = m * xmax + b

# Basic setup for the graph
fig, ax = plt.subplots()
plt.axis([xmin, xmax, ymin, ymax])  # window size
plt.plot([xmin, xmax], [0, 0], 'b')  # blue x axis
plt.plot([0, 0], [ymin, ymax], 'b')  # blue y axis

ax.set_xlabel("x verdier")
ax.set_ylabel("y verdier")
ax.set_title("En graf")
ax.grid(True)

# Plot the linear function as a red line
plt.plot([xmin, xmax], [y3, y4], 'r', label=f'y = {m}x + {b}')
plt.legend()
plt.show()

In [ ]:
# Stigningstallet til den rette linjen dersom du får vite koordinatene (x1,y1) og (x2, y2)
x1 = 1
y1 = 7
x2 = 2
y2 = 10

slope = (y2 - y1) / (x2 - x1)

print("slope = ", slope)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = -10
ymax = 10
points = 2*(xmax-xmin)
x = np.linspace(xmin, xmax, points)

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

ax.set_xlabel("x verdier")
ax.set_ylabel("y verdier")
ax.set_title("En graf")
ax.grid(True)

ax.set_xticks(np.arange(xmin, xmax, 1))
ax.set_yticks(np.arange(ymin, ymax, 1))

y = 2*x +1
plt.plot(x,y, label='Den rette linjen 2x+1')
plt.plot([5],[6], 'ro', label='Punkt (5,6)')
plt.plot(x,3*x, label='Den rette linjen 3x')
plt.legend()
plt.show()

# $\color{coral}{\text{Kapittel 6 - Matematiske modeller}}$

In [ ]:
# Lineær funksjon; f(x) = ax + b hvor a er stigningstallet og b er konstantleddet 
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math

# Datasett
x = [0, 40]                                 # Endre tallene så de passer til din oppgave
y = [45, 1845]                              # Endre tallene så de passer til din oppgave

# Konverter listene til numpy arrays
x = np.array(x)
y = np.array(y)

# Beregn gjennomsnittet av x- og y-verdiene
x_mean = np.mean(x)
y_mean = np.mean(y)

# Beregn variansen til x-verdiene
var_x = np.sum((x - x_mean) ** 2)

# Beregn kovariansen mellom x- og y-verdiene
cov_xy = np.sum((x - x_mean) * (y - y_mean))

# Beregn koeffisientene a og b
a = cov_xy / var_x
b = y_mean - a * x_mean

# Lineær funksjon
def calculate_linear_regression(x, a, b):
    return a * x + b

# Skriv ut lineær funksjon med LaTeX-syntaks
latex_str = f"Lineær funksjon: y(x) = {a:.2f} \\cdot x + {b:.2f}"
display(Math(latex_str))

# Skriv ut lineær funksjon uten LaTeX-syntaks
print(f"Lineær funksjon: y(x) = {a:.2f} * x + {b:.2f}")

# Plot datasettet og lineær funksjon
plt.scatter(x, y, label='Inndata')
plt.plot(x, calculate_linear_regression(x, a, b), label='Lineær funksjon', color="r")
plt.xlabel('Antall dager kyllingen lever')                                                      # x-aksen navn For opphøyd i andre (kvadrat, superskript) bruker du ^2, altså $^2$ eller y$^2$ → y²
plt.ylabel('Antall gram kyllingen veier')                                                       # y-aksen navn $_2$ betyr senket skrift (subskript) – altså “2 nede” – slik at f.eks. y$_2$ blir y₂
plt.legend()
plt.title("Kyllingens liv")                                                                     # Overskrift
plt.grid(False)
plt.show()

print("Vekstfarten/Stigningstallet a er: {:.2f}".format(a))
print("Konstantleddet/skjæringspunktet med y-aksen b er: {:.2f}".format(b))

# Finn y-verdien når x = 60
x_val = 60                                                                   # Forandre x-verdien for å finne tilhørende y-verdi
y_val = b + (a * x_val)
print(f"y-verdi for x={x_val:.2f}: {y_val:.2f}")

# Finn x-verdien når y = 500
y_val = 400                                                                   # Forandre y-verdien for å finne tilhørende x-verdi
x_val = (y_val - b) / a
print(f"x-verdi for y={y_val:.2f}: {x_val:.2f}")

# Beregn R²-verdien
y_pred = calculate_linear_regression(x, a, b)
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - y_mean) ** 2)
r2 = 1 - (ss_res / ss_tot)
print(f"R²-verdi: {r2:.2f}")

In [ ]:
# Lineær regresjon ifra Google AI studio
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# 1. Datasett (med flere punkter for et mer realistisk eksempel)
x = np.array([0, 10, 20, 30, 40])
y = np.array([45, 550, 900, 1500, 1845])

# 2. Utfør lineær regresjon med én enkelt funksjon
# slope = stigningstall (a), intercept = konstantledd (b), r_value = R-verdi
res = stats.linregress(x, y)
slope = res.slope
intercept = res.intercept
r_value = res.rvalue

# 3. Skriv ut resultatene
print(f"Lineær funksjon: y(x) = {slope:.2f} * x + {intercept:.2f}")
print(f"Vekstfarten/Stigningstallet (a) er: {slope:.2f}")
print(f"Konstantleddet/skjæringspunktet med y-aksen (b) er: {intercept:.2f}")
print(f"R²-verdi: {r_value**2:.2f}") # R-kvadrat er r_value i andre

# 4. Plot datasettet og den lineære funksjonen
plt.figure(figsize=(8, 6)) # Juster størrelsen på plottet
plt.scatter(x, y, label='Inndata', s=50) # s=50 gjør punktene større
plt.plot(x, slope * x + intercept, label='Lineær funksjon', color="r")
plt.xlabel('Antall dager kyllingen lever')
plt.ylabel('Antall gram kyllingen veier')
plt.title("Kyllingens liv (med SciPy)")
plt.legend()
plt.grid(False)                                      # Grid er ofte nyttig for å lese av verdier True/False for å skru av og på
plt.show()

# 5. Bruk funksjonen til å forutsi verdier
# Finn y-verdien når x = 60
x_val = 60
y_val = slope * x_val + intercept
print(f"\nForutsigelse:")
print(f"Når kyllingen er {x_val} dager gammel, er forventet vekt: {y_val:.2f} gram")

# Finn x-verdien når y = 500
y_val = 500
x_val = (y_val - intercept) / slope
print(f"For å oppnå en vekt på {y_val} gram, må kyllingen være ca. {x_val:.2f} dager gammel")

In [ ]:
# Versjon 2 av Google AI studio
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from typing import Tuple, List

def run_linear_regression_analysis(
    x_data: List[float], y_data: List[float]
) -> Tuple[object, plt.Figure]:  # ENDRET: stats.LinregressResult -> object
    """
    Performs a linear regression analysis, prints results, and returns a plot.

    Args:
        x_data: A list of independent variable values.
        y_data: A list of dependent variable values.

    Returns:
        A tuple containing the regression result object and the generated plot figure.
        
    Raises:
        ValueError: If input lists have different lengths or fewer than 2 data points.
    """
    if len(x_data) != len(y_data):
        raise ValueError("Input lists x_data and y_data must have the same length.")
    if len(x_data) < 2:
        raise ValueError("Linear regression requires at least two data points.")

    # 1. Konverter til numpy arrays og utfør regresjon
    x = np.array(x_data)
    y = np.array(y_data)
    res = stats.linregress(x, y)

    # 2. Lag plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(x, y, label='Inndata', s=50)
    ax.plot(x, res.slope * x + res.intercept, label='Lineær funksjon', color="r")
    ax.set_xlabel('Antall dager kyllingen lever')
    ax.set_ylabel('Antall gram kyllingen veier')
    ax.set_title("Kyllingens liv (Strukturert Analyse)")
    ax.legend()
    ax.grid(True)

    return res, fig

# --- Hoveddel av skriptet som bruker funksjonen ---
if __name__ == "__main__":
    # Definer data
    dager = [0, 10, 20, 30, 40]
    vekt = [45, 550, 900, 1500, 1845]

    try:
        # Kjør analysen
        resultat, figur = run_linear_regression_analysis(dager, vekt)

        # Print resultater
        print(f"Lineær funksjon: y(x) = {resultat.slope:.2f} * x + {resultat.intercept:.2f}")
        print(f"R²-verdi: {resultat.rvalue**2:.2f}\n")

        # Vis plot
        plt.show()

        # Eksempel på prediksjon
        x_pred = 60
        y_pred = resultat.slope * x_pred + resultat.intercept
        print(f"Forutsigelse: Ved {x_pred} dager er forventet vekt {y_pred:.2f} gram.")

    except ValueError as e:
        print(f"En feil oppstod: {e}")

In [ ]:
# Eksponentialregresjon
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math

# Datasett
x = np.array([0, 1, 2, 3, 4, 5])                                              # Endre tallene så de passer til din oppgave
y = np.array([5335, 7035, 9467, 12780, 17208, 24008])                            # Endre tallene så de passer til din oppgave

# Utfør eksponentiell regresjon ved hjelp av np.polyfit
log_y = np.log(y)
coeffs = np.polyfit(x, log_y, 1)
a = np.exp(coeffs[1])
b = np.exp(coeffs[0])

# Skriv ut eksponentiell funksjon med LaTeX
latex_str = f"Eksponentialfunksjon: f(x) = {a:.1f} \\cdot {b:.3f}^x"
display(Math(latex_str))

# Skriv ut eksponentiell funksjon uten LaTeX
print(f"Eksponentialfunksjon: f(x) = {a:.1f} * {b:.3f}^x")

# Plot datasettet og eksponentiell funksjon
plt.scatter(x, y, label='Inndata')
plt.plot(x, a * b ** x, label='Eksponentiell funksjon', color="r")
plt.xlabel('Antall år etter 2015')                                                       # x-aksen navn
plt.ylabel('Totalt antall besøkende til attraksjoner')                                  # y-aksen navn
plt.legend()
plt.grid(False)
plt.title("Turistbesøk til attraksjoner i nærområdet")                                 # Overskrift
plt.show()

# Beregn vekstfaktor og prosentvis økning/minking
vekstfaktor = b
prosentvis_endring = (vekstfaktor - 1) * 100
print(f"Vekstfaktor b: {vekstfaktor:.3f}")
print(f"Prosentvis endring i hele dataperioden per x-enhet er: {prosentvis_endring:.2f} %")
print(f"Startverdi a er: {a:.1f}")

# Finn y-verdien når x = …
x_val = 0                                                    # Endre denne x verdien for å finne tilhørende y verdi
y_val = a * (b ** x_val)
print(f"y-verdi for x={x_val}: {y_val:.2f}")

# Finn x-verdien når y = ...
y_val = 500                                                  # Endre denne y verdien for å finne tilhørende x verdi
x_val = np.log(y_val/a) / np.log(b)
print(f"x-verdi for y={y_val:.2f}: {x_val:.2f}")

# Definer funksjoner
def power_func(x, a, b):
    return a * (b ** x)

def power_func_derivative(x, a, b):
    return a * b ** x * np.log(b)

# Finn stigningstallet mellom to punkter (rett linje)
x1 = 1                                                     # Endre denne x verdien for å finne stigningstalllet mellom 2 punkter
x2 = 6                                                      # Endre denne x verdien for å finne stigningstalllet mellom 2 punkter

y1 = power_func(x1, a, b)
y2 = power_func(x2, a, b)

m = (y2 - y1) / (x2 - x1)
print("Stigningstallet mellom punktene ({:.2f}, {:.2f}) og ({:.2f}, {:.2f}) er: {:.1f}".format(x1, y1, x2, y2, m))

# Finn stigningstallet til tangenten i et valgt punkt
x3 = 1                                                        # Endre denne x verdien for å finne stigningstallet til tangenten i ett gitt punkt
y3 = power_func(x3, a, b)

m_tangent = power_func_derivative(x3, a, b)
print("Stigningstallet til tangenten i punktet ({}, {:.2f}) er: {:.1f}".format(x3, y3, m_tangent))

# Finn stigningstallet til tangenten i et valgt punkt
x4 = 6                                                       # Endre denne x verdien for å finne stigningstallet til tangenten i ett gitt punkt
y4 = power_func(x4, a, b)

m_tangent = power_func_derivative(x4, a, b)
print("Stigningstallet til tangenten i punktet ({}, {:.2f}) er: {:.1f}".format(x4, y4, m_tangent))

# Beregn R²-verdien
y_pred = a * b ** x
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r2 = 1 - (ss_res / ss_tot)
print(f"R²-verdi: {r2:.2f}")

In [ ]:
# Oppgave 6.41 Eksponentialfunksjon på antall innbyggere etter x-antall år og dato
import numpy as np
from datetime import datetime
from IPython.display import display, Math

# Funksjon for å beregne innbyggertallet
def beregn_innbyggertall(start_aar, start_innbyggertall, vekstfaktor, aar):
    x = aar - start_aar
    return start_innbyggertall * (vekstfaktor ** x)

# Funksjon for å beregne innbyggertallet på en bestemt dato
def beregn_innbyggertall_dato(start_aar, start_innbyggertall, vekstfaktor, dato):
    start_dato = datetime(start_aar, 1, 1)
    dager_forskjell = (dato - start_dato).days
    aar_forskjell = dager_forskjell / 365  # Gjennomsnittlig antall dager i et år
    return start_innbyggertall * (vekstfaktor ** aar_forskjell)

# Startår og innbyggertall
start_aar = 2017                                                                            # Start år
start_innbyggertall = 34066                                                                 # Start inbyggertall
vekstfaktor = 1.013                                                                        # Vekstfaktor på 1.3% per år

# Funksjonsuttrykket f(x) med Latex
display(Math(f"f(x) = {start_innbyggertall} \\cdot {vekstfaktor}^x"))  # Forandre utskriften

# Skriv ut funksjonsuttrykket uten LaTeX-syntaks
print(f"Funksjonsuttrykk: f(x) = {start_innbyggertall} * {vekstfaktor}^x")

# Beregn innbyggertallet for en bestemt dato
onsket_dato = datetime(2020, 10, 1)  # Bytt ut med ønsket (år, måned og dato)
innbyggertall_dato = beregn_innbyggertall_dato(start_aar, start_innbyggertall, vekstfaktor, onsket_dato)
dato_str = onsket_dato.strftime('%d. %B %Y').lstrip('0')
print(f"Innbyggertallet {dato_str} var {innbyggertall_dato:.0f}")

In [ ]:
# Polynomregresjon hvor du selv må velge polynomgrad altså: grad 2 for x^2, grad 3 for x^3 osv...
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math

# Dummy data
x = np.array([0, 4, 8, 12, 16, 20, 24])                      # Endre tallene så de passer til din oppgave
y = np.array([8, 7, 11, 16, 20, 17, 9])                      # Endre tallene så de passer til din oppgave

# Funksjon for å beregne polynomregresjon og la brukeren velge grad
def polynomial_regression(x, y, n):
    # Beregn polynomkoeffisientene for valgt grad
    coefficients = np.polyfit(x, y, n)
    p = np.poly1d(coefficients)
    
    # Skriv ut polynomfunksjonen med LaTeX-syntaks
    poly_str = " + ".join([f"{coeff:.4f}x^{i}" if i > 0 else f"{coeff:.4f}" for i, coeff in enumerate(coefficients[::-1])])
    poly_str = poly_str.replace("x^1", "x")  # Fjern "x^1"
    poly_str = poly_str.replace("x^0", "")   # Fjern "x^0"
    poly_str = poly_str.replace(" + -", " - ")  # Forhindre feil ved negative koeffisienter
    display(Math(f"Polynomfunksjon: f(x) = {poly_str}"))
    
    # Skriv ut polynomfunksjonen uten LaTeX-syntaks
    print(f"Polynomfunksjon: f(x) = {poly_str}")
    
    return p

# ----------------------------- Lar brukeren selv velge grad på polynomet, endre verdien på n ----------------------------------------------------------
n = 3
# ----------------------------- Lar brukeren selv velge grad på polynomet, endre verdien på n ----------------------------------------------------------

# Beregn polynomregresjon med brukerens valgte grad
p = polynomial_regression(x, y, n)

# Lag et plott av dataene og polynomregresjonen
x_plot = np.linspace(min(x), max(x), 100)
plt.scatter(x, y, label='Inndata')
plt.plot(x_plot, p(x_plot), color='red', label=f'Polynomgrad {n}')

# Sett aksetitler
plt.xlabel("Antall timer etter midnatt")                              # Navn på x-aksen
plt.ylabel("Temperatur i °C")                                         # Navn på y-aksen

# Legg til en overskrift
plt.title("Temperatur i °C en sommerdag")                            # Navn på overskrift
plt.legend()
plt.grid(False)
plt.show()

# Finn y-verdien når x = ...
x_verdi = 16                                                         # Endre denne x verdien for å finne tilhørende y verdi
y_verdi = p(x_verdi)
print(f"y-verdien når x = {x_verdi:.2f} er {y_verdi:.2f}")

# Finn x-verdiene når y = ...
y_verdi = 12                                                         # Endre denne y verdien for å finne tilhørende x verdi

# Funksjon for å finne x-verdi for en gitt y-verdi ved bruk av polynomregresjonsmodellen
def find_x_for_y(p, y_value):
    coefficients = p.coefficients
    coefficients[-1] -= y_value
    roots = np.roots(coefficients)
    
    # Filtrer ut kun de reelle røttene
    real_roots = [root.real for root in roots if np.isreal(root)]
    return real_roots

x_values_for_y = find_x_for_y(p, y_verdi)
print(f"x-verdiene når y = {y_verdi} er {[round(float(x), 2) for x in x_values_for_y]}")

# Finn stigningstallet mellom to punkter (rett linje)
x1 = 1                                                    # Endre denne x verdien for å finne stigningstsallet mellom to x-verdier eller punkter             
x2 = 20                                                   # Endre denne x verdien for å finne stigningstsallet mellom to x-verdier eller punkter 
y1 = p(x1)
y2 = p(x2)

m = (y2 - y1) / (x2 - x1)

print("Stigningstallet mellom punktene ({:.2f}, {:.2f}) og ({:.2f}, {:.2f}) er: {:.2f}".format(x1, y1, x2, y2, m))

# Finn stigningstallet til tangenten i et valgt punkt
x3 = 6                                                 #  Endre denne x-verdien for å finne stigningstallet til tangenten i punktet
y3 = p(x3)

m_tangent = np.polyder(p)(x3)

print("Stigningstallet til tangenten i punktet ({:.2f}, {:.2f}) er: {:.2f}".format(x3, y3, m_tangent))

# Finn nullpunkter innenfor et definert intervall
def find_roots_in_interval(p, interval):
    roots = np.roots(p.coefficients)
    real_roots_in_interval = [root.real for root in roots if np.isreal(root) and interval[0] <= root.real <= interval[1]]
    return real_roots_in_interval

interval = (0, 24)
roots_in_interval = find_roots_in_interval(p, interval)
print(f"Nullpunkter innenfor intervallet {interval} er: {[round(float(root), 2) for root in roots_in_interval]}")

# Finn ekstremalpunkter innenfor et definert intervall
def find_extrema_in_interval(p, interval):
    derivative_coefficients = np.polyder(p).coefficients
    critical_points = np.roots(derivative_coefficients)
    real_critical_points_in_interval = [point.real for point in critical_points if np.isreal(point) and interval[0] <= point.real <= interval[1]]
    extrema_points_in_interval = [(point.real, p(point.real)) for point in real_critical_points_in_interval]
    return extrema_points_in_interval
extrema_in_interval = find_extrema_in_interval(p, interval)
print(f"Ekstremalpunkter innenfor intervallet {interval} er: {[(round(float(point[0]), 2), round(float(point[1]), 2)) for point in extrema_in_interval]}")

# Beregn R²
y_pred = p(x)
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r2 = 1 - (ss_res / ss_tot)
print(f"R²-verdi: {r2:.2f}")

In [ ]:
# Potensregresjon med dummy data + stigningstallet til tangenten i ett valgt punkt og mellom to punkter - kode
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math

# Datasett
x = [1, 8, 22, 36, 50, 64]                        # Endre tallene så de passer til din oppgave
y = [10, 20, 28, 33, 37, 40]                      # Endre tallene så de passer til din oppgave

# Konverter y-verdiene til logaritmisk form
log_y = np.log10(y)

# Konverter x-verdiene til logaritmisk form
log_x = np.log10(x)

# Beregn gjennomsnittet av de logaritmiske verdiene
log_y_mean = np.mean(log_y)
log_x_mean = np.mean(log_x)

# Beregn variansen til de logaritmiske x-verdiene
var_log_x = np.sum((log_x - log_x_mean) ** 2)

# Beregn kovariansen mellom de logaritmiske x- og y-verdiene
cov_log_xy = np.sum((log_x - log_x_mean) * (log_y - log_y_mean))

# Beregn koeffisientene a og b
b = cov_log_xy / var_log_x
a = 10 ** (log_y_mean - b * log_x_mean)

# Potensfunksjonen
def power_func(x, a, b):
    return a * x ** b

# Derivert potensfunksjon
def power_func_derivative(x, a, b):
    return a * b * x ** (b - 1)

# Skriv ut potensfunksjonen
latex_str = f"Potensfunksjon: f(x) = {a:.2f} \\cdot x^{{{b:.3f}}}"
display(Math(latex_str))
print(f"Potensfunksjon: f(x) = {a:.2f}*x^{b:.3f}")

# Plot datasettet og potensfunksjonen
plt.scatter(x, y, label='Inndata')
plt.plot(x, power_func(np.array(x), a, b), label='Potensfunksjon', color="r")
plt.xlabel('Dager')                                                                      # Navn på x-aksen
plt.ylabel('Antall minutter')                                                            # Navn på y-aksen
plt.legend()
plt.title('Hanne løper sammenhengende')                                                  # Overskriften på grafen
plt.grid(False)
plt.show()

# Finn y-verdien når x = ...
x_verdi = 0
y_verdi = power_func(x_verdi, a, b)
print(f"y-verdien når x = {x_verdi} er {y_verdi:.2f}")

# Finn x-verdien når y = ...
y_verdi = 45                                                                 # Endre dette tallet for å finne x-verdi fra en y verdi
x_verdi = (y_verdi / a) ** (1 / b)
print(f"x-verdien når y = {y_verdi} er {x_verdi:.2f}")

# Finn stigningstallet/gjennomsnitt mellom to punkter (rett linje)
x1 = 1                                                                  # Velg valgfri x verdi for å finne stigningstallet mellom 2 punkter
x2 = 60                                                                 # Velg valgfri x verdi for å finne stigningstallet mellom 2 punkter

y1 = power_func(x1, a, b)
y2 = power_func(x2, a, b)

# Beregn stigningstallet
m = (y2 - y1) / (x2 - x1)

# Skriv ut stigningstallet
print("Stigningstallet mellom punktene ({:.2f}, {:.2f}) og ({:.2f}, {:.2f}) er: {:.2f}".format(x1, y1, x2, y2, m))

# Finn stigningstallet til tangenten i et valgt punkt
x3 = 14                                                            # Endre denne x-verdien for å finne stigningstallet til tangenten i punktet
y3 = power_func(x3, a, b)                               

# Beregn stigningstallet i punktet (x3, y3)
m_tangent = power_func_derivative(x3, a, b)

# Skriv ut stigningstallet
print("Stigningstallet til tangenten i punktet ({}, {:.2f}) er: {:.2f}".format(x3, y3, m_tangent))

# Beregn R^2 (det bestemte koeffisienten)
y_pred = power_func(np.array(x), a, b)  
ss_residual = np.sum((y - y_pred) ** 2) 
ss_total = np.sum((y - np.mean(y)) ** 2)
r_squared = 1 - (ss_residual / ss_total)

# Skriv ut R^2
print(f"R²-verdi: {r_squared:.2f}")

In [ ]:
# Søyle, sektor og linjediagram med 1 sett med frekvenser, Farger : https://matplotlib.org/stable/gallery/color/named_colors.html
import matplotlib.pyplot as plt

# Liste med navn og frekvenser
navn = ["Eple", "Pære", "Granateple", "Kiwi", "Banan"]        # Endre tallene så de passer til din oppgave
frekvenser = [20, 30, 10, 25, 15]                             # Endre tallene så de passer til din oppgave

# Fargevalg
farger = ['hotpink','coral','yellowgreen','b','g']

# Beregn totalfrekvensen
total = sum(frekvenser)

# Beregn andelene (i prosent) for hver sektor
andelene = [100 * frek / total for frek in frekvenser]

# Kakediagram
plt.pie(andelene, labels=navn, autopct='%1.1f%%', colors=farger)
plt.title("Oversikt over frukt")                                  # Endre overskriften 
plt.axis('equal')
plt.show()

# Søylediagram
plt.bar(navn, frekvenser, color=farger)
plt.title("Frekvenser av navn")                                  # Endre overskriften 
plt.xlabel("Frukt")                                              # Endre x-akse navnet
plt.ylabel("Frekvens")                                           # Endre y-akse navnet
plt.ylim(ymin=0)
plt.show()

# Linjediagram
plt.plot(navn, frekvenser, marker='o', color='#ff7f0e')
plt.title("Frekvenser av navn")                                  # Endre overskriften 
plt.xlabel("Navn")                                               # Endre x-akse navnet
plt.ylabel("Frekvens")                                           # Endre y-akse navnet
plt.ylim(ymin=0)
plt.show()

In [ ]:
# Søyle, sektor og linjediagram med 2 sett med frekvenser + Frekvenstabell med relative og kumulative frekvenser
import matplotlib.pyplot as plt
import numpy as np

# --------------------------------------------------------------------------
# DATA OG FARGEVALG
# --------------------------------------------------------------------------
# Her kan du endre dataene slik at de passer til din oppgave.
navn = ["A", "B", "C", "D", "E"]
frekvenser_1 = [3, 6, 5, 5, 3]
frekvenser_2 = [7, 9, 6, 7, 12]

# Fargevalg for diagrammene
farger_1 = 'red'
farger_2 = 'blue'
farger_sektor = ['#66b3ff', '#99ff99', '#ffcc99', '#ffadad', '#c2c2f0']

# --------------------------------------------------------------------------
# BEREGNINGER
# --------------------------------------------------------------------------
# Beregn sum, relative frekvenser og kumulative frekvenser

# Summer
total_frekvenser_1 = sum(frekvenser_1)
total_frekvenser_2 = sum(frekvenser_2)

# Relative frekvenser (i prosent)
relative_frekvenser_1 = [(f / total_frekvenser_1) * 100 for f in frekvenser_1]
relative_frekvenser_2 = [(f / total_frekvenser_2) * 100 for f in frekvenser_2]

# Kumulative frekvenser (oppsummering)
kumulative_frekvenser_1 = np.cumsum(frekvenser_1)
kumulative_frekvenser_2 = np.cumsum(frekvenser_2)

# --------------------------------------------------------------------------
# 1. GRUPPERT SØYLEDIAGRAM
# Gir en god sammenligning av de to frekvenssettene side om side.
# --------------------------------------------------------------------------
x_pos = np.arange(len(navn))  # Posisjonene til gruppene på x-aksen
bredde = 0.35  # Bredden på søylene

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x_pos - bredde/2, frekvenser_1, bredde, label='Frekvenser 1', color=farger_1)
rects2 = ax.bar(x_pos + bredde/2, frekvenser_2, bredde, label='Frekvenser 2', color=farger_2)

# Legg til tittel og akse-navn
ax.set_title("1. Sammenligning av frekvenser (Gruppert)")
ax.set_xlabel("Navn")
ax.set_ylabel("Frekvens")
ax.set_xticks(x_pos)
ax.set_xticklabels(navn)
ax.legend()
plt.ylim(0, max(max(frekvenser_1), max(frekvenser_2)) + 3) # Setter y-aksen litt høyere enn høyeste verdi
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# --------------------------------------------------------------------------
# 2. SEKTORDIAGRAM (KAKEDIAGRAM) - PER FREKVENSSETT
# Viser fordelingen innad i hvert frekvenssett.
# --------------------------------------------------------------------------
plt.figure(figsize=(14, 7))
plt.suptitle("2. Fordeling innad i hvert frekvenssett", fontsize=16)

# Første diagram
plt.subplot(1, 2, 1) # 1 rad, 2 kolonner, plott nr. 1
plt.pie(frekvenser_1, labels=navn, autopct='%1.1f%%', colors=farger_sektor, startangle=90)
plt.title("Frekvenser 1")
plt.axis('equal')  # Sikrer at kakediagrammet er en sirkel

# Andre diagram
plt.subplot(1, 2, 2) # 1 rad, 2 kolonner, plott nr. 2
plt.pie(frekvenser_2, labels=navn, autopct='%1.1f%%', colors=farger_sektor, startangle=90)
plt.title("Frekvenser 2")
plt.axis('equal')

plt.show()

# --------------------------------------------------------------------------
# 3. LINJEDIAGRAM
# Viser trenden eller endringen over kategoriene.
# --------------------------------------------------------------------------
plt.figure(figsize=(10, 6))
plt.plot(navn, frekvenser_1, marker='o', color=farger_1, label='Frekvenser 1')
plt.plot(navn, frekvenser_2, marker='o', color=farger_2, label='Frekvenser 2')

plt.title("3. Frekvenser som linjediagram")
plt.xlabel("Navn")
plt.ylabel("Frekvens")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.ylim(ymin=0)
plt.show()

# --------------------------------------------------------------------------
# 4. FREKVENSTABELL
# --------------------------------------------------------------------------
table_data = []
for i in range(len(navn)):
    row = [navn[i], frekvenser_1[i], frekvenser_2[i], f'{relative_frekvenser_1[i]:.1f}%',
           f'{relative_frekvenser_2[i]:.1f}%', kumulative_frekvenser_1[i], kumulative_frekvenser_2[i]]
    table_data.append(row)

sum_row = ['Sum', total_frekvenser_1, total_frekvenser_2, f'{sum(relative_frekvenser_1):.1f}%',
           f'{sum(relative_frekvenser_2):.1f}%', '-', '-']
table_data.append(sum_row)

table_columns = ['Navn', 'Frekvens 1', 'Frekvens 2', 'Relativ Frek. 1', 'Relativ Frek. 2',
                  'Kumulativ Frek. 1', 'Kumulativ Frek. 2']

fig, ax = plt.subplots(figsize=(12, 4))
ax.axis('tight'); ax.axis('off')
the_table = ax.table(cellText=table_data, colLabels=table_columns, loc='center', cellLoc='center')
the_table.auto_set_font_size(False); the_table.set_fontsize(12); the_table.scale(1.2, 1.8)
plt.title("4. Frekvenstabell", fontsize=16, y=1.05)
plt.show()

# --------------------------------------------------------------------------
# 5. STABLET SØYLEDIAGRAM (OVERLAPPENDE) - EKSTRA
# Viser hvordan de to frekvenssettene bygger på hverandre for hver kategori.
# --------------------------------------------------------------------------
plt.figure(figsize=(10, 6))
# Plott den ene serien i bunnen, og den andre på toppen ved hjelp av 'bottom'
plt.bar(navn, frekvenser_2, color=farger_2, label='Frekvenser 2')
plt.bar(navn, frekvenser_1, bottom=frekvenser_2, color=farger_1, label='Frekvenser 1')

plt.title("5. Stablet søylediagram av frekvenser")
plt.xlabel("Navn")
plt.ylabel("Total frekvens (stablet)")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# --------------------------------------------------------------------------
# 6. SEKTORDIAGRAM AV TOTAL SUM - EKSTRA
# Viser forholdet mellom den totale summen av frekvenser_1 og frekvenser_2.
# --------------------------------------------------------------------------
plt.figure(figsize=(8, 8))
summer = [total_frekvenser_1, total_frekvenser_2]
etiketter = [
    f'Total Frekvenser 1 ({total_frekvenser_1})',
    f'Total Frekvenser 2 ({total_frekvenser_2})'
]
farger_sum = ['gold', 'lightskyblue']

plt.pie(summer, labels=etiketter, autopct='%1.1f%%', colors=farger_sum, startangle=140,
        textprops={'fontsize': 12})
plt.title("6. Forholdet mellom total sum av frekvensene", fontsize=16)
plt.axis('equal')  # Sikrer en sirkel
plt.show()

In [ ]:
# Funksjonskalkulator Bjarne 1.0
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brentq
import ipywidgets as widgets
from IPython.display import display, clear_output

# Trygg evaluering av funksjon
def evaluer_funksjon(funksjon_str, x_verdi):
    funksjon_str = funksjon_str.replace("^", "**").replace("ln", "np.log")
    allowed_names = {
        "np": np,
        "sqrt": np.sqrt,
        "abs": np.abs,
        "sin": np.sin,
        "cos": np.cos,
        "tan": np.tan,
        "exp": np.exp,
        "log": np.log,
        "pi": np.pi,
        "e": np.e,
        "x": x_verdi
    }
    try:
        return eval(funksjon_str, {"__builtins__": {}}, allowed_names)
    except Exception:
        return np.full_like(x_verdi, np.nan) if isinstance(x_verdi, np.ndarray) else np.nan

# Numerisk derivasjon
def numerisk_derivert(funksjon_str, x, dx=1e-5):
    return (evaluer_funksjon(funksjon_str, x + dx) - evaluer_funksjon(funksjon_str, x - dx)) / (2 * dx)

# Nullpunkter
def finn_nullpunkter(funksjon_str, x_vals, y_vals):
    nullpunkter = []
    for i in range(len(x_vals) - 1):
        if y_vals[i] * y_vals[i + 1] < 0:
            try:
                x0 = brentq(lambda x: evaluer_funksjon(funksjon_str, x), x_vals[i], x_vals[i + 1])
                nullpunkter.append(x0)
            except Exception:
                continue
    return nullpunkter

# Ekstremalpunkter
def finn_ekstremalpunkter(funksjon_str, x_vals):
    ekstremalpunkter = []
    for i in range(len(x_vals) - 1):
        df1 = numerisk_derivert(funksjon_str, x_vals[i])
        df2 = numerisk_derivert(funksjon_str, x_vals[i + 1])
        if df1 * df2 < 0:
            try:
                x0 = brentq(lambda x: numerisk_derivert(funksjon_str, x), x_vals[i], x_vals[i + 1])
                ekstremalpunkter.append(x0)
            except Exception:
                continue
    return ekstremalpunkter

# Tegn graf
def tegn_graf(funksjon_str, x_min, x_max, tittel, x_label, y_label):
    x_vals = np.linspace(x_min, x_max, 400)
    y_vals = evaluer_funksjon(funksjon_str, x_vals)
    
    plt.figure(figsize=(10, 5))
    plt.plot(x_vals, y_vals, label=f"f(x) = {funksjon_str}")
    
    # Nullpunkter
    nullpunkter = finn_nullpunkter(funksjon_str, x_vals, y_vals)
    for npunkt in nullpunkter:
        plt.plot(npunkt, 0, 'ro')
        plt.annotate(f"x ≈ {npunkt:.2f}", (npunkt, 0), textcoords="offset points", xytext=(5,10), ha='left', color='red')
        display(widgets.HTML(f"🔵 Nullpunkt ved x ≈ {npunkt:.2f}"))
    
    # Ekstremalpunkter
    ekstremalpunkter = finn_ekstremalpunkter(funksjon_str, x_vals)
    for x_ekstrem in ekstremalpunkter:
        y_ekstrem = evaluer_funksjon(funksjon_str, x_ekstrem)
        plt.plot(x_ekstrem, y_ekstrem, 'go')
        plt.annotate(f"({x_ekstrem:.2f}, {y_ekstrem:.2f})", (x_ekstrem, y_ekstrem), textcoords="offset points", xytext=(5,10), ha='left', color='green')
        display(widgets.HTML(f"🟢 Ekstremalpunkt ved x ≈ {x_ekstrem:.2f}, f(x) ≈ {y_ekstrem:.2f}"))
    
    # Forklaringsmarkører i legenden
    if nullpunkter:
        plt.plot([], [], 'ro', label='Nullpunkt')
    if ekstremalpunkter:
        plt.plot([], [], 'go', label='Ekstremalpunkt')

    plt.title(tittel)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.grid(True)
    plt.legend(loc="best")
    plt.show()

# Analyse-knapp
out = widgets.Output()

def kalkuler(_=None):
    with out:
        clear_output(wait=True)
        display(widgets.HTML("<h2>Interaktiv funksjonsanalyse</h2>"))

        funksjon_str = funksjon_input.value
        try:
            x_min = float(x_min_input.value)
            x_max = float(x_max_input.value)
        except Exception:
            display(widgets.HTML("<b style='color:red;'>⚠️ Feil i intervallverdier!</b>"))
            return

        x_vals = np.linspace(x_min, x_max, 400)
        y_vals = evaluer_funksjon(funksjon_str, x_vals)

        tegn_graf(funksjon_str, x_min, x_max, tittel_input.value, x_akse_input.value, y_akse_input.value)

        try:
            if vekst_checkbox.value:
                y0, y1 = evaluer_funksjon(funksjon_str, 0), evaluer_funksjon(funksjon_str, 1)
                vekstfaktor = y1 / y0
                prosent = (vekstfaktor - 1) * 100
                import re
                match = re.search(r'([\d\.]+)\s*\*\s*([\d\.]+)\s*\*\*\s*x', funksjon_str.replace(' ', ''))
                if match:
                    r = float(match.group(2))
                    ekte_prosent = (r - 1) * 100
                    display(widgets.HTML(f"📉 <b>Den ekte vekstfaktoren fra funksjonen:</b> {r:.3f} → {ekte_prosent:.2f}% per x"))
                display(widgets.HTML(f"📈 <b>Effektiv vekstfaktor i perioden:</b> {vekstfaktor:.3f}, Prosentendring: {prosent:.2f}%"))

            if prosent_okning_checkbox.value:
                try:
                    prosent = float(prosent_okning_input.value)
                    y0 = evaluer_funksjon(funksjon_str, 0)
                    y_target = y0 * (1 + prosent / 100)
                    x_result = brentq(lambda x: evaluer_funksjon(funksjon_str, x) - y_target, x_min, x_max)
                    display(widgets.HTML(f"📈 {prosent}% økning skjer ved x ≈ {x_result:.2f}"))
                except Exception as e:
                    display(widgets.HTML(f"⚠️ Feil ved prosentvis økning: {e}"))
        
            if y_fra_x_checkbox.value:
                x_val = float(x_verdi_input.value)
                y_val = evaluer_funksjon(funksjon_str, x_val)
                display(widgets.HTML(f"🔎 f({x_val}) = {y_val:.2f}"))

            if x_fra_y_checkbox.value:
                y_target = float(y_verdi_input.value)
                x_result = brentq(lambda x: evaluer_funksjon(funksjon_str, x) - y_target, x_min, x_max)
                display(widgets.HTML(f"🔁 f(x) = {y_target} ⇨ x ≈ {x_result:.2f}"))

            if tangent_checkbox.value:
                x0 = float(tangent_x_input.value)
                df = numerisk_derivert(funksjon_str, x0)
                display(widgets.HTML(f"📐 Tangentens stigning ved x = {x0}: {df:.2f}"))

            if stigning_checkbox.value:
                x1 = float(stigning_x1_input.value)
                x2 = float(stigning_x2_input.value)
                y1 = evaluer_funksjon(funksjon_str, x1)
                y2 = evaluer_funksjon(funksjon_str, x2)
                stigning = (y2 - y1) / (x2 - x1)
                display(widgets.HTML(f"📊 Stigning mellom x = {x1} og x = {x2}: {stigning:.2f}"))

            if diff_checkbox.value:
                x1 = float(diff_x1_input.value)
                x2 = float(diff_x2_input.value)
                diff = evaluer_funksjon(funksjon_str, x2) - evaluer_funksjon(funksjon_str, x1)
                display(widgets.HTML(f"➖ Forskjell: f({x2}) - f({x1}) = {diff:.2f}"))

            if prosent_checkbox.value:
                x1 = float(prosent_x1_input.value)
                x2 = float(prosent_x2_input.value)
                y1 = evaluer_funksjon(funksjon_str, x1)
                y2 = evaluer_funksjon(funksjon_str, x2)
                prosent_endring = ((y2 - y1) / y1) * 100
                display(widgets.HTML(f"📈 Prosentvis endring fra x={x1} til x={x2}: {prosent_endring:.2f}%"))

            if prosent_reduksjon_checkbox.value:
                prosent = float(prosent_reduksjon_input.value)
                y0 = evaluer_funksjon(funksjon_str, 0)
                y_target = y0 * (1 - prosent / 100)
                x_result = brentq(lambda x: evaluer_funksjon(funksjon_str, x) - y_target, x_min, x_max)
                display(widgets.HTML(f"⏳ {prosent}% reduksjon skjer ved x ≈ {x_result:.2f}"))

            if terskel_checkbox.value:
                y_terskel = float(terskel_input.value)
                x_result = brentq(lambda x: evaluer_funksjon(funksjon_str, x) - y_terskel, x_min, x_max)
                display(widgets.HTML(f"📌 f(x) = {y_terskel} oppnås ved x ≈ {x_result:.2f}"))

        except Exception as e:
            display(widgets.HTML(f"<b style='color:red;'>⚠️ Feil under analyse: {e}</b>"))

# Widgets
funksjon_input = widgets.Text(value="6000 * 0.958**x", description="f(x) =")
x_min_input = widgets.FloatText(value=0, description="x min:")
x_max_input = widgets.FloatText(value=30, description="x max:")
tittel_input = widgets.Text(value="Funksjonsgraf", description="Tittel:")
x_akse_input = widgets.Text(value="x", description="x-akse:")
y_akse_input = widgets.Text(value="f(x)", description="y-akse:")

vekst_checkbox = widgets.Checkbox(value=True, description="Vekstfaktor")
y_fra_x_checkbox = widgets.Checkbox(value=False, description="Finn en y-verdi fra en x-verdi")
x_verdi_input = widgets.FloatText(value=10)
x_fra_y_checkbox = widgets.Checkbox(value=False, description="Finn en x-verdi fra en y-verdi")
y_verdi_input = widgets.FloatText(value=3000)
tangent_checkbox = widgets.Checkbox(value=False, description="Tangenten i en valgt x-verdi")
tangent_x_input = widgets.FloatText(value=5)
stigning_checkbox = widgets.Checkbox(value=False, description="Stigningstall mellom 2 x-verdier")
stigning_x1_input = widgets.FloatText(value=0)
stigning_x2_input = widgets.FloatText(value=10)
diff_checkbox = widgets.Checkbox(value=False, description="Forskjell 2 y-verdier")
diff_x1_input = widgets.FloatText(value=0)
diff_x2_input = widgets.FloatText(value=10)
prosent_checkbox = widgets.Checkbox(value=False, description="Prosentvis endring mellom to x-verdier")
prosent_x1_input = widgets.FloatText(value=0)
prosent_x2_input = widgets.FloatText(value=10)
prosent_reduksjon_checkbox = widgets.Checkbox(value=False, description="% reduksjon")
prosent_okning_checkbox = widgets.Checkbox(value=False, description="% økning")
prosent_okning_input = widgets.FloatText(value=20)
prosent_reduksjon_input = widgets.FloatText(value=20)
terskel_checkbox = widgets.Checkbox(value=False, description="Terskelverdi")
terskel_input = widgets.FloatText(value=3000)

knapp = widgets.Button(description="Kjør analyse")
knapp.on_click(kalkuler)

display(widgets.HTML("<h1>Interaktiv funksjonsanalyse</h1>"))

# Grunninnstillinger
grunninnstillinger = widgets.VBox([
    widgets.HBox([funksjon_input]),
    widgets.HBox([x_min_input, x_max_input]),
    widgets.HBox([tittel_input, x_akse_input, y_akse_input])
])

# Direkte verdier
direkte_analyse = widgets.VBox([
    widgets.HTML("<h3>🔎 Direkte verdier</h3>"),
    vekst_checkbox,
    widgets.HBox([y_fra_x_checkbox, widgets.Label("x verdi:"), x_verdi_input]),
    widgets.HBox([x_fra_y_checkbox, widgets.Label("y verdi:"), y_verdi_input])
])

# Tangent og stigning
tangent_stigning = widgets.VBox([
    widgets.HTML("<h3>📐 Tangent og stigning</h3>"),
    widgets.HBox([tangent_checkbox, widgets.Label("x for tangent:"), tangent_x_input]),
    stigning_checkbox,
    widgets.HBox([widgets.Label("x1:"), stigning_x1_input, widgets.Label("x2:"), stigning_x2_input])
])

# Forskjell og prosentendring
forskjell_prosent = widgets.VBox([
    widgets.HTML("<h3>📊 Forskjell og prosent</h3>"),
    diff_checkbox,
    widgets.HBox([widgets.Label("x1:"), diff_x1_input, widgets.Label("x2:"), diff_x2_input]),
    prosent_checkbox,
    widgets.HBox([widgets.Label("x1:"), prosent_x1_input, widgets.Label("x2:"), prosent_x2_input])
])

# Økning / Reduksjon
vekst_reduksjon = widgets.VBox([
    widgets.HTML("<h3>📈 Økning / 📉 Reduksjon</h3>"),
    widgets.HBox([prosent_okning_checkbox, widgets.Label("% økning:"), prosent_okning_input]),
    widgets.HBox([prosent_reduksjon_checkbox, widgets.Label("% reduksjon:"), prosent_reduksjon_input])
])

# Terskelverdi
terskelgruppe = widgets.VBox([
    widgets.HTML("<h3>📌 Terskelverdi</h3>"),
    widgets.HBox([terskel_checkbox, widgets.Label("y terskel:"), terskel_input])
])

# Alt sammen
full_gui = widgets.VBox([
    grunninnstillinger,
    direkte_analyse,
    tangent_stigning,
    forskjell_prosent,
    vekst_reduksjon,
    terskelgruppe,
    knapp,
    out
])

display(full_gui)

In [ ]:
# Funksjonskalkulator Bjarne 2.0
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brentq
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Kjernefunksjoner for analyse ---

class FunksjonsAnalyse:
    """En klasse for å holde på funksjonen og utføre beregninger."""
    def __init__(self, funksjon_str, x_min, x_max):
        self.funksjon_str = funksjon_str.replace("^", "**").replace("ln", "np.log")
        self.x_min = x_min
        self.x_max = x_max
        self.allowed_names = {
            "np": np, "sqrt": np.sqrt, "abs": np.abs, "sin": np.sin,
            "cos": np.cos, "tan": np.tan, "exp": np.exp, "log": np.log,
            "pi": np.pi, "e": np.e, "x": None
        }
        self.x_vals = np.linspace(x_min, x_max, 500)
        self.y_vals = self.evaluer(self.x_vals)

    def evaluer(self, x_verdi):
        self.allowed_names["x"] = x_verdi
        try:
            # Sikker evaluering
            return eval(self.funksjon_str, {"__builtins__": {}}, self.allowed_names)
        except Exception:
            # Returner NaN (Not a Number) ved feil, som håndteres pent av Matplotlib
            return np.full_like(x_verdi, np.nan) if isinstance(x_verdi, np.ndarray) else np.nan

    def derivert(self, x, dx=1e-6):
        return (self.evaluer(x + dx) - self.evaluer(x - dx)) / (2 * dx)

    def finn_nullpunkter(self):
        punkter = []
        for i in range(len(self.x_vals) - 1):
            if self.y_vals[i] * self.y_vals[i+1] < 0:
                try:
                    rot = brentq(self.evaluer, self.x_vals[i], self.x_vals[i+1])
                    punkter.append(rot)
                except (ValueError, RuntimeError):
                    continue
        return punkter

    def finn_ekstremalpunkter(self):
        punkter = []
        deriverte = self.derivert(self.x_vals)
        for i in range(len(self.x_vals) - 1):
            if deriverte[i] * deriverte[i+1] < 0:
                try:
                    rot = brentq(self.derivert, self.x_vals[i], self.x_vals[i+1])
                    punkter.append((rot, self.evaluer(rot)))
                except (ValueError, RuntimeError):
                    continue
        return punkter

    def los_for_y(self, y_target):
        # Definer en hjelpefunksjon for brentq
        ligning = lambda x: self.evaluer(x) - y_target
        # Søk etter løsning i hele intervallet
        try:
            # Vi må finne et subintervall hvor løsningen er bracketed
            for i in range(len(self.x_vals) - 1):
                if (ligning(self.x_vals[i]) * ligning(self.x_vals[i+1])) < 0:
                    return brentq(ligning, self.x_vals[i], self.x_vals[i+1])
            # Hvis ingen løsning er funnet etter løkken
            raise ValueError
        except (ValueError, RuntimeError):
            # Returner None hvis ingen løsning finnes
            return None

# --- Grafisk fremstilling ---

def tegn_graf(analyse: FunksjonsAnalyse, tittel, x_label, y_label, resultater):
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(12, 6))

    # Plot funksjonen
    plt.plot(analyse.x_vals, analyse.y_vals, label=f"f(x) = {analyse.funksjon_str.replace('**', '^')}", color='royalblue', lw=2)

    # Plot nullpunkter
    nullpunkter = analyse.finn_nullpunkter()
    if nullpunkter:
        plt.plot(nullpunkter, [0]*len(nullpunkter), 'o', color='crimson', markersize=8, label='Nullpunkt')
        for npunkt in nullpunkter:
            resultater.append(f"🔵 Nullpunkt funnet ved <b>x ≈ {npunkt:.3f}</b>")

    # Plot ekstremalpunkter
    ekstremalpunkter = analyse.finn_ekstremalpunkter()
    if ekstremalpunkter:
        x_ekstrem, y_ekstrem = zip(*ekstremalpunkter)
        plt.plot(x_ekstrem, y_ekstrem, 'o', color='forestgreen', markersize=8, label='Ekstremalpunkt')
        for x, y in ekstremalpunkter:
            resultater.append(f"🟢 Ekstremalpunkt funnet ved <b>(x, y) ≈ ({x:.3f}, {y:.3f})</b>")

    plt.title(tittel, fontsize=16)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.legend(loc="best", fontsize=10)
    plt.axhline(0, color='black', linewidth=0.5)
    plt.show()

# --- Hovedfunksjon og GUI ---

# Output-widget for resultater
out = widgets.Output()

def kjør_analyse_knapp_klikk(_=None):
    with out:
        clear_output(wait=True)
        
        try:
            funksjon_str = funksjon_input.value
            x_min = float(x_min_input.value)
            x_max = float(x_max_input.value)
            if x_min >= x_max:
                display(widgets.HTML("<b style='color:red;'>⚠️ 'x min' må være mindre enn 'x max'.</b>"))
                return
        except (ValueError, TypeError):
            display(widgets.HTML("<b style='color:red;'>⚠️ Ugyldig funksjon eller intervall. Sjekk input.</b>"))
            return

        # Opprett analyseobjekt
        analyse = FunksjonsAnalyse(funksjon_str, x_min, x_max)
        
        # Sjekk om funksjonen kunne evalueres i det hele tatt
        if np.isnan(analyse.y_vals).all():
            display(widgets.HTML(f"<b style='color:red;'>⚠️ Kunne ikke evaluere funksjonen '{funksjon_str}'. Sjekk syntaks (f.eks. bruk '*' for multiplikasjon).</b>"))
            return

        resultater = []
        
        # --- Utfør valgte analyser ---
        
        # Finn y fra x
        if y_fra_x_checkbox.value:
            x = float(x_verdi_input.value)
            y = analyse.evaluer(x)
            resultater.append(f"🔎 Funksjonsverdi: f({x}) = <b>{y:.3f}</b>")

        # Finn x fra y
        if x_fra_y_checkbox.value:
            y = float(y_verdi_input.value)
            x = analyse.los_for_y(y)
            if x is not None:
                resultater.append(f"🔁 Løsning for f(x) = {y}: <b>x ≈ {x:.3f}</b>")
            else:
                resultater.append(f"🔁 <span style='color:orange;'>f(x) = {y} har ingen løsning i intervallet [{analyse.x_min}, {analyse.x_max}].</span>")

        # Tangentstigning
        if tangent_checkbox.value:
            x = float(tangent_x_input.value)
            stigning = analyse.derivert(x)
            resultater.append(f"📐 Momentan vekstfart (stigning til tangent) ved x = {x}: <b>{stigning:.3f}</b>")

        # Gjennomsnittlig vekstfart
        if stigning_checkbox.value:
            x1, x2 = float(stigning_x1_input.value), float(stigning_x2_input.value)
            y1, y2 = analyse.evaluer(x1), analyse.evaluer(x2)
            if x1 != x2:
                stigning = (y2 - y1) / (x2 - x1)
                resultater.append(f"📊 Gjennomsnittlig vekstfart fra x={x1} til x={x2}: <b>{stigning:.3f}</b>")

        # Prosentvis endring
        if prosent_checkbox.value:
            x1, x2 = float(prosent_x1_input.value), float(prosent_x2_input.value)
            y1, y2 = analyse.evaluer(x1), analyse.evaluer(x2)
            if y1 != 0:
                prosent = ((y2 - y1) / y1) * 100
                resultater.append(f"📈 Prosentvis endring fra x={x1} til x={x2}: <b>{prosent:.2f}%</b>")
            else:
                resultater.append("📈 <span style='color:orange;'>Kan ikke beregne prosentvis endring fra y=0.</span>")

        # --- Vis graf og resultater ---
        display(widgets.HTML("<h2>Resultater</h2>"))
        tegn_graf(analyse, tittel_input.value, x_akse_input.value, y_akse_input.value, resultater)
        
        if resultater:
            html_resultat = "<ul>" + "".join([f"<li>{r}</li>" for r in sorted(resultater)]) + "</ul>"
            display(widgets.HTML(f"<h3>Analyseoppsummering:</h3>{html_resultat}"))
        else:
            display(widgets.HTML("<p><i>Ingen analyser ble valgt. Grafen viser null- og ekstremalpunkter som standard.</i></p>"))


# --- Bygg brukergrensesnitt (GUI) med ipywidgets ---

# Grunninnstillinger
funksjon_input = widgets.Text(value="45 * x * exp(-0.2*x)", description="f(x) =")
x_min_input = widgets.FloatText(value=0, description="x min:")
x_max_input = widgets.FloatText(value=30, description="x max:")
tittel_input = widgets.Text(value="Analyse av funksjon", description="Tittel:")
x_akse_input = widgets.Text(value="x (tid, enheter etc.)", description="x-akse:")
y_akse_input = widgets.Text(value="f(x) (mengde, verdi etc.)", description="y-akse:")

# Seksjon 1: Finn verdier
y_fra_x_checkbox = widgets.Checkbox(value=False, description="Finn y-verdi for en x-verdi")
x_verdi_input = widgets.FloatText(value=10, layout={'visibility': 'hidden'})
x_fra_y_checkbox = widgets.Checkbox(value=False, description="Finn x-verdi for en y-verdi (løs ligning)")
y_verdi_input = widgets.FloatText(value=50, layout={'visibility': 'hidden'})

# Seksjon 2: Vekstfart
tangent_checkbox = widgets.Checkbox(value=False, description="Finn momentan vekstfart (tangent)")
tangent_x_input = widgets.FloatText(value=5, layout={'visibility': 'hidden'})
stigning_checkbox = widgets.Checkbox(value=False, description="Finn gjennomsnittlig vekstfart")
stigning_x1_input = widgets.FloatText(value=0, layout={'visibility': 'hidden'})
stigning_x2_input = widgets.FloatText(value=10, layout={'visibility': 'hidden'})

# Seksjon 3: Endring
prosent_checkbox = widgets.Checkbox(value=False, description="Finn prosentvis endring")
prosent_x1_input = widgets.FloatText(value=0, layout={'visibility': 'hidden'})
prosent_x2_input = widgets.FloatText(value=10, layout={'visibility': 'hidden'})

# Funksjonalitet for å vise/skjule input-felt
def toggle_visibility(checkbox, *inputs):
    def on_change(change):
        visibility = 'visible' if change.new else 'hidden'
        for item in inputs:
            item.layout.visibility = visibility
    checkbox.observe(on_change, names='value')

toggle_visibility(y_fra_x_checkbox, x_verdi_input)
toggle_visibility(x_fra_y_checkbox, y_verdi_input)
toggle_visibility(tangent_checkbox, tangent_x_input)
toggle_visibility(stigning_checkbox, stigning_x1_input, stigning_x2_input)
toggle_visibility(prosent_checkbox, prosent_x1_input, prosent_x2_input)

# Knapp for å kjøre analysen
knapp = widgets.Button(description="Kjør analyse", button_style='success', icon='cogs')
knapp.on_click(kjør_analyse_knapp_klikk)

# Organiser GUI-elementer
grunninnstillinger = widgets.VBox([
    widgets.HTML("<h3>1. Angi funksjon og grafegenskaper</h3>"),
    funksjon_input,
    widgets.HBox([x_min_input, x_max_input]),
    widgets.HBox([tittel_input, x_akse_input, y_akse_input])
])

analysevalg = widgets.Accordion(
    children=[
        widgets.VBox([
            widgets.HBox([y_fra_x_checkbox, x_verdi_input]),
            widgets.HBox([x_fra_y_checkbox, y_verdi_input])
        ]),
        widgets.VBox([
            widgets.HBox([tangent_checkbox, tangent_x_input]),
            widgets.HBox([stigning_checkbox, stigning_x1_input, stigning_x2_input])
        ]),
        widgets.VBox([
            widgets.HBox([prosent_checkbox, prosent_x1_input, prosent_x2_input])
        ])
    ]
)
analysevalg.set_title(0, 'Finn verdier (x og y)')
analysevalg.set_title(1, 'Vekstfart (momentan og gjennomsnittlig)')
analysevalg.set_title(2, 'Prosentvis endring')

# Vis hele grensesnittet
display(widgets.VBox([
    widgets.HTML("<h1>Funksjonskalkulator for 2P-Y</h1>"),
    grunninnstillinger,
    widgets.HTML("<h3>2. Velg analyser som skal utføres</h3>"),
    analysevalg,
    knapp,
    out
]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brentq
import ipywidgets as widgets
from IPython.display import display, clear_output
import re

# --- Kjernefunksjoner for analyse i en klasse ---

class FunksjonsAnalyse:
    """En klasse for å holde på funksjonen og utføre alle relevante beregninger."""
    def __init__(self, funksjon_str, x_min, x_max):
        self.funksjon_str_original = funksjon_str
        self.funksjon_str = funksjon_str.replace("^", "**").replace("ln", "np.log")
        self.x_min = x_min
        self.x_max = x_max
        self.allowed_names = {
            "np": np, "sqrt": np.sqrt, "abs": np.abs, "sin": np.sin,
            "cos": np.cos, "tan": np.tan, "exp": np.exp, "log": np.log,
            "pi": np.pi, "e": np.e, "x": None
        }
        self.x_vals = np.linspace(x_min, x_max, 500)
        self.y_vals = self.evaluer(self.x_vals)

    def evaluer(self, x_verdi):
        self.allowed_names["x"] = x_verdi
        try:
            return eval(self.funksjon_str, {"__builtins__": {}}, self.allowed_names)
        except Exception:
            return np.full_like(x_verdi, np.nan) if isinstance(x_verdi, np.ndarray) else np.nan

    def derivert(self, x, dx=1e-6):
        return (self.evaluer(x + dx) - self.evaluer(x - dx)) / (2 * dx)

    def finn_nullpunkter(self):
        punkter = []
        for i in range(len(self.x_vals) - 1):
            # Sjekk for tegnskifte og at verdiene er gyldige
            if not np.isnan(self.y_vals[i]) and not np.isnan(self.y_vals[i+1]) and self.y_vals[i] * self.y_vals[i+1] < 0:
                try:
                    rot = brentq(self.evaluer, self.x_vals[i], self.x_vals[i+1])
                    punkter.append(rot)
                except (ValueError, RuntimeError):
                    continue
        return punkter

    def finn_ekstremalpunkter(self):
        punkter = []
        deriverte = self.derivert(self.x_vals)
        for i in range(len(self.x_vals) - 1):
            if not np.isnan(deriverte[i]) and not np.isnan(deriverte[i+1]) and deriverte[i] * deriverte[i+1] < 0:
                try:
                    rot = brentq(self.derivert, self.x_vals[i], self.x_vals[i+1])
                    if self.x_min <= rot <= self.x_max:
                        punkter.append((rot, self.evaluer(rot)))
                except (ValueError, RuntimeError):
                    continue
        return punkter

    def los_for_y(self, y_target):
        ligning = lambda x: self.evaluer(x) - y_target
        losninger = []
        for i in range(len(self.x_vals) - 1):
            if not np.isnan(ligning(self.x_vals[i])) and not np.isnan(ligning(self.x_vals[i+1])) and ligning(self.x_vals[i]) * ligning(self.x_vals[i+1]) < 0:
                try:
                    rot = brentq(ligning, self.x_vals[i], self.x_vals[i+1])
                    losninger.append(rot)
                except (ValueError, RuntimeError):
                    continue
        return losninger

# --- Grafisk fremstilling ---

def tegn_graf(analyse: FunksjonsAnalyse, tittel, x_label, y_label, resultater):
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(12, 7))

    plt.plot(analyse.x_vals, analyse.y_vals, label=f"f(x) = {analyse.funksjon_str_original}", color='royalblue', lw=2)

    nullpunkter = analyse.finn_nullpunkter()
    if nullpunkter:
        plt.plot(nullpunkter, [0]*len(nullpunkter), 'o', color='crimson', markersize=8, label='Nullpunkt')
        for npunkt in nullpunkter:
            resultater.append(f"🔵 Nullpunkt funnet ved <b>x ≈ {npunkt:.3f}</b>")

    ekstremalpunkter = analyse.finn_ekstremalpunkter()
    if ekstremalpunkter:
        x_ekstrem, y_ekstrem = zip(*ekstremalpunkter)
        plt.plot(x_ekstrem, y_ekstrem, 'o', color='forestgreen', markersize=8, label='Ekstremalpunkt')
        for x, y in ekstremalpunkter:
            resultater.append(f"🟢 Ekstremalpunkt funnet ved <b>(x, y) ≈ ({x:.3f}, {y:.3f})</b>")

    plt.title(tittel, fontsize=16)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.legend(loc="best", fontsize=10)
    plt.axhline(0, color='black', linewidth=0.5)
    plt.xlim(analyse.x_min, analyse.x_max)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

# --- Hovedfunksjon og GUI ---

out = widgets.Output()

def kjør_analyse_knapp_klikk(_=None):
    with out:
        clear_output(wait=True)
        
        try:
            funksjon_str = funksjon_input.value
            x_min = float(x_min_input.value)
            x_max = float(x_max_input.value)
            if x_min >= x_max:
                display(widgets.HTML("<b style='color:red;'>⚠️ 'x min' må være mindre enn 'x max'.</b>"))
                return
        except (ValueError, TypeError):
            display(widgets.HTML("<b style='color:red;'>⚠️ Ugyldig funksjon eller intervall. Sjekk input.</b>"))
            return

        analyse = FunksjonsAnalyse(funksjon_str, x_min, x_max)
        
        if np.isnan(analyse.y_vals).all():
            display(widgets.HTML(f"<b style='color:red;'>⚠️ Kunne ikke evaluere funksjonen '{funksjon_str}'. Sjekk syntaks (f.eks. bruk '*' for multiplikasjon, '.' for desimaltall).</b>"))
            return

        resultater = []
        
        # --- Utfør valgte analyser ---
        try:
            if vekst_checkbox.value:
                y0, y1 = analyse.evaluer(0), analyse.evaluer(1)
                if y0 is not None and y0 != 0:
                    vekstfaktor = y1 / y0
                    prosent = (vekstfaktor - 1) * 100
                    resultater.append(f"🌿 Effektiv vekstfaktor fra x=0 til x=1: <b>{vekstfaktor:.3f}</b> (tilsvarer <b>{prosent:.2f}%</b> endring)")
                
                match = re.search(r'([\d\.]+)\s*\*\s*([\d\.]+)\s*\^+\s*x', funksjon_str.replace(' ', ''))
                if match:
                    r = float(match.group(2))
                    ekte_prosent = (r - 1) * 100
                    resultater.append(f"🌱 Teoretisk vekstfaktor fra funksjonsuttrykk: <b>{r:.3f}</b> (tilsvarer <b>{ekte_prosent:.2f}%</b> per enhet x)")

            if y_fra_x_checkbox.value:
                x = float(x_verdi_input.value)
                y = analyse.evaluer(x)
                resultater.append(f"🔎 Funksjonsverdi: f({x}) = <b>{y:.3f}</b>")

            if x_fra_y_checkbox.value:
                y = float(y_verdi_input.value)
                losninger = analyse.los_for_y(y)
                if losninger:
                    los_str = ", ".join([f"<b>x ≈ {x:.3f}</b>" for x in losninger])
                    resultater.append(f"🎯 Løsning(er) for f(x) = {y}: {los_str}")
                else:
                    resultater.append(f"⚠️ f(x) = {y} har ingen løsning i intervallet.")

            if tangent_checkbox.value:
                x = float(tangent_x_input.value)
                stigning = analyse.derivert(x)
                resultater.append(f"📐 Momentan vekstfart (tangent) ved x = {x}: <b>{stigning:.3f}</b>")

            if stigning_checkbox.value:
                x1, x2 = float(stigning_x1_input.value), float(stigning_x2_input.value)
                if x1 != x2:
                    y1, y2 = analyse.evaluer(x1), analyse.evaluer(x2)
                    stigning = (y2 - y1) / (x2 - x1)
                    resultater.append(f"📊 Gjennomsnittlig vekstfart fra x={x1} til x={x2}: <b>{stigning:.3f}</b>")

            if diff_checkbox.value:
                x1, x2 = float(diff_x1_input.value), float(diff_x2_input.value)
                y1, y2 = analyse.evaluer(x1), analyse.evaluer(x2)
                resultater.append(f"➖ Forskjell i funksjonsverdi: f({x2}) - f({x1}) = {y2:.2f} - {y1:.2f} = <b>{y2 - y1:.2f}</b>")

            if prosent_checkbox.value:
                x1, x2 = float(prosent_x1_input.value), float(prosent_x2_input.value)
                y1, y2 = analyse.evaluer(x1), analyse.evaluer(x2)
                if y1 != 0:
                    prosent = ((y2 - y1) / abs(y1)) * 100
                    resultater.append(f"📈 Prosentvis endring fra x={x1} til x={x2}: <b>{prosent:.2f}%</b>")

            if prosent_okning_checkbox.value:
                prosent = float(prosent_okning_input.value)
                y0 = analyse.evaluer(x_min)
                y_target = y0 * (1 + prosent / 100)
                losninger = analyse.los_for_y(y_target)
                if losninger:
                    resultater.append(f"📈 {prosent}% økning fra f({x_min}) nås ved <b>x ≈ {losninger[0]:.3f}</b>")
            
            if prosent_reduksjon_checkbox.value:
                prosent = float(prosent_reduksjon_input.value)
                y0 = analyse.evaluer(x_min)
                y_target = y0 * (1 - prosent / 100)
                losninger = analyse.los_for_y(y_target)
                if losninger:
                    resultater.append(f"📉 {prosent}% reduksjon fra f({x_min}) nås ved <b>x ≈ {losninger[0]:.3f}</b>")

        except Exception as e:
            resultater.append(f"<b style='color:red;'>⚠️ En analysefeil oppstod: {e}</b>")

        # --- Vis graf og resultater ---
        display(widgets.HTML("<h2>Resultater</h2>"))
        tegn_graf(analyse, tittel_input.value, x_akse_input.value, y_akse_input.value, resultater)
        
        if resultater:
            html_resultat = "<ul>" + "".join([f"<li style='margin-bottom: 5px;'>{r}</li>" for r in sorted(resultater)]) + "</ul>"
            display(widgets.HTML(f"<h3>Analyseoppsummering:</h3>{html_resultat}"))
        else:
            display(widgets.HTML("<p><i>Ingen analyser ble valgt. Grafen viser null- og ekstremalpunkter som standard.</i></p>"))


# --- Bygg brukergrensesnitt (GUI) med ipywidgets ---

# Grunninnstillinger
funksjon_input = widgets.Text(value="6000 * 0.958^x", description="f(x) = ", layout={'width': '400px'})
x_min_input = widgets.FloatText(value=0, description="x min:")
x_max_input = widgets.FloatText(value=30, description="x max:")
tittel_input = widgets.Text(value="Populasjonsmodell for en dyreart", description="Tittel:")
x_akse_input = widgets.Text(value="Antall år", description="x-akse:")
y_akse_input = widgets.Text(value="Antall dyr", description="y-akse:")

# Seksjon 1: Verdier
y_fra_x_checkbox = widgets.Checkbox(value=False, description="Finn y for en gitt x")
x_verdi_input = widgets.FloatText(value=10, layout={'visibility': 'hidden'})
x_fra_y_checkbox = widgets.Checkbox(value=True, description="Finn x for en gitt y (terskel)")
y_verdi_input = widgets.FloatText(value=3000, layout={'visibility': 'visible'})

# Seksjon 2: Vekst og endring
vekst_checkbox = widgets.Checkbox(value=True, description="Analyser vekstfaktor (for eksponentialfunksjoner)")
tangent_checkbox = widgets.Checkbox(value=False, description="Finn momentan vekstfart (tangent)")
tangent_x_input = widgets.FloatText(value=5, layout={'visibility': 'hidden'})
stigning_checkbox = widgets.Checkbox(value=False, description="Finn gjennomsnittlig vekstfart")
stigning_x1_input = widgets.FloatText(value=0, layout={'visibility': 'hidden'})
stigning_x2_input = widgets.FloatText(value=10, layout={'visibility': 'hidden'})

# Seksjon 3: Forskjell og prosent
diff_checkbox = widgets.Checkbox(value=False, description="Finn forskjell mellom to y-verdier")
diff_x1_input = widgets.FloatText(value=0, layout={'visibility': 'hidden'})
diff_x2_input = widgets.FloatText(value=10, layout={'visibility': 'hidden'})
prosent_checkbox = widgets.Checkbox(value=False, description="Finn prosentvis endring")
prosent_x1_input = widgets.FloatText(value=0, layout={'visibility': 'hidden'})
prosent_x2_input = widgets.FloatText(value=10, layout={'visibility': 'hidden'})

# Seksjon 4: Prosentmål
prosent_okning_checkbox = widgets.Checkbox(value=False, description="Når har verdien økt med X %?")
prosent_okning_input = widgets.FloatText(value=20, layout={'visibility': 'hidden'})
prosent_reduksjon_checkbox = widgets.Checkbox(value=False, description="Når har verdien sunket med X %?")
prosent_reduksjon_input = widgets.FloatText(value=50, layout={'visibility': 'hidden'})

# Funksjonalitet for å vise/skjule input-felt
def toggle_visibility(checkbox, *inputs):
    def on_change(change):
        visibility = 'visible' if change.new else 'hidden'
        for item in inputs:
            item.layout.visibility = visibility
    checkbox.observe(on_change, names='value')

toggle_visibility(y_fra_x_checkbox, x_verdi_input)
toggle_visibility(x_fra_y_checkbox, y_verdi_input)
toggle_visibility(tangent_checkbox, tangent_x_input)
toggle_visibility(stigning_checkbox, stigning_x1_input, stigning_x2_input)
toggle_visibility(diff_checkbox, diff_x1_input, diff_x2_input)
toggle_visibility(prosent_checkbox, prosent_x1_input, prosent_x2_input)
toggle_visibility(prosent_okning_checkbox, prosent_okning_input)
toggle_visibility(prosent_reduksjon_checkbox, prosent_reduksjon_input)

# Knapp for å kjøre analysen
knapp = widgets.Button(description="Kjør analyse", button_style='success', icon='calculator', layout={'width': '98%'})
knapp.on_click(kjør_analyse_knapp_klikk)

# Organiser GUI-elementer
grunninnstillinger = widgets.VBox([
    widgets.HTML("<h3>1. Angi funksjon og grafegenskaper</h3>"),
    funksjon_input,
    widgets.HBox([x_min_input, x_max_input]),
    widgets.HBox([tittel_input, x_akse_input, y_akse_input])
])

analysevalg = widgets.Accordion(
    children=[
        widgets.VBox([
            widgets.HBox([y_fra_x_checkbox, x_verdi_input]),
            widgets.HBox([x_fra_y_checkbox, y_verdi_input]),
        ]),
        widgets.VBox([
            vekst_checkbox,
            widgets.HBox([tangent_checkbox, tangent_x_input]),
            widgets.HBox([stigning_checkbox, stigning_x1_input, widgets.Label("til"), stigning_x2_input]),
        ]),
        widgets.VBox([
            widgets.HBox([diff_checkbox, diff_x1_input, widgets.Label("og"), diff_x2_input]),
            widgets.HBox([prosent_checkbox, prosent_x1_input, widgets.Label("til"), prosent_x2_input]),
        ]),
        widgets.VBox([
             widgets.HBox([prosent_okning_checkbox, prosent_okning_input, widgets.Label("%")]),
             widgets.HBox([prosent_reduksjon_checkbox, prosent_reduksjon_input, widgets.Label("%")]),
        ])
    ]
)
analysevalg.set_title(0, 'Finn verdier (x og y)')
analysevalg.set_title(1, 'Vekstfart og vekstfaktor')
analysevalg.set_title(2, 'Forskjell og prosentvis endring')
analysevalg.set_title(3, 'Prosentmål fra startverdi')

# Vis hele grensesnittet
display(widgets.VBox([
    widgets.HTML("<h1>⚡ Funksjonsanalyse 3.0 (Bjarne-kalkulator) ⚡</h1>"),
    widgets.HTML("<p>Skriv inn en funksjon, velg analyser og trykk på knappen. Bruk <b>*</b> for multiplikasjon, <b>/</b> for divisjon og <b>^</b> for potens.</p>"),
    grunninnstillinger,
    widgets.HTML("<h3>2. Velg analyser som skal utføres</h3>"),
    analysevalg,
    widgets.HTML("<br>"),
    knapp,
    out
]))

# $\color{hotpink}{\text{Masse ulike grafer + glidere for grafer}}$

In [ ]:
# Kvadratroten av x : y = sqrt(x)
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

# Definer grafefunksjonen
def f(a, b, c, zoom):
    xmin = -zoom
    xmax = zoom
    ymin = -zoom
    ymax = zoom
    points = 1000  # Økt antall punkter for jevnere graf
    x = np.linspace(xmin, xmax, points)
    
    # Filtrer ut negative verdier for kvadratroten
    x = x[x >= b]

    plt.figure(figsize=(8, 8))  # Større figur for bedre visning
    plt.axis([xmin, xmax, ymin, ymax])  # Vindu størrelse
    plt.axhline(0, color='black')  # Svart x-akse
    plt.axvline(0, color='black')  # Svart y-akse
    
    # Linje 1
    y1 = a * np.sqrt(x - b) + c
    plt.plot(x, y1, label=f'y = {a}√(x - {b}) + {c}')
    plt.legend()
    plt.show()

# Sett opp sliderne
interactive_plot = interactive(f, a=(-9, 9, 0.1), b=(-9, 9, 0.1), c=(-9, 9, 0.1), zoom=(1, 100, 1))
interactive_plot

In [ ]:
#Her er pythonprogrammet som simulerer eksperimentet med terningkast i Utforsk halveringstid på s.215. 
from random import randint

antallTerninger = 2000
antallKast = 0

while antallTerninger > 0:
  for i in range(antallTerninger):
    terning = randint(1, 6)
    if terning == 6:
      antallTerninger -= 1
  antallKast += 1
  print("Etter", antallKast, "kast er det", antallTerninger, "terninger igjen.")

In [ ]:
# sliders and zoom
# y = x**4 + ...

%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

# Define the graphing function
def f(a,b,c,d,e,zoom):
    xmin = -zoom
    xmax = zoom
    ymin = -zoom
    ymax = zoom
    points = 10*xmax-xmin
    x = np.linspace(xmin, xmax, points)

    plt.axis([xmin,xmax,ymin,ymax]) # window size
    plt.plot([xmin,xmax],[0,0],'black') # black x axis
    plt.plot([0,0],[ymin,ymax], 'black') # black y axis
    
    # Line 1
    y1 = a*x**4 + b*x**3 + c*x**2 + d*x + e
    plt.plot(x, y1)
    plt.show()

# Set up the sliders
i_plot=interactive(f,a=(-9,9),b=(-9,9),c=(-9,9),d=(-9,9),e=(-9,9),zoom=(1,100))
i_plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(xmin,xmax,points) 


fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot([xmin,xmax],[5,5],'r')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(xmin,xmax,points) 

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot(x, x) 
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(xmin,xmax,points) 

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot(x, x**2) 
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(xmin,xmax,points) 

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

# plt.plot(x, x**3) 
plt.plot(x, x**3 - 2*x**2 - 3*x + 4)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(xmin,xmax,points) 

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot(x, x**4)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(xmin,xmax,points) 

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot(x, x**5)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(0,xmax,points) #

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot(x, np.sqrt(x)) 
# plt.plot(x, -np.sqrt(x))

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(0,xmax,points) 

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot(x, x**(1/2))

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


xmin = -10
xmax = 10
ymin = - 10
ymax = 10
points = 4*(xmax-xmin)
x = np.linspace(xmin,xmax,points) #

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot(x, np.cbrt(x))


plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


xmin = -10
xmax = 10
ymin = -10
ymax = 10
points = 3*(xmax-xmin)
x = np.linspace(xmin,xmax,points) #


fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

plt.plot(x, 2**x)


plt.show()

In [ ]:
# Ulikheter
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = -10
ymax = 10
points = 2*(xmax-xmin)
x = np.linspace(xmin,xmax,points)

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

# line 1
y1 = x+6
plt.plot(x, y1,'-')
plt.fill_between(x, y1, ymax, facecolor='red')

# line 2
y2 = x+3
plt.plot(x, y2,'-')
plt.fill_between(x, y2, y1, facecolor='yellow')

# line 3
y3 = x-1
plt.plot(x, y3)
plt.fill_between(x, y3, y2, facecolor='green')

# line 4
y4 = x-4
plt.plot(x, y4)
plt.fill_between(x, y4, y3, facecolor='blue')

plt.show()

In [ ]:
# Bruk NUMPY til å lage grafer :)
import matplotlib.pyplot as plt
import numpy as np

xmin = -10
xmax = 10
ymin = -10
ymax = 10

# Define how many points to plot 
points = 10*(xmax-xmin)

# Define the array of x values once
x = np.linspace(xmin,xmax,points)

fig, ax = plt.subplots()
plt.axis([xmin,xmax,ymin,ymax]) # window size
plt.plot([xmin,xmax],[0,0],'b') # blue x axis
plt.plot([0,0],[ymin,ymax], 'b') # blue y axis

# line 1
y1 = 3*x
plt.plot(x, y1) 

# line 2
y2 = x**3
plt.plot(x, y2) 

ax.grid(True)
plt.show()

# $\color{blue}{\text{Snake spill :)}}$

In [ ]:
import pygame
import sys
import random
import json
from enum import Enum

# --- Konstanter ---
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
BLOCK_SIZE = 20
COLOR_BACKGROUND = (173, 216, 230)
COLOR_GRID = (153, 196, 210)
COLOR_SNAKE_HEAD = (0, 100, 0)
COLOR_SNAKE_BODY = (0, 128, 0)
COLOR_FOOD = (213, 50, 80)
COLOR_TEXT = (255, 255, 255)
COLOR_TITLE = (255, 255, 102)
COLOR_OVERLAY = (0, 0, 0, 150)
INITIAL_SNAKE_SPEED = 10
SPEED_INCREASE_INTERVAL = 5
SPEED_INCREMENT = 1
HIGHSCORE_FILE = "highscore.json"

class GameState(Enum):
    MAIN_MENU = 1
    PLAYING = 2
    PAUSED = 3
    GAME_OVER = 4

class Game:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption('Avansert Snake')
        self.clock = pygame.time.Clock()
        self.font_title = pygame.font.SysFont('Consolas', 70, bold=True)
        self.font_main = pygame.font.SysFont('Consolas', 30)
        self.font_score = pygame.font.SysFont('Consolas', 25)
        self.high_score = self.load_highscore()
        self.game_state = GameState.MAIN_MENU
        self.snake_speed = INITIAL_SNAKE_SPEED
        self.running = True  # Kontrollerer hovedløkken

    def load_highscore(self):
        try:
            with open(HIGHSCORE_FILE, 'r') as f:
                return json.load(f)['high_score']
        except (FileNotFoundError, json.JSONDecodeError):
            return 0

    def save_highscore(self):
        with open(HIGHSCORE_FILE, 'w') as f:
            json.dump({'high_score': self.high_score}, f)

    def reset_game(self):
        self.snake_pos = [SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2]
        self.snake_body = [list(self.snake_pos)]
        self.food_pos = self.spawn_food()
        self.direction = 'RIGHT'
        self.change_to = self.direction
        self.score = 0
        self.snake_speed = INITIAL_SNAKE_SPEED
        self.game_state = GameState.PLAYING

    def spawn_food(self):
        while True:
            pos = [random.randrange(0, SCREEN_WIDTH // BLOCK_SIZE) * BLOCK_SIZE,
                   random.randrange(0, SCREEN_HEIGHT // BLOCK_SIZE) * BLOCK_SIZE]
            if pos not in self.snake_body:
                return pos

    def draw_text(self, text, font, color, x, y, center=True):
        text_surface = font.render(text, True, color)
        text_rect = text_surface.get_rect()
        if center: text_rect.center = (x, y)
        else: text_rect.topleft = (x, y)
        self.screen.blit(text_surface, text_rect)

    def draw_grid(self):
        for x in range(0, SCREEN_WIDTH, BLOCK_SIZE):
            pygame.draw.line(self.screen, COLOR_GRID, (x, 0), (x, SCREEN_HEIGHT))
        for y in range(0, SCREEN_HEIGHT, BLOCK_SIZE):
            pygame.draw.line(self.screen, COLOR_GRID, (0, y), (SCREEN_WIDTH, y))
    
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False # Avslutter hovedløkken elegant
            
            if event.type == pygame.KEYDOWN:
                if self.game_state == GameState.MAIN_MENU:
                    if event.key == pygame.K_p: self.reset_game()
                    elif event.key == pygame.K_q: self.running = False

                elif self.game_state == GameState.PLAYING:
                    if (event.key == pygame.K_UP or event.key == pygame.K_w) and self.direction != 'DOWN': self.change_to = 'UP'
                    elif (event.key == pygame.K_DOWN or event.key == pygame.K_s) and self.direction != 'UP': self.change_to = 'DOWN'
                    elif (event.key == pygame.K_LEFT or event.key == pygame.K_a) and self.direction != 'RIGHT': self.change_to = 'LEFT'
                    elif (event.key == pygame.K_RIGHT or event.key == pygame.K_d) and self.direction != 'LEFT': self.change_to = 'RIGHT'
                    elif event.key == pygame.K_ESCAPE or event.key == pygame.K_p: self.game_state = GameState.PAUSED
                
                elif self.game_state == GameState.PAUSED:
                    if event.key == pygame.K_ESCAPE or event.key == pygame.K_p: self.game_state = GameState.PLAYING
                
                elif self.game_state == GameState.GAME_OVER:
                    if event.key == pygame.K_c: self.reset_game()
                    elif event.key == pygame.K_q: self.game_state = GameState.MAIN_MENU

    def update_game_logic(self):
        if self.game_state != GameState.PLAYING: return
        self.direction = self.change_to
        if self.direction == 'UP': self.snake_pos[1] -= BLOCK_SIZE
        elif self.direction == 'DOWN': self.snake_pos[1] += BLOCK_SIZE
        elif self.direction == 'LEFT': self.snake_pos[0] -= BLOCK_SIZE
        elif self.direction == 'RIGHT': self.snake_pos[0] += BLOCK_SIZE

        self.snake_body.insert(0, list(self.snake_pos))
        if self.snake_pos == self.food_pos:
            self.score += 1
            self.food_pos = self.spawn_food()
            if self.score % SPEED_INCREASE_INTERVAL == 0: self.snake_speed += SPEED_INCREMENT
        else: self.snake_body.pop()

        if not (0 <= self.snake_pos[0] < SCREEN_WIDTH and 0 <= self.snake_pos[1] < SCREEN_HEIGHT): self.trigger_game_over()
        for block in self.snake_body[1:]:
            if self.snake_pos == block: self.trigger_game_over()

    def trigger_game_over(self):
        if self.score > self.high_score:
            self.high_score = self.score
            self.save_highscore()
        self.game_state = GameState.GAME_OVER

    def draw_elements(self):
        self.screen.fill(COLOR_BACKGROUND)
        self.draw_grid()

        if self.game_state == GameState.MAIN_MENU:
            self.draw_text("SNAKE", self.font_title, COLOR_TITLE, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
            self.draw_text(f"Rekord: {self.high_score}", self.font_main, COLOR_TEXT, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2)
            self.draw_text("Trykk [P] for å spille", self.font_main, COLOR_TEXT, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)
            self.draw_text("Trykk [Q] for å avslutte", self.font_main, COLOR_TEXT, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 100)

        elif self.game_state in [GameState.PLAYING, GameState.PAUSED, GameState.GAME_OVER]:
            pulse = abs(1.5 * (pygame.time.get_ticks() // 200 % 2) - 1.5)
            food_rect = pygame.Rect(self.food_pos[0], self.food_pos[1], BLOCK_SIZE, BLOCK_SIZE)
            pygame.draw.rect(self.screen, COLOR_FOOD, food_rect.inflate(pulse, pulse))
            for i, pos in enumerate(self.snake_body):
                color = COLOR_SNAKE_HEAD if i == 0 else COLOR_SNAKE_BODY
                pygame.draw.rect(self.screen, color, pygame.Rect(pos[0], pos[1], BLOCK_SIZE, BLOCK_SIZE))
            self.draw_text(f"Poeng: {self.score}", self.font_score, COLOR_TEXT, 80, 20)
            self.draw_text(f"Fart: {self.snake_speed}", self.font_score, COLOR_TEXT, 720, 20)

            if self.game_state == GameState.PAUSED or self.game_state == GameState.GAME_OVER:
                overlay = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT), pygame.SRCALPHA)
                overlay.fill(COLOR_OVERLAY)
                self.screen.blit(overlay, (0, 0))
                if self.game_state == GameState.PAUSED:
                    self.draw_text("PAUSE", self.font_title, COLOR_TITLE, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2)
                else:
                    self.draw_text("GAME OVER", self.font_title, COLOR_FOOD, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 3)
                    self.draw_text(f"Din poengsum: {self.score}", self.font_main, COLOR_TEXT, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2)
                    self.draw_text("Trykk [C] for å spille igjen", self.font_main, COLOR_TEXT, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)
                    self.draw_text("Trykk [Q] for å gå til menyen", self.font_main, COLOR_TEXT, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 100)

        pygame.display.flip()

    def run(self):
        """Hovedløkken som kjører hele spillet."""
        while self.running:
            self.handle_events()
            self.update_game_logic()
            self.draw_elements()
            self.clock.tick(self.snake_speed)

if __name__ == '__main__':
    game = Game()
    game.run()

    # Avslutter rent etter at hovedløkken er ferdig
    pygame.quit()
    # sys.exit() er fjernet for å unngå SystemExit i interaktive konsoller